In [ ]:
from agential.prompting.cot.prompting import CoT

from agential.prompting.cot.prompts import (
    COT_INSTRUCTION_AMBIGNQ, 
    COT_INSTRUCTION_FEVER, 
    COT_INSTRUCTION_GSM8K, 
    COT_INSTRUCTION_HOTPOTQA, 
    COT_INSTRUCTION_SVAMP, 
    COT_INSTRUCTION_TRIVIAQA,
    COT_INSTRUCTION_TABMWP,
    COT_INSTRUCTION_HUMANEVAL,
    COT_INSTRUCTION_MBPP,
)
from agential.core.fewshots.ambignq import AMBIGNQ_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.fever import FEVER_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.svamp import SVAMP_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.triviaqa import TRIVIAQA_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.tabmwp import TABMWP_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_COT

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo")

In [1]:
import json

# Open and read the JSON file
with open('../data/hotpot_dev_v1_simplified.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data[:5])

[{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'answer': 'yes', 'type': 'comparison'}, {'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', 'answer': 'Chief of Protocol', 'type': 'bridge'}, {'question': 'What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?', 'answer': 'Animorphs', 'type': 'bridge'}, {'question': 'Are the Laleli Mosque and Esma Sultan Mansion located in the same neighborhood?', 'answer': 'no', 'type': 'comparison'}, {'question': 'The director of the romantic comedy "Big Stone Gap" is based in what New York city?', 'answer': 'Greenwich Village, New York City', 'type': 'bridge'}]


In [2]:
len(data)

7405

In [4]:
Shuffle1 = [6417, 360, 3435, 1563, 3232, 2769, 3692, 2189, 2223, 3883, 3745, 2539, 6361, 6809, 5154, 4187, 349, 6013, 1292, 6210, 3647, 3033, 2105, 3041, 57, 5447, 4988, 7149, 6844, 630, 7195, 913, 591, 6388, 2613, 314, 3930, 5259, 3079, 5370, 2929, 4790, 5316, 655, 3440, 4838, 2761, 2824, 477, 7346, 2258, 169, 5092, 3543, 6331, 1480, 3601, 6100, 5669, 800, 6355, 3783, 855, 4406, 1356, 7102, 4911, 5104, 1822, 5706, 2276, 5428, 2952, 4484, 6655, 2771, 160, 4461, 5783, 1857, 4821, 1436, 216, 78, 3021, 3123, 2200, 4800, 5513, 6442, 5654, 1991, 4177, 5492, 1847, 6123, 4172, 1252, 775, 2898, 4723, 5898, 6088, 1966, 5767, 4624, 6692, 2401, 721, 993, 556, 6623, 1686, 2391, 5060, 5063, 2577, 2979, 653, 3315, 2940, 592, 1135, 5334, 4194, 982, 1832, 1376, 6399, 65, 3942, 3983, 437, 415, 539, 6284, 2284, 3182, 6670, 4230, 4128, 3650, 6403, 4456, 5954, 7383, 3369, 6124, 3995, 5351, 2297, 6749, 4228, 5343, 7358, 3689, 2368, 1378, 3387, 5621, 4274, 787, 1501, 627, 4900, 6591, 3718, 4896, 1434, 6204, 3491, 327, 6967, 1155, 921, 289, 2272, 656, 4422, 5239, 4942, 2453, 5997, 2299, 4466, 6815, 483, 745, 5837, 4063, 6919, 469, 3644, 3122, 3799, 238, 7370, 2080, 2758, 2496, 6343, 6842, 132, 2757, 4537, 255, 2403, 6167, 5137, 148, 940, 4309, 6764, 7013, 3212, 2656, 2531, 4317, 2398, 2617, 3887, 1742, 4439, 615, 4653, 3153, 1343, 692, 7157, 5611, 6732, 2987, 6599, 7021, 6016, 5250, 3814, 266, 4064, 4576, 5644, 3461, 408, 3751, 487, 2264, 7378, 6796, 5980, 3699, 1156, 6629, 478, 5762, 4398, 5878, 7190, 2583, 3501, 1733, 3429, 4746, 6162, 287, 6524, 1492, 515, 4314, 3003, 1956, 3859, 5444, 5970, 367, 4055, 3093, 4405, 206, 1409, 2552, 803, 6931, 6347, 6201, 6628, 89, 542, 876, 6107, 1750, 2159, 2239, 736, 4676, 3802, 5243, 2058, 3193, 4798, 2370, 1833, 4088, 2002, 857, 1629, 2440, 598, 5843, 1754, 2049, 5662, 2064, 3478, 3047, 5814, 1103, 5718, 1902, 1140, 6816, 1494, 5178, 4379, 4076, 5820, 999, 284, 4886, 1864, 4923, 172, 2715, 5220, 5831, 6489, 5290, 488, 209, 7371, 5530, 4796, 7101, 1585, 5497, 1774, 3959, 6030, 6847, 6367, 5821, 3276, 715, 3223, 3793, 1313, 6091, 6491, 298, 6964, 4186, 5572, 4122, 3089, 5683, 6562, 3259, 3074, 6214, 1084, 1206, 7328, 3921, 2447, 3729, 221, 3129, 643, 503, 1540, 5067, 5866, 6139, 5230, 4519, 480, 2030, 2878, 7226, 3460, 1173, 5001, 74, 3192, 1953, 2381, 3988, 3896, 4822, 6431, 3734, 5427, 1728, 5039, 4301, 5682, 1666, 6454, 1071, 4594, 4234, 2858, 3316, 2722, 5376, 6475, 4996, 4909, 2315, 5778, 5380, 6232, 867, 6281, 4536, 5359, 2703, 7074, 4771, 3726, 6488, 2630, 4618, 6470, 145, 1398, 561, 2924, 911, 2449, 3247, 6849, 695, 3306, 5592, 6181, 812, 28, 1091, 6869, 6570, 604, 4229, 6249, 2673, 3701, 2854, 4146, 7314, 6725, 2283, 2198, 3198, 846, 2065, 836, 1200, 1205, 5414, 1285, 1986, 3934, 6484, 1185, 5448, 5696, 4134, 4246, 99, 3672, 1615, 4459, 6344, 4617, 6306, 686, 252, 6530, 1685, 155, 4425, 1450, 3069, 6706, 6838, 338, 6187, 4344, 150, 2023, 5819, 1488, 5527, 6753, 6972, 4361, 1763, 4000, 3559, 86, 6760, 2830, 6573, 1369, 3541, 2892, 989, 4237, 1503, 5024, 5566, 6439, 7062, 2865, 6790, 4114, 2685, 4482, 1437, 2082, 4812, 6251, 1002, 5036, 4111, 1136, 4462, 1321, 4038, 4453, 1934, 6630, 6642, 1881, 1722, 3492, 4993, 1528, 7280, 133, 4117, 4053, 6593, 1707, 1510, 985, 1371, 4239, 326, 4096, 3558, 2343, 6422, 1572, 7375, 4714, 1316, 587, 2362, 2584, 4894, 3660, 4939, 3242, 6236, 5646, 2174, 4661, 4012, 6447, 3717, 699, 905, 4080, 1739, 6779, 1452, 3873, 1630, 383, 2966, 5498, 2517, 5476, 6410, 6547, 1587, 1143, 1283, 5386, 2366, 4336, 6808, 1214, 5727, 3059, 4787, 2219, 395, 1535, 608, 1621, 2853, 5047, 5462, 2571, 156, 3341, 6510, 6594, 6063, 6138, 2590, 4201, 4975, 7125, 6898, 6334, 5867, 2609, 5993, 4907, 1635, 1536, 1004, 610, 2839, 4469, 3255, 1698, 5131, 6121, 4994, 4986, 3665, 4522, 2035, 4813, 2207, 1192, 2306, 5780, 6999, 254, 6189, 6934, 5602, 1735, 4288, 3138, 1827, 6460, 912, 571, 4850, 724, 7353, 7228, 4131, 4295, 2385, 3515, 7291, 3175, 3567, 278, 2363, 5789, 5317, 746, 4263, 1804, 373, 30, 2521, 662, 1650, 4434, 6269, 688, 7030, 6061, 2936, 2324, 7330, 6998, 4720, 2266, 2690, 339, 6644, 6199, 7366, 4761, 1900, 877, 1344, 632, 2978, 3040, 4007, 2260, 2687, 1915, 4989, 6903, 6192, 7255, 1359, 1157, 69, 4495, 3968, 328, 113, 2893, 6116, 2482, 2116, 6730, 2905, 95, 1515, 2214, 865, 5405, 3006, 5856, 3393, 3055, 6340, 2445, 2417, 7333, 1971, 3609, 5785, 4513, 4409, 7287, 2826, 2487, 801, 364, 5436, 4949, 6486, 5835, 5870, 5768, 6104, 313, 6893, 2736, 3199, 4589, 421, 1769, 7193, 7267, 1047, 3334, 84, 5577, 3881, 6487, 1032, 4243, 6093, 6465, 946, 2750, 4998, 3725, 1242, 3512, 3598, 6787, 6207, 3146, 3197, 1595, 6095, 873, 1336, 521, 3897, 4332, 506, 5375, 4721, 1939, 2007, 2404, 906, 2650, 644, 3777, 3633, 7338, 1165, 6806, 6131, 2518, 4277, 3732, 6420, 1513, 5027, 5503, 4040, 2625, 5320, 4508, 4817, 5270, 2727, 4460, 2672, 2216, 1233, 3206, 4126, 1678, 3863, 1296, 3837, 3484, 4706, 7266, 5985, 1663, 4107, 957, 3361, 2477, 5209, 3372, 6017, 5688, 2870, 529, 2492, 1922, 879, 1121, 2423, 2365, 773, 3283, 7172, 2282, 584, 5673, 3409, 6763, 4238, 1542, 7121, 2301, 6513, 7318, 3900, 4166, 4930, 1172, 2338, 230, 6969, 4855, 3317, 6653, 4631, 4413, 6784, 5032, 2631, 191, 470, 2637, 7236, 6631, 165, 6206, 2177, 2923, 2651, 5149, 7362, 2676, 5010, 3708, 6352, 5906, 4717, 5748, 6607, 6793, 6358, 2130, 3507, 2286, 5276, 3786, 3895, 986, 6871, 3108, 3329, 5894, 232, 5887, 5678, 2087, 7034, 1706, 5421, 1284, 6765, 6421, 3991, 4511, 3781, 5179, 3181, 3398, 1448, 5803, 6994, 714, 6926, 4604, 1605, 1704, 5740, 3586, 1132, 1848, 6499, 2732, 6937, 567, 6727, 7098, 1614, 2844, 518, 2410, 303, 5251, 4457, 6866, 6246, 6191, 1239, 4070, 5125, 1147, 3513, 1894, 501, 4725, 5908, 5574, 6056, 3704, 2474, 6126, 6968, 2182, 2393, 4454, 895, 1015, 2599, 5983, 2754, 6659, 4603, 7080, 3753, 5796, 2816, 6134, 6737, 1419, 4346, 6948, 6945, 4895, 6106, 5416, 2137, 3911, 2777, 870, 2946, 1512, 6578, 4259, 1940, 4109, 7308, 4451, 646, 3779, 29, 6930, 2549, 3667, 195, 7057, 4323, 63, 1790, 772, 6733, 1126, 682, 4116, 5496, 3058, 4671, 3981, 720, 3149, 6135, 3017, 4448, 2027, 3417, 4654, 5782, 448, 3207, 322, 5863, 675, 949, 1603, 3565, 5431, 2813, 1745, 5181, 66, 7100, 5959, 4408, 1290, 1326, 3355, 2808, 6074, 215, 2598, 2724, 3761, 4104, 6997, 4745, 1537, 696, 62, 4226, 2126, 2158, 376, 7028, 968, 1489, 537, 5536, 6971, 6478, 6202, 6545, 5248, 1166, 3305, 3688, 6978, 2143, 256, 358, 7388, 4438, 2646, 6041, 2660, 3184, 5973, 2536, 6065, 3735, 1914, 3479, 1527, 6882, 1514, 1979, 2037, 6170, 7189, 6171, 4810, 5260, 5170, 831, 5907, 4069, 964, 797, 491, 212, 6459, 5984, 6616, 3001, 7147, 991, 3548, 4629, 19, 5633, 1309, 4334, 5065, 6006, 3436, 3712, 1043, 3722, 3605, 455, 2481, 1226, 5598, 5311, 4958, 6695, 5242, 6910, 2038, 3506, 7241, 6835, 4073, 4840, 2459, 4605, 7365, 2097, 4567, 7344, 7110, 2469, 1959, 102, 2567, 3205, 4572, 2986, 5813, 4178, 3096, 3084, 516, 229, 5857, 4856, 3254, 1016, 3449, 6951, 4060, 7381, 1312, 2711, 5694, 4140, 5630, 6110, 2699, 1472, 3354, 2205, 1869, 4743, 2379, 1366, 3423, 1112, 7115, 2457, 2354, 1149, 645, 2917, 937, 7297, 6932, 1364, 7319, 7132, 2682, 4587, 4296, 995, 5542, 1179, 7303, 1798, 1660, 4622, 5653, 2589, 307, 4728, 4470, 6620, 4429, 4534, 4739, 4503, 1526, 6080, 6744, 5321, 3709, 2377, 218, 7270, 1643, 6915, 2060, 2837, 4652, 3240, 3282, 5435, 863, 2407, 3359, 4251, 1468, 3111, 6974, 371, 3171, 5035, 1134, 4724, 276, 5294, 5342, 3495, 5252, 1695, 6086, 1304, 4854, 3979, 4544, 4933, 5921, 600, 4510, 3325, 3902, 2774, 4650, 4376, 6219, 4964, 1543, 61, 817, 4396, 2948, 7372, 3251, 4601, 6533, 3765, 1524, 2971, 7361, 2165, 5974, 2347, 4047, 5411, 6224, 4627, 994, 1529, 6778, 1012, 5912, 4427, 2829, 1810, 5052, 5635, 5531, 848, 91, 618, 304, 3570, 2150, 2358, 3043, 3915, 5529, 1776, 6129, 4280, 7049, 3618, 805, 3640, 5254, 210, 3428, 6646, 6385, 4147, 4569, 1422, 7283, 6279, 4148, 4190, 4340, 197, 4539, 939, 916, 6585, 1903, 4127, 137, 4612, 2400, 4494, 7139, 4754, 1502, 6178, 3997, 1099, 7218, 1089, 1029, 4431, 1987, 46, 6880, 6258, 2466, 5407, 3602, 4764, 6738, 1568, 1080, 676, 5116, 5282, 4512, 2632, 2066, 2995, 1372, 3339, 2867, 2224, 919, 6566, 4775, 620, 6966, 5728, 4563, 920, 3421, 1943, 629, 3045, 590, 5365, 77, 2178, 7389, 7301, 3875, 3278, 7390, 4781, 2321, 3890, 6719, 7225, 2448, 3368, 5262, 2980, 1982, 370, 417, 5992, 6315, 10, 3606, 6837, 3955, 2635, 6174, 2927, 207, 6078, 3418, 7097, 3740, 6398, 2671, 279, 4920, 6636, 3693, 3957, 4121, 1567, 5825, 5726, 5425, 1060, 5739, 5842, 4591, 813, 6018, 6857, 1082, 5579, 6688, 4354, 3510, 1063, 1330, 7373, 2199, 4428, 3263, 1561, 1579, 1335, 5754, 2629, 7066, 5050, 4402, 3986, 225, 6731, 2944, 379, 4581, 825, 2823, 213, 6865, 3861, 3156, 1228, 2885, 1795, 6916, 4768, 2359, 4294, 2183, 6164, 3927, 3977, 5352, 1692, 3266, 1602, 2104, 2113, 2461, 2573, 3016, 925, 5777, 217, 1100, 5361, 1265, 2610, 1726, 260, 5480, 891, 1278, 5751, 6875, 5510, 2329, 3165, 6241, 5453, 5659, 5760, 2851, 2081, 6115, 7208, 4444, 4011, 2752, 3785, 5689, 3053, 2860, 243, 789, 2483, 4819, 2394, 1394, 4532, 1714, 5978, 467, 1715, 1068, 6589, 7038, 2160, 2265, 6310, 3519, 3463, 4957, 5877, 2694, 1466, 5088, 5731, 5059, 1812, 4941, 4265, 1751, 5759, 5624, 5398, 4338, 3878, 2763, 6569, 2802, 6220, 223, 2209, 5493, 2257, 2807, 6710, 5000, 5292, 100, 2515, 1391, 2947, 3125, 4833, 2945, 2235, 5676, 5667, 2506, 5374, 6825, 6774, 4531, 6963, 918, 3493, 1333, 6195, 2739, 5705, 6879, 6833, 6928, 5434, 2340, 6758, 3066, 3454, 4820, 5051, 6604, 4738, 1061, 2175, 6821, 1689, 2170, 4240, 7340, 7230, 6761, 135, 4705, 3397, 5927, 1053, 3213, 3363, 1961, 7180, 5033, 4255, 2475, 7238, 838, 7010, 351, 5146, 398, 7091, 5381, 6687, 4777, 6735, 5875, 1632, 5567, 2832, 723, 674, 4784, 5828, 64, 4915, 52, 2245, 6715, 1415, 4480, 2345, 5166, 1782, 4919, 14, 1897, 5620, 3653, 6901, 3117, 6032, 510, 514, 1118, 6036, 6342, 4992, 3048, 2196, 1837, 3714, 4940, 1664, 6814, 388, 1069, 4628, 1646, 3071, 6051, 6425, 3098, 1093, 460, 1821, 2821, 5743, 4395, 2792, 2339, 5045, 854, 1637, 5491, 1770, 3333, 5156, 4906, 5224, 5655, 881, 1428, 1838, 1148, 1946, 1279, 4420, 771, 6693, 4763, 6493, 1577, 2904, 5355, 2666, 3946, 633, 4846, 2229, 6508, 837, 1634, 1386, 6509, 6381, 6349, 1729, 5174, 2669, 5933, 3593, 760, 4210, 5642, 718, 2411, 1560, 1871, 270, 754, 7167, 6870, 4560, 3842, 5855, 1485, 1306, 6873, 7140, 2017, 4914, 2558, 5134, 7355, 2, 6681, 3124, 5388, 5129, 582, 1642, 4970, 5031, 6615, 1067, 1886, 6157, 914, 4575, 3410, 4767, 2863, 1174, 7084, 1993, 3442, 5007, 3386, 1786, 3868, 2578, 443, 7317, 7015, 4417, 1188, 4217, 2350, 2292, 3155, 3064, 2020, 1841, 3326, 7089, 2847, 907, 2753, 7217, 2233, 4835, 3624, 5249, 485, 5139, 3004, 1983, 2938, 5112, 68, 5629, 3635, 975, 5918, 1623, 1456, 543, 4607, 4103, 3676, 6357, 4538, 3702, 1056, 7178, 4261, 7206, 6666, 901, 1912, 39, 693, 2828, 3594, 5022, 4931, 6035, 2460, 5879, 4483, 265, 3200, 3448, 6305, 3525, 4918, 3344, 1342, 6473, 2471, 2838, 6297, 2718, 6661, 1365, 4904, 617, 2157, 3057, 5606, 4552, 3424, 5733, 2697, 5433, 1412, 3529, 3488, 2880, 5080, 3139, 2574, 5424, 1680, 6175, 6250, 1691, 1023, 3496, 3011, 3332, 6394, 1196, 6011, 6058, 7391, 3787, 5679, 6190, 7198, 269, 3082, 4009, 4203, 188, 3757, 5490, 4606, 1170, 2331, 2473, 2280, 53, 1352, 3396, 2665, 6372, 5864, 6691, 4424, 5215, 2677, 2086, 5014, 4285, 1941, 4319, 4528, 623, 7233, 5466, 1895, 2062, 2208, 2955, 4710, 1066, 1490, 6802, 4192, 259, 1297, 48, 6961, 3568, 294, 6221, 3464, 5923, 4516, 3362, 5301, 1146, 5640, 1346, 3382, 7405, 2928, 3338, 119, 684, 4984, 6743, 1432, 6544, 2728, 5581, 6021, 6259, 6427, 4235, 7380, 3730, 1260, 4407, 568, 5994, 1578, 6120, 5764, 4546, 7252, 2373, 3347, 1905, 2406, 1752, 4196, 3132, 5344, 4934, 5832, 4514, 3661, 5612, 3439, 5750, 5752, 7379, 5725, 6982, 3853, 2964, 2432, 6145, 770, 3634, 4550, 6025, 2069, 2032, 3933, 3756, 3706, 6683, 2168, 2021, 1145, 4577, 701, 2488, 5273, 5222, 945, 6952, 4831, 5315, 1519, 1277, 4704, 384, 482, 2704, 5049, 1236, 6751, 6531, 6649, 319, 4559, 830, 5304, 1238, 935, 6625, 4364, 3061, 6550, 1547, 1820, 3705, 3014, 341, 472, 2568, 5604, 3000, 1511, 961, 3576, 4985, 3922, 5157, 1949, 6180, 2203, 3256, 5710, 4843, 5658, 902, 835, 4963, 1863, 4057, 4164, 3766, 6301, 1658, 5935, 6252, 2416, 6228, 5506, 722, 2871, 7174, 5544, 670, 7096, 5807, 3031, 3941, 6617, 6909, 2141, 6701, 4458, 3352, 4530, 412, 553, 5410, 5483, 2305, 7325, 5336, 6242, 219, 2795, 520, 1675, 5158, 1299, 7079, 2658, 6658, 5685, 6028, 4353, 3945, 1875, 4345, 5141, 6782, 6026, 2455, 1375, 5005, 6048, 1801, 651, 5717, 4616, 5840, 3486, 4414, 581, 2140, 2712, 2465, 6836, 4888, 2493, 6610, 1743, 4110, 884, 5886, 3656, 1954, 5719, 1390, 4688, 3227, 4371, 2161, 1267, 5787, 6001, 3025, 5851, 277, 6859, 1965, 6940, 5271, 791, 4747, 5066, 2128, 6929, 3662, 7213, 1182, 3224, 6502, 3923, 2298, 6456, 6543, 6198, 5952, 2009, 5409, 5354, 3996, 5508, 6376, 2076, 3771, 2388, 7033, 4830, 2855, 2047, 6697, 4869, 5144, 4509, 4681, 4515, 5790, 2811, 922, 2740, 5956, 6307, 541, 4709, 2098, 6716, 3613, 1826, 4252, 2705, 6791, 4198, 3210, 2185, 7332, 4037, 5288, 1661, 6292, 350, 2252, 6609, 1802, 1892, 3727, 3947, 2707, 5195, 3498, 5442, 6696, 7345, 5408, 5609, 3062, 3068, 1732, 6022, 2051, 365, 6521, 1370, 896, 4750, 1566, 910, 1251, 6611, 2796, 5240, 6863, 1582, 782, 3020, 4304, 3789, 2749, 5327, 4056, 330, 4275, 5184, 6413, 1779, 20, 4423, 2036, 1980, 6130, 299, 6402, 3022, 6523, 5147, 6285, 3217, 2611, 4189, 3459, 4749, 3596, 2735, 5275, 5382, 3829, 526, 1580, 530, 1275, 4419, 2438, 1851, 6368, 1229, 7042, 605, 5046, 3758, 3901, 6341, 3978, 47, 1644, 3091, 897, 2686, 1679, 7041, 5233, 4278, 129, 1266, 4755, 3458, 2056, 2217, 2478, 5836, 3655, 3587, 3453, 7221, 4805, 2652, 5594, 700, 6449, 4161, 7254, 3675, 1683, 2054, 159, 844, 6068, 1025, 3265, 5514, 536, 6689, 356, 24, 1958, 1855, 5793, 425, 1619, 7155, 5238, 776, 144, 1883, 3879, 4773, 6428, 1442, 6618, 3407, 6052, 4061, 4071, 7240, 475, 966, 540, 6409, 1800, 3998, 1933, 6664, 2745, 2726, 5988, 5737, 1724, 1496, 4287, 2767, 2926, 3966, 6643, 3148, 2442, 1930, 6429, 601, 3427, 3299, 962, 4212, 7262, 4112, 2005, 3767, 4615, 4937, 363, 6519, 309, 5519, 4595, 6291, 4447, 430, 5109, 5584, 152, 6073, 1756, 833, 4163, 1324, 5500, 2494, 2820, 4663, 2273, 5795, 7154, 4029, 6155, 5247, 3748, 4098, 6518, 6338, 1641, 6312, 6147, 3434, 5965, 4691, 6668, 2360, 4387, 90, 5299, 5632, 7249, 459, 3275, 3684, 965, 6094, 3067, 5044, 882, 5971, 7396, 7159, 6148, 7134, 3166, 3116, 2684, 6238, 1574, 1721, 4641, 1705, 4598, 3531, 468, 306, 4862, 6122, 247, 6217, 385, 2501, 4858, 2031, 967, 5106, 6360, 5930, 111, 3864, 3788, 5951, 3302, 6540, 898, 6669, 2902, 4170, 4254, 1368, 4882, 1673, 744, 2586, 3337, 5802, 6405, 4291, 4693, 6230, 2634, 2636, 3899, 6868, 6441, 1777, 7232, 5779, 1687, 2809, 4703, 707, 2949, 6905, 4445, 1968, 7156, 1234, 4898, 2230, 3279, 4033, 1995, 67, 6480, 6479, 4792, 7018, 1817, 5313, 120, 6680, 2412, 2547, 6792, 5331, 1911, 6606, 4401, 60, 1970, 3691, 3536, 4674, 295, 4824, 6321, 1036, 4218, 7347, 6169, 900, 97, 4912, 602, 283, 1039, 438, 264, 442, 2627, 2516, 2057, 168, 7068, 1539, 6973, 2730, 1050, 7012, 5324, 4828, 573, 2593, 1608, 166, 2033, 7313, 1668, 7122, 3249, 1479, 6991, 3980, 2169, 7144, 3220, 2981, 2679, 5631, 576, 2291, 763, 641, 4903, 7133, 6407, 1077, 2563, 1667, 1070, 3815, 5329, 3679, 2491, 6077, 2110, 5191, 4526, 1839, 5295, 7286, 1255, 502, 5467, 2152, 3300, 1168, 5395, 4685, 182, 1414, 706, 2085, 1725, 927, 3632, 6558, 6654, 5028, 4068, 5588, 5891, 5516, 4578, 588, 3865, 6979, 1301, 1104, 6601, 3849, 1495, 4668, 2709, 6452, 6632, 1438, 4551, 1431, 5441, 2238, 2078, 6575, 7354, 2698, 3371, 1586, 2674, 6, 5746, 981, 2191, 7020, 4081, 878, 5576, 4276, 1041, 2267, 5199, 1210, 2695, 5839, 4059, 528, 2661, 4464, 2243, 3960, 2092, 3686, 268, 886, 6076, 5784, 5880, 4588, 2706, 4350, 5976, 4580, 1957, 6383, 1694, 5212, 5720, 6705, 6958, 1035, 3330, 474, 2375, 3109, 1318, 5590, 1712, 6282, 6709, 7253, 5721, 1836, 998, 4488, 5029, 934, 3862, 1599, 1654, 3415, 3081, 2323, 2600, 1868, 3133, 134, 7077, 1873, 616, 5169, 285, 5333, 2956, 2836, 6373, 6149, 3289, 5526, 1405, 4962, 2985, 3086, 3307, 4945, 1248, 3816, 7222, 5073, 5207, 2420, 1187, 5641, 6889, 7260, 824, 649, 3584, 3646, 3319, 2689, 5012, 6811, 461, 3884, 7026, 2962, 3346, 6698, 6798, 378, 1884, 2256, 1589, 2194, 4829, 5150, 2390, 1470, 2535, 1362, 4430, 4388, 2562, 1464, 5401, 1401, 6685, 3335, 6953, 7316, 4477, 6600, 5445, 3202, 4386, 517, 6401, 3489, 6188, 2399, 6917, 1925, 3389, 1418, 5687, 2888, 6980, 311, 5095, 1657, 125, 5369, 5800, 2527, 7312, 1885, 5538, 2167, 3852, 5202, 2701, 1176, 5081, 2522, 7258, 1570, 2317, 5318, 665, 6234, 4145, 108, 497, 3027, 1338, 5020, 2602, 2710, 862, 7196, 5628, 2897, 5142, 4003, 4713, 6888, 6561, 6483, 1349, 5278, 6231, 6299, 1814, 2008, 1253, 5557, 7103, 1483, 2132, 3044, 1789, 6542, 2834, 6227, 1794, 1727, 3562, 196, 2124, 747, 1241, 2825, 7404, 5475, 3564, 2099, 4733, 187, 5854, 1588, 3920, 4827, 5387, 6290, 5162, 194, 4207, 4697, 3668, 4090, 5468, 1888, 6296, 2957, 2176, 3241, 566, 301, 7400, 5991, 5693, 7162, 4502, 6766, 271, 2499, 3569, 5391, 4795, 1917, 2451, 697, 4585, 6436, 4370, 5512, 222, 321, 4734, 1237, 3827, 7248, 5636, 3169, 7017, 1552, 5190, 7082, 3659, 4306, 6665, 1074, 1164, 5306, 625, 5268, 1138, 3631, 6463, 2353, 7199, 3723, 1719, 5972, 4517, 738, 4184, 3769, 726, 1175, 439, 3987, 3869, 4586, 5882, 2089, 5749, 5, 7376, 5430, 3551, 2639, 6614, 3290, 6212, 1291, 4468, 5986, 1755, 6976, 3612, 4823, 683, 6443, 5200, 2288, 4748, 810, 3233, 3023, 614, 44, 2372, 3973, 2869, 1806, 2369, 4179, 1261, 2526, 734, 7131, 6379, 6559, 2604, 7279, 6852, 246, 5287, 4248, 4443, 4001, 5849, 6516, 781, 2991, 548, 6286, 6572, 2678, 1198, 404, 1457, 6832, 6024, 5339, 1286, 6275, 1024, 261, 3954, 6253, 3157, 6674, 6876, 6830, 3630, 7039, 4446, 7130, 4859, 6316, 555, 4961, 5297, 2642, 2919, 4367, 1854, 7356, 2289, 1746, 6047, 1097, 7045, 1044, 1731, 2231, 826, 6140, 4349, 3103, 3645, 1223, 5622, 2123, 2173, 4808, 2429, 3803, 2977, 4397, 4649, 1862, 353, 2998, 4582, 764, 1385, 5198, 4492, 5070, 79, 251, 5829, 709, 6560, 2389, 4085, 1449, 4342, 179, 908, 2746, 6911, 1723, 4626, 4311, 5196, 3447, 2462, 5905, 5596, 713, 3080, 8, 5392, 3775, 1426, 4849, 6856, 180, 7153, 2886, 4832, 5373, 5314, 1062, 3176, 6804, 3928, 492, 5753, 6266, 420, 2278, 1594, 1669, 1190, 2657, 5826, 2793, 6263, 2108, 5486, 2510, 3121, 808, 433, 6128, 5133, 7001, 2649, 3432, 4044, 5699, 5979, 953, 648, 2999, 3579, 5362, 512, 7307, 3694, 938, 5608, 7083, 2470, 6136, 1114, 419, 2976, 6339, 6096, 811, 1491, 756, 3351, 465, 5319, 4418, 6099, 2392, 112, 4932, 519, 7184, 1413, 1096, 589, 1317, 2765, 4977, 3811, 3808, 5582, 184, 5943, 6039, 6038, 3401, 6081, 4343, 850, 6713, 426, 5472, 7367, 2210, 1813, 1908, 5562, 6686, 308, 2088, 4010, 4365, 6891, 7290, 1033, 5555, 642, 2120, 7256, 3590, 652, 4041, 1859, 667, 6527, 4541, 6240, 3556, 6271, 4284, 5794, 4999, 2067, 899, 3832, 7065, 7047, 4283, 2618, 3365, 2778, 6084, 3504, 2934, 2190, 2300, 2620, 6673, 636, 336, 5761, 199, 594, 3916, 6864, 5048, 37, 6159, 3943, 1387, 3687, 5890, 1918, 5192, 4540, 4987, 551, 4324, 4707, 554, 757, 2071, 4659, 798, 1730, 4305, 431, 1474, 930, 7142, 4879, 7357, 5309, 6101, 410, 5494, 5625, 451, 4825, 6887, 2275, 2415, 3746, 6182, 5175, 5341, 1852, 235, 5724, 3085, 1783, 2890, 5357, 4497, 6588, 6440, 3215, 6721, 586, 5729, 6565, 2225, 6621, 7397, 58, 6702, 6768, 3790, 3187, 4095, 5140, 3356, 7040, 613, 178, 597, 4927, 1648, 7339, 6872, 5565, 6845, 4881, 2000, 4889, 544, 1484, 5698, 6677, 7242, 5680, 4411, 574, 6295, 6788, 679, 4871, 422, 5847, 765, 3535, 2402, 5903, 6057, 2803, 4839, 2133, 1688, 6507, 17, 27, 3924, 5426, 2162, 6396, 5660, 3466, 3312, 3649, 7327, 1379, 337, 3583, 2776, 5871, 1906, 2856, 3296, 5505, 5809, 3643, 7188, 7114, 3951, 1699, 984, 490, 4554, 5038, 334, 1293, 6437, 6471, 552, 4489, 4619, 7231, 471, 3972, 904, 5187, 3678, 7087, 3052, 2993, 3580, 4623, 1898, 4101, 2965, 2950, 3467, 2422, 2628, 4507, 1467, 4807, 167, 1874, 7053, 4592, 4579, 4143, 6325, 4868, 4803, 6337, 6055, 3384, 2655, 4890, 7285, 1781, 1034, 2192, 7126, 5556, 1348, 4089, 3005, 2290, 5263, 5708, 868, 5735, 2149, 2277, 71, 3271, 4250, 3350, 4072, 3144, 947, 4028, 1819, 6193, 2313, 6111, 5666, 3143, 101, 5546, 6935, 4965, 1396, 871, 3571, 788, 6323, 6671, 1720, 4902, 413, 1040, 2543, 3102, 5934, 347, 2444, 6261, 5692, 1120, 5194, 4099, 1109, 3490, 3225, 5742, 1856, 5911, 3792, 7135, 4634, 4892, 3741, 1106, 2723, 6450, 6639, 769, 932, 3196, 4058, 3147, 2237, 5303, 547, 2001, 1955, 6526, 4256, 2206, 5367, 6813, 3621, 6288, 6304, 5554, 6772, 4834, 2968, 4183, 2788, 1208, 1244, 3054, 7269, 1932, 4951, 6303, 6679, 6605, 3888, 4225, 1787, 4027, 3301, 5090, 3394, 2794, 5461, 743, 273, 2348, 4394, 4449, 6699, 6382, 6211, 6783, 2973, 5122, 2418, 5204, 2967, 185, 7105, 124, 1973, 286, 1656, 3034, 464, 202, 2713, 5774, 5379, 690, 6455, 6886, 3162, 4322, 5016, 316, 1950, 2544, 546, 7011, 4694, 7294, 2325, 2663, 3297, 6265, 2249, 3380, 5470, 4910, 4521, 3280, 4382, 56, 7227, 3860, 3989, 7335, 5152, 5186, 1811, 2144, 2969, 570, 2766, 6392, 4380, 6567, 549, 3817, 680, 1287, 2889, 7128, 3392, 5455, 3186, 806, 1834, 814, 2512, 3245, 6438, 6215, 951, 418, 2495, 2450, 2921, 4872, 2121, 2270, 832, 3866, 1880, 677, 2667, 978, 2872, 3465, 4770, 834, 7173, 128, 2532, 2664, 3533, 2559, 4679, 1295, 5889, 2680, 6374, 5704, 780, 6254, 3588, 1620, 1086, 342, 5773, 1640, 4690, 87, 4222, 7384, 751, 3738, 2125, 4052, 92, 872, 5806, 1858, 2850, 5419, 6332, 6114, 2747, 5967, 1616, 1122, 7146, 162, 6633, 203, 2895, 4670, 3929, 3348, 3893, 4972, 3719, 5664, 505, 6854, 1184, 1520, 4818, 7067, 6645, 3092, 4501, 6769, 3798, 2251, 406, 2334, 2293, 5772, 6173, 6505, 3083, 1360, 5213, 1153, 2382, 4711, 6087, 4880, 118, 4753, 5159, 5117, 2186, 4471, 2523, 5084, 7043, 6944, 6010, 1406, 2414, 3743, 885, 3012, 7023, 6184, 1076, 4913, 5216, 6900, 4435, 3294, 3664, 2557, 2744, 5568, 2916, 558, 5235, 5043, 407, 5615, 5128, 2939, 2592, 2759, 5173, 6983, 5459, 1193, 4852, 2500, 7106, 2913, 4097, 6272, 1829, 2490, 3975, 777, 2548, 5504, 971, 3357, 5865, 5684, 5541, 226, 3446, 3029, 6897, 807, 6150, 4270, 5110, 822, 1555, 4486, 4929, 1809, 5741, 6987, 2691, 2307, 154, 4689, 1052, 5770, 147, 4742, 2094, 3538, 2900, 170, 1083, 712, 1475, 5946, 6960, 5816, 6348, 4752, 6318, 7069, 3885, 5677, 1048, 397, 3839, 5418, 2073, 5893, 3395, 4574, 5057, 3932, 2218, 7304, 2079, 6213, 1178, 2670, 6435, 3204, 3629, 5892, 4004, 955, 4583, 4557, 585, 2541, 4769, 4561, 2342, 5188, 2248, 4352, 3617, 7364, 2309, 423, 7004, 2006, 1923, 6097, 2961, 1747, 1380, 2458, 3919, 4779, 6109, 4241, 2572, 3433, 6165, 3239, 2848, 189, 7239, 6194, 6328, 7302, 1443, 3908, 5904, 3218, 4562, 5364, 5457, 5603, 479, 1718, 819, 3812, 4518, 818, 5437, 6046, 5964, 6920, 929, 3912, 2063, 5171, 1590, 2115, 3304, 1108, 335, 6707, 7246, 2553, 2296, 94, 1652, 7349, 3211, 4257, 3229, 51, 4341, 3173, 2876, 3472, 5266, 5599, 3441, 3690, 2287, 6500, 5325, 2840, 7351, 531, 1150, 7175, 2737, 1508, 2332, 4206, 2762, 4590, 3328, 7099, 4362, 4093, 146, 5643, 7350, 1549, 1458, 2614, 6993, 6222, 793, 1481, 1964, 245, 5182, 25, 1805, 4115, 2090, 4667, 2476, 1737, 1425, 1030, 1162, 6828, 4718, 5869, 1808, 3805, 1307, 4973, 2520, 741, 5232, 5172, 3327, 5089, 4297, 34, 3615, 5205, 1815, 5002, 6861, 2899, 4108, 35, 3522, 5968, 859, 4392, 3455, 1141, 5663, 3641, 4156, 534, 3177, 6734, 5736, 5279, 1748, 2721, 1202, 3388, 142, 749, 5097, 5061, 2151, 2798, 7311, 2044, 5115, 1525, 4804, 21, 2241, 2688, 3909, 1504, 4683, 2274, 6089, 1018, 5307, 578, 7192, 5703, 106, 2908, 4433, 4677, 2743, 7214, 6678, 1017, 6501, 5509, 2012, 4054, 5489, 2612, 2028, 3695, 4359, 2894, 6921, 1363, 1846, 6959, 3408, 1505, 6302, 3137, 740, 5219, 4479, 2045, 4712, 26, 1799, 3870, 3554, 6525, 158, 2029, 1625, 2970, 6117, 5488, 3288, 2605, 2010, 513, 55, 2349, 1280, 33, 81, 2068, 1268, 3683, 4138, 4377, 5302, 3877, 1636, 139, 5417, 7244, 2773, 394, 4801, 2337, 6464, 5940, 1085, 2772, 4078, 5995, 484, 4772, 4455, 6308, 6667, 5164, 1058, 5085, 1562, 7363, 7237, 6324, 6335, 3616, 4847, 3925, 3009, 104, 1102, 1767, 3094, 2569, 1532, 6112, 4490, 6161, 4084, 4778, 5272, 6877, 457, 2641, 6823, 3970, 123, 5910, 1270, 6504, 2519, 175, 7104, 7277, 2874, 174, 4672, 121, 3573, 2430, 4039, 6446, 4185, 4393, 2653, 6146, 2622, 5897, 6923, 3619, 5265, 6003, 462, 7152, 1382, 7016, 687, 2508, 5121, 5482, 3713, 4399, 3546, 853, 3168, 4019, 3360, 3658, 6770, 1497, 4715, 6037, 380, 4021, 1569, 4176, 1548, 1545, 1258, 5876, 4487, 809, 5286, 114, 7003, 4124, 6714, 6590, 2050, 1169, 958, 4983, 1327, 4247, 6353, 2145, 6166, 3854, 1861, 3226, 3806, 3794, 31, 828, 3406, 2341, 1963, 2259, 6223, 3264, 4120, 1049, 6481, 3499, 3298, 4269, 3470, 3377, 5244, 1305, 2731, 3431, 2882, 2524, 6819, 5214, 7295, 5560, 7235, 5868, 3652, 728, 4233, 6587, 792, 4214, 4154, 1823, 7259, 6482, 704, 13, 6267, 5712, 4358, 3151, 4378, 7081, 2361, 5738, 3778, 6356, 3508, 3244, 1916, 1213, 1550, 6675, 3331, 3274, 3345, 2148, 4092, 5525, 2783, 6127, 979, 3076, 4864, 409, 6641, 4841, 6276, 6822, 4952, 5210, 5524, 2019, 888, 527, 7299, 381, 1583, 6957, 3030, 5180, 4328, 3353, 3236, 5896, 6256, 1400, 4614, 5668, 4564, 6364, 2729, 275, 6054, 5573, 3260, 416, 4369, 4885, 804, 4529, 6962, 4390, 4091, 5744, 6045, 6423, 6700, 401, 1765, 6581, 673, 4504, 4979, 5221, 3856, 3159, 3953, 4730, 5571, 3534, 4678, 7329, 5456, 4031, 5674, 4570, 45, 6177, 6557, 752, 5585, 6924, 1328, 3, 1674, 1325, 1404, 1463, 6387, 4174, 5999, 2781, 6801, 3844, 7143, 1308, 1151, 4861, 3819, 7176, 3964, 6071, 1493, 5231, 5815, 1180, 1920, 2467, 4816, 7197, 5132, 450, 5217, 5349, 2822, 1533, 1990, 3379, 6350, 5223, 976, 3742, 5691, 5533, 4736, 5895, 5515, 1977, 1969, 1334, 3403, 5446, 1435, 6741, 5626, 3405, 76, 3796, 525, 4087, 5591, 6563, 5535, 355, 1059, 2384, 786, 3013, 6103, 3851, 7111, 2849, 1105, 6411, 5102, 5613, 449, 1289, 6365, 6457, 2997, 5845, 6634, 4788, 4966, 5874, 5123, 727, 1351, 3303, 1665, 2755, 4082, 3971, 331, 4708, 4465, 4326, 4066, 2764, 4315, 2615, 6397, 2351, 5711, 7293, 4137, 4956, 4347, 6824, 3638, 2434, 4383, 996, 4026, 4475, 340, 272, 2565, 4357, 1947, 6176, 302, 4432, 6172, 4648, 3780, 440, 6511, 5593, 3292, 1201, 1459, 7323, 3323, 5695, 977, 6795, 4308, 640, 719, 5450, 6283, 5501, 3087, 5023, 241, 6287, 4867, 3537, 2179, 5532, 2951, 4067, 2831, 6874, 3891, 1064, 4018, 7275, 924, 3400, 1948, 4403, 4169, 5208, 6458, 3322, 2994, 1926, 3840, 959, 6196, 3425, 6908, 3248, 4216, 5942, 4373, 117, 3736, 6153, 4158, 5368, 2180, 4416, 3373, 1340, 2513, 4264, 1498, 5363, 1985, 550, 5322, 5400, 2181, 1670, 441, 759, 5619, 3637, 4756, 1690, 190, 3036, 6989, 4946, 3134, 3872, 2960, 6522, 131, 4765, 7342, 3221, 5901, 6720, 3608, 5861, 799, 5686, 7296, 5119, 1604, 1054, 493, 3514, 107, 5083, 4273, 3422, 7292, 2943, 3378, 3627, 5165, 23, 6914, 4132, 2556, 2780, 1762, 1772, 4789, 1613, 5267, 1516, 1161, 4167, 6803, 5449, 6682, 2864, 366, 377, 5955, 4700, 4702, 368, 257, 3804, 3038, 5068, 1893, 6754, 851, 211, 4644, 1303, 2875, 6229, 109, 730, 5818, 6595, 5953, 6492, 4333, 7281, 5502, 2405, 5548, 2346, 1220, 4325, 6152, 5021, 2322, 83, 4321, 3677, 7009, 1972, 2862, 3931, 1610, 5176, 2093, 3826, 1037, 3557, 1367, 1072, 1627, 5675, 3648, 3520, 2022, 6820, 201, 5473, 3518, 2294, 1840, 3154, 847, 4675, 4954, 4621, 5916, 5397, 4640, 717, 5127, 7160, 2700, 5671, 2246, 2989, 6400, 2111, 1221, 1329, 3733, 2117, 1962, 6371, 4766, 6143, 4476, 2534, 1649, 4208, 609, 3874, 5094, 2861, 6988, 1928, 3391, 1046, 6226, 6366, 1127, 5193, 1381, 4074, 186, 5915, 3831, 2383, 1107, 4024, 973, 2236, 1227, 875, 5413, 4836, 3755, 7036, 845, 4312, 2004, 7387, 5300, 3553, 5082, 1131, 4815, 3904, 2364, 7166, 3336, 5058, 5914, 6726, 3056, 6034, 3813, 1899, 2801, 5474, 234, 3383, 2215, 2380, 2318, 2585, 1785, 2220, 3037, 4922, 3939, 6042, 6619, 3810, 2551, 1919, 2193, 2055, 2784, 4065, 2931, 2760, 3540, 4794, 3097, 1383, 333, 1556, 5145, 3797, 4751, 5228, 1007, 3582, 4421, 1803, 737, 5471, 2748, 4584, 6424, 626, 181, 4565, 6724, 500, 3188, 6243, 6855, 143, 4473, 6536, 7124, 2443, 3267, 4182, 5390, 4696, 3367, 3060, 6380, 4848, 244, 668, 6144, 7179, 263, 3065, 3759, 1981, 6598, 3214, 3008, 2024, 5902, 917, 1697, 2575, 4744, 7093, 1402, 6748, 105, 402, 49, 5241, 1711, 4022, 6377, 2472, 2596, 5549, 631, 72, 3791, 2119, 7061, 5429, 5559, 4968, 2100, 5578, 5734, 4990, 1055, 2741, 7399, 6384, 1573, 4151, 3680, 6418, 6546, 2624, 6938, 1397, 6895, 5183, 954, 4643, 1576, 3222, 116, 2463, 5053, 5543, 3007, 5377, 6092, 1749, 2489, 6925, 6393, 6008, 3610, 507, 5034, 5284, 332, 1388, 2419, 4549, 1119, 7341, 2003, 6943, 3295, 4267, 2396, 748, 5775, 6049, 2942, 1565, 5888, 4368, 1320, 5347, 1028, 5605, 5597, 3411, 2015, 2638, 6885, 1341, 7300, 6300, 3818, 70, 4005, 7352, 3482, 5135, 7211, 5253, 5700, 7161, 5225, 6072, 3284, 3724, 4079, 4936, 5054, 6416, 3623, 5881, 5340, 6451, 6640, 5332, 16, 4969, 73, 1597, 1423, 3178, 2240, 4149, 3773, 1257, 75, 5013, 739, 432, 2681, 3940, 2675, 4412, 7072, 6662, 2643, 2271, 6711, 4023, 1001, 666, 3937, 7170, 1989, 1453, 535, 5575, 1090, 3654, 5929, 2538, 5919, 5177, 362, 6490, 5961, 7168, 839, 6902, 2326, 750, 3231, 3258, 32, 4374, 5298, 1204, 5420, 345, 2911, 3412, 1607, 393, 4191, 1319, 6786, 2479, 603, 4200, 293, 3426, 5160, 3578, 1374, 5120, 5077, 2953, 3049, 1189, 1984, 4687, 4048, 4051, 3051, 4851, 1759, 2719, 7078, 7204, 7070, 3104, 3511, 6747, 2184, 1439, 5830, 974, 663, 4199, 4610, 3370, 5234, 22, 4491, 1522, 5797, 242, 5277, 3105, 5310, 5469, 2135, 6467, 3414, 2285, 6555, 4782, 3136, 2958, 5681, 4959, 1764, 2922, 5255, 2616, 5226, 669, 2040, 5098, 3666, 1521, 4032, 3262, 7051, 7194, 593, 7127, 6264, 5495, 2188, 7385, 3698, 2833, 4094, 4281, 4776, 735, 6694, 7046, 3420, 4991, 1416, 7278, 3801, 3843, 5701, 3209, 3480, 5805, 1246, 3481, 1849, 5227, 1793, 2314, 3174, 504, 6742, 2454, 7234, 3238, 6582, 7141, 3880, 1617, 2095, 5812, 779, 1696, 3657, 2915, 1323, 2072, 3944, 1011, 4938, 5423, 214, 3516, 1758, 2810, 1791, 3728, 4129, 6651, 5824, 3090, 3402, 5206, 4493, 4982, 785, 6807, 2333, 4556, 4760, 563, 454, 1744, 173, 3867, 5485, 1092, 2232, 1929, 671, 5763, 183, 4123, 2530, 7289, 42, 6690, 1186, 3399, 6273, 4193, 4870, 453, 4837, 5518, 274, 4020, 1797, 1546, 6314, 7014, 6255, 96, 6956, 3509, 3114, 2070, 3050, 224, 3485, 768, 458, 1183, 866, 4014, 2409, 436, 4236, 4944, 5651, 6839, 1921, 4105, 1866, 3002, 4474, 3697, 621, 988, 3597, 2074, 1465, 5452, 7138, 4593, 1935, 5075, 5338, 4943, 2909, 7392, 7274, 5003, 7315, 3364, 3992, 6313, 6767, 4249, 694, 414, 280, 4268, 6554, 6756, 2974, 705, 444, 2756, 4391, 1019, 1332, 1264, 2509, 2202, 489, 5026, 1506, 115, 2550, 7223, 3126, 5258, 1842, 753, 1235, 5093, 2640, 6729, 2330, 2153, 4258, 1087, 4866, 1, 6776, 6415, 1469, 7268, 3969, 6497, 3366, 732, 7245, 7205, 2626, 5996, 2221, 6294, 2717, 950, 6067, 3430, 3035, 7032, 2253, 315, 110, 827, 1111, 6233, 4655, 3696, 3603, 7048, 5944, 6336, 1124, 2654, 5617, 3445, 6031, 4887, 1451, 2201, 3443, 533, 6740, 4901, 2925, 3140, 5511, 3903, 2884, 1927, 4926, 6495, 4658, 3636, 1410, 4442, 122, 4153, 7076, 4100, 5858, 606, 5056, 716, 5105, 2112, 1446, 3749, 5237, 3475, 1424, 4916, 6586, 3216, 802, 4759, 4348, 6445, 2310, 5827, 346, 5211, 6014, 4286, 6955, 7305, 3277, 2279, 4806, 3119, 5528, 1952, 3651, 3160, 4205, 3670, 3505, 5928, 3318, 6592, 511, 3739, 6070, 960, 6009, 3135, 4136, 6922, 5269, 3444, 3674, 5713, 2446, 1462, 3669, 5037, 1624, 374, 7095, 5570, 4316, 5926, 4642, 4853, 1541, 7331, 7025, 3462, 1584, 2075, 1639, 6186, 4863, 2295, 3547, 6179, 3982, 1681, 1682, 5312, 6529, 4221, 41, 1824, 2529, 6860, 3965, 6663, 3764, 3194, 2503, 193, 3179, 5714, 3128, 2789, 5345, 2903, 3993, 5167, 1429, 1231, 575, 678, 4924, 681, 403, 1225, 5833, 4272, 4197, 7163, 4686, 1373, 4542, 6468, 3075, 5460, 2988, 6289, 267, 2227, 6469, 4928, 1008, 7261, 1626, 6363, 6466, 6260, 1701, 1042, 4799, 445, 6657, 2234, 5941, 1575, 2910, 2542, 4741, 658, 127, 790, 3572, 3287, 6789, 3469, 4635, 2166, 755, 5690, 7264, 4533, 952, 320, 5872, 192, 4978, 4242, 7306, 1647, 1487, 1269, 3494, 1259, 3990, 1551, 1005, 2992, 6197, 2312, 4600, 5589, 5114, 4571, 1271, 2608, 4002, 3949, 3474, 6277, 164, 498, 1073, 4527, 1835, 6326, 1433, 5634, 2441, 6576, 4613, 3285, 4437, 2591, 4878, 2963, 6841, 5627, 3824, 5041, 5350, 6369, 5781, 4160, 3671, 3343, 6239, 5722, 3962, 6762, 6183, 5808, 5648, 524, 4568, 6280, 1473, 5189, 6098, 6704, 5587, 4016, 6850, 7063, 4995, 4030, 6612, 396, 1618, 5025, 7377, 1460, 3252, 2818, 248, 4657, 6375, 4553, 6205, 639, 1079, 3850, 7060, 2413, 7209, 1760, 5810, 2787, 7219, 6430, 1766, 2059, 821, 4611, 4400, 6840, 4303, 1896, 1796, 2352, 5709, 3604, 1771, 5534, 5396, 1753, 1843, 6033, 7282, 5096, 5841, 6970, 1232, 3375, 3614, 4165, 5537, 2835, 3253, 499, 2846, 7403, 6851, 1006, 3269, 3595, 5614, 1224, 6137, 7183, 5564, 1408, 5464, 6638, 1158, 2972, 6894, 942, 4719, 883, 2525, 5201, 3185, 7145, 292, 1038, 1653, 5522, 151, 3894, 1163, 5569, 6044, 291, 387, 4935, 6637, 7006, 3527, 6954, 1357, 4665, 6027, 5291, 2261, 3858, 3503, 6752, 7229, 7288, 5788, 6537, 4555, 5463, 4204, 5852, 4481, 6278, 4899, 1998, 7321, 2845, 6574, 282, 2242, 6676, 3910, 4171, 742, 5873, 4282, 4335, 3848, 6160, 2954, 6626, 3019, 6552, 7158, 3190, 5274, 664, 5730, 5348, 1850, 3273, 486, 290, 5218, 375, 829, 6432, 2714, 6853, 2048, 2779, 5353, 1144, 5817, 5143, 6884, 2344, 7024, 7086, 860, 5328, 4662, 7031, 4955, 5883, 6119, 4404, 6799, 580, 4786, 894, 5553, 5285, 4535, 1717, 2720, 1671, 2984, 7210, 1867, 5330, 5998, 3152, 80, 1302, 2311, 1531, 6717, 1160, 4062, 3272, 4735, 2843, 7064, 6805, 3917, 3172, 6771, 7075, 6362, 2302, 5323, 3575, 6981, 2941, 5358, 2320, 3101, 6647, 2502, 3167, 1942, 452, 7359, 7185, 3095, 4245, 2896, 6200, 2560, 2782, 3750, 5948, 3015, 2812, 1500, 3321, 1945, 1736, 4699, 3158, 2645, 4293, 2546, 7201, 943, 6390, 3591, 6329, 4106, 5637, 2376, 2464, 5378, 969, 5308, 6075, 7395, 1557, 6477, 6759, 1399, 1345, 2505, 2581, 4231, 4298, 7398, 2597, 5100, 3958, 5454, 400, 3268, 3776, 4974, 130, 5015, 889, 4141, 5586, 1247, 5372, 5006, 1273, 1212, 3477, 4505, 2619, 4211, 2528, 5443, 5069, 1999, 4597, 2692, 3170, 6514, 3528, 3967, 6817, 509, 3208, 1672, 6351, 6245, 565, 5957, 869, 2920, 972, 3381, 572, 4873, 990, 7108, 4857, 5087, 2428, 7037, 3876, 3195, 1831, 5296, 2576, 4162, 4609, 4695, 7273, 7129, 4125, 4118, 2799, 446, 4452, 987, 3163, 3770, 3010, 2814, 4077, 4202, 5702, 5900, 5062, 6579, 538, 4302, 4905, 5356, 6059, 6007, 3120, 6102, 1354, 6785, 6406, 1013, 1411, 2647, 1022, 5481, 6461, 2932, 6386, 3145, 5432, 5958, 4716, 4260, 3497, 2452, 1139, 1020, 2328, 4573, 2425, 731, 840, 2879, 1509, 4499, 1014, 2648, 6118, 7151, 1152, 4356, 4262, 4008, 1010, 38, 1300, 5639, 3286, 1358, 1581, 944, 7073, 5716, 2254, 7118, 1478, 2659, 1272, 3112, 7243, 3807, 5289, 2806, 3823, 708, 4035, 4015, 1000, 6708, 892, 3517, 909, 4440, 3809, 399, 3314, 4757, 1211, 4948, 5072, 6395, 6603, 4013, 4313, 3835, 6780, 1830, 933, 2582, 4213, 6185, 4271, 1553, 1377, 2163, 1761, 6773, 5672, 1571, 3820, 3270, 220, 5558, 3549, 5335, 3625, 5561, 4826, 5019, 6517, 2975, 3984, 1775, 3078, 1768, 5922, 7059, 4025, 1065, 6002, 3985, 4, 2734, 1116, 4372, 2554, 4545, 3308, 3784, 2702, 1865, 2127, 5601, 5799, 1395, 6066, 1655, 4320, 7171, 1137, 6082, 4811, 1440, 4450, 2436, 4630, 1818, 638, 4809, 5385, 6794, 583, 657, 4737, 2367, 228, 4947, 2540, 4632, 1322, 1021, 5479, 4050, 4043, 4310, 2255, 852, 7000, 3483, 2827, 236, 5071, 392, 140, 796, 4224, 6535, 2211, 4669, 5086, 6040, 3552, 1461, 5647, 1159, 1216, 3046, 7216, 4119, 7203, 1217, 3744, 5950, 6745, 4159, 6580, 5136, 231, 4845, 233, 766, 1051, 3219, 1123, 557, 1773, 7090, 1026, 1598, 5960, 7326, 161, 5499, 5798, 3822, 136, 6652, 5185, 6085, 784, 7044, 2881, 1219, 2304, 5517, 1974, 3487, 6125, 5130, 5786, 2156, 5838, 820, 5099, 2564, 6244, 59, 2136, 296, 5769, 1612, 1281, 5982, 4351, 7220, 1638, 842, 6322, 6829, 4980, 7055, 1057, 6485, 1128, 3039, 6462, 6941, 6268, 6549, 4673, 5547, 54, 6858, 7116, 1659, 2269, 2498, 579, 5366, 2131, 3763, 4953, 7265, 3313, 6004, 1709, 6079, 7113, 6050, 1455, 4758, 4046, 7117, 4647, 6965, 6878, 1523, 4331, 3201, 5256, 634, 1693, 176, 1218, 3711, 3836, 6622, 1420, 3545, 4049, 5947, 4135, 2555, 12, 6777, 1740, 2187, 6105, 7109, 3150, 1078, 126, 2129, 2804, 3821, 6548, 3309, 1807, 4299, 4566, 2486, 4355, 6939, 936, 2906, 6907, 2431, 2046, 1544, 6812, 6551, 1482, 5600, 7058, 5539, 992, 6757, 494, 5245, 7272, 4682, 5406, 6942, 250, 3721, 3577, 6319, 2118, 1133, 1860, 5697, 4637, 7224, 5124, 4731, 1110, 1606, 1596, 3451, 103, 3682, 2114, 5776, 1445, 4150, 3994, 4633, 5520, 6354, 5550, 2171, 3456, 983, 6913, 3754, 6419, 5966, 923, 5478, 2507, 3707, 5850, 1353, 6069, 4558, 5937, 2437, 5393, 1738, 253, 5151, 3042, 391, 2790, 88, 1142, 6936, 2914, 1889, 6800, 5645, 5126, 5758, 2395, 5823, 3620, 7298, 2579, 6434, 856, 98, 2053, 5346, 2014, 1538, 6476, 7310, 4113, 6996, 6412, 864, 3142, 1611, 2817, 5899, 2262, 7368, 3180, 1876, 1713, 1389, 2480, 5932, 2011, 2901, 6444, 7200, 1130, 2696, 6984, 2281, 1913, 2077, 6404, 4525, 1477, 5884, 1784, 4195, 1181, 2603, 794, 329, 4415, 619, 1559, 2545, 1088, 1992, 2623, 2511, 560, 2930, 956, 611, 1757, 2606, 6345, 5040, 4244, 3574, 18, 4083, 3073, 3342, 1909, 2595, 6346, 2768, 5017, 2096, 357, 6060, 3914, 7386, 1254, 1882, 4034, 6583, 6062, 2357, 3183, 5118, 5008, 2959, 1194, 2775, 6298, 2797, 1622, 1631, 4722, 6985, 5155, 3468, 2484, 4045, 2356, 3762, 6718, 1910, 3115, 1230, 3938, 1195, 1592, 4130, 7112, 4253, 6274, 2570, 2204, 3857, 5415, 2013, 6918, 3107, 1315, 7085, 5399, 2109, 624, 5563, 4042, 4157, 1937, 5623, 758, 495, 2439, 3639, 1276, 262, 1975, 3720, 3825, 1263, 2805, 149, 310, 2918, 3088, 1027, 5745, 2866, 1003, 2263, 2883, 795, 7393, 343, 5153, 6627, 4917, 3952, 305, 1651, 5113, 1427, 3457, 711, 522, 4384, 7002, 1444, 3628, 7, 3681, 1845, 1384, 3413, 1094, 3626, 5103, 4133, 3963, 599, 6359, 7022, 6712, 4036, 1554, 523, 2039, 3131, 5163, 6797, 4017, 3250, 7182, 4168, 2226, 43, 6141, 5844, 849, 6602, 2873, 7120, 6827, 5197, 3473, 6890, 1471, 359, 6728, 5384, 4701, 2580, 6848, 1075, 6433, 5403, 3070, 2247, 6883, 6672, 6327, 6597, 3907, 3502, 6512, 508, 9, 1700, 6370, 354, 3585, 6262, 6650, 3747, 2937, 348, 7250, 5264, 562, 6947, 7202, 1924, 2742, 2485, 733, 1534, 778, 7401, 6020, 4289, 3913, 1530, 3599, 2933, 2424, 3228, 1901, 4874, 6090, 6986, 5438, 2504, 5765, 4684, 6571, 1591, 6474, 4520, 240, 4219, 4908, 6862, 1628, 325, 4139, 5108, 4960, 4645, 4981, 5383, 7212, 4651, 3234, 783, 2588, 6408, 5747, 7169, 5834, 767, 4500, 637, 3261, 6995, 6142, 5018, 703, 710, 4075, 1816, 5757, 6270, 5670, 1677, 5616, 6029, 312, 6977, 6235, 3715, 1129, 3833, 4227, 6556, 6311, 2091, 4389, 4142, 3063, 3918, 7334, 7136, 5540, 2213, 4967, 424, 2018, 5030, 6635, 3437, 3847, 3800, 3032, 4596, 6248, 2683, 2154, 2693, 2043, 6950, 447, 7056, 2990, 660, 1951, 7343, 456, 3561, 5920, 5939, 612, 685, 3703, 5924, 4602, 4865, 6896, 3390, 5822, 3871, 843, 2244, 3830, 429, 153, 372, 4666, 3471, 4363, 7187, 4485, 1171, 2426, 559, 1684, 3230, 6216, 7007, 880, 7148, 2101, 3685, 5791, 1936, 6564, 1417, 6389, 1967, 1081, 389, 2421, 6237, 1331, 4646, 5804, 427, 6330, 5091, 7402, 1245, 2138, 3191, 3737, 4620, 3203, 3560, 6608, 3768, 4883, 1101, 2139, 2561, 761, 928, 4366, 163, 4266, 3795, 2083, 5551, 4290, 3774, 1240, 5404, 1853, 2122, 4785, 4793, 1350, 6750, 5657, 40, 4360, 4086, 6448, 6015, 1298, 3024, 5055, 93, 1403, 7309, 4797, 4891, 1518, 5771, 2433, 7324, 4426, 1393, 4152, 2662, 2134, 915, 1392, 2222, 5885, 5931, 2197, 4877, 4327, 3772, 3760, 5422, 1310, 3500, 7088, 36, 4506, 3716, 2668, 1878, 476, 2042, 4875, 5649, 1564, 2303, 1870, 7008, 3834, 1262, 2708, 1872, 5074, 5293, 2052, 3452, 5925, 4339, 2877, 281, 3438, 4783, 3526, 1294, 1447, 5064, 6927, 300, 3563, 7207, 7374, 2891, 4307, 208, 4971, 7360, 5859, 5962, 237, 6846, 6414, 227, 435, 7005, 1976, 6209, 204, 6906, 6320, 6990, 3566, 5650, 3611, 3521, 1249, 3358, 6568, 5811, 382, 6992, 4436, 4897, 7029, 5723, 5484, 7150, 5552, 4292, 6912, 6133, 5860, 890, 3846, 2435, 1600, 5458, 1355, 970, 6043, 2327, 323, 3974, 1282, 5862, 7164, 4893, 2738, 2859, 463, 2355, 6158, 1825, 1115, 4463, 3542, 5111, 1996, 4329, 816, 7382, 157, 4318, 1243, 5042, 3622, 4780, 5440, 496, 5936, 858, 4884, 6498, 1430, 2146, 1844, 2468, 6023, 6528, 6225, 1421, 6496, 1877, 5990, 1215, 344, 7177, 3948, 258, 7123, 5138, 5011, 4180, 4625, 3099, 1167, 3235, 6503, 5281, 577, 5949, 5656, 2155, 5665, 6203, 2016, 3072, 5801, 596, 1890, 5079, 5638, 1517, 4925, 6208, 3845, 6309, 6154, 2316, 4548, 5913, 7191, 6064, 3237, 1997, 4698, 2107, 1904, 941, 1828, 4692, 6899, 386, 390, 288, 2164, 2084, 3189, 7092, 3581, 1274, 4385, 1177, 1507, 4175, 6515, 5394, 1209, 7337, 6584, 6156, 2374, 3324, 3100, 6933, 2785, 647, 6391, 5792, 3905, 1499, 249, 7119, 3752, 7320, 7181, 2733, 5987, 1960, 200, 2852, 815, 5203, 1288, 4478, 5360, 4762, 1347, 5337, 6108, 2228, 3882, 5148, 3592, 5969, 948, 1207, 138, 6553, 5004, 1222, 2815, 702, 6834, 650, 6946, 1311, 903, 1662, 5989, 6624, 1197, 411, 5236, 2982, 3385, 1879, 5981, 5607, 3886, 3663, 2147, 6113, 5523, 2335, 473, 6293, 3700, 4876, 7336, 4732, 3530, 177, 2537, 3550, 1441, 1454, 6317, 7186, 1633, 532, 5945, 4102, 2770, 3828, 2857, 997, 3374, 4860, 1741, 3450, 3293, 1994, 2791, 6736, 2408, 4006, 4223, 1988, 7035, 5715, 1601, 1476, 4608, 3246, 2371, 5756, 1676, 2800, 5487, 893, 6534, 4727, 2983, 5389, 1098, 7263, 5977, 1256, 2587, 2601, 2026, 6333, 1593, 5507, 4188, 4791, 4279, 5909, 2566, 3524, 3311, 405, 4844, 4300, 1113, 3589, 1117, 874, 861, 6892, 3673, 725, 3320, 3642, 1009, 5402, 1337, 2633, 11, 4524, 5755, 4381, 1710, 0, 3257, 5846, 2594, 3018, 352, 6218, 4496, 6648, 6656, 841, 5545, 1891, 595, 1788, 6453, 3544, 6755, 6975, 4375, 3281, 4472, 6506, 7284, 318, 1887, 6722, 672, 434, 628, 297, 2041, 3838, 6843, 2644, 4209, 5439, 3419, 5326, 50, 6378, 1702, 4921, 7019, 6881, 3710, 2397, 5261, 6538, 5707, 6723, 5521, 3935, 3028, 4639, 5610, 3889, 6684, 5661, 369, 2841, 6577, 141, 1792, 6596, 6012, 698, 5853, 15, 7348, 6949, 6247, 2102, 2427, 4547, 1558, 6539, 6005, 762, 2061, 3892, 2935, 564, 980, 4729, 5078, 1361, 4410, 2514, 7322, 545, 3926, 6781, 6472, 6775, 6019, 6660, 2607, 3404, 4523, 6826, 2142, 3310, 1339, 4441, 6132, 5101, 2912, 5009, 4638, 5917, 5766, 2387, 2907, 931, 569, 4664, 4181, 1944, 3555, 823, 4232, 5107, 2533, 3600, 3110, 1314, 659, 1780, 6739, 3127, 4997, 607, 5580, 2716, 4802, 1716, 4680, 5161, 3349, 4726, 6810, 2725, 5229, 3106, 6904, 5305, 2336, 198, 2497, 7137, 2212, 2786, 2319, 3077, 3164, 4215, 3291, 2308, 6867, 7369, 7247, 1978, 6426, 4498, 6083, 1703, 2106, 5076, 7271, 2456, 6257, 7165, 1203, 3130, 2887, 3523, 1191, 3026, 2996, 239, 4660, 6520, 3532, 622, 317, 481, 6703, 1031, 6818, 2378, 5732, 1486, 2250, 6831, 1045, 5412, 1645, 4173, 3243, 691, 466, 2751, 1778, 1931, 5280, 3961, 3950, 3340, 5963, 6151, 5246, 3956, 729, 963, 2842, 7394, 6746, 4337, 661, 5652, 2819, 7215, 6494, 5283, 3906, 6168, 774, 2034, 7257, 5848, 7251, 5257, 4330, 5938, 4599, 7050, 5465, 3113, 3161, 3976, 5618, 3841, 6541, 2868, 3782, 3376, 4842, 3416, 4774, 1734, 3731, 5595, 5371, 4467, 1250, 205, 85, 689, 3898, 926, 3476, 1609, 2195, 2025, 4155, 5451, 2386, 361, 2268, 4656, 4740, 2621, 6053, 4543, 2103, 887, 3999, 5477, 1907, 5583, 4976, 3855, 428, 2172, 82, 4950, 7071, 3539, 6532, 171, 7276, 1938, 324, 6000, 635, 5168, 7052, 3936, 4814, 1199, 7027, 1708, 3118, 3607, 654, 3141, 4144, 1125, 1407, 7054, 7094, 6613, 1095, 5975, 6163, 4636, 4220, 1154, 7107]
Shuffle2 = [5953, 1856, 3808, 4437, 3152, 6641, 2692, 1991, 3616, 2565, 1820, 6118, 7379, 2900, 1631, 3649, 4611, 3383, 3086, 2180, 707, 7333, 3381, 5201, 6574, 2833, 4115, 5287, 5858, 2888, 3575, 1256, 4563, 6935, 4398, 1342, 1150, 4186, 5947, 6268, 1638, 3704, 2226, 4086, 283, 5967, 7329, 7278, 1801, 4921, 648, 6242, 6285, 1686, 4655, 4962, 2661, 5483, 7186, 4367, 2331, 864, 992, 20, 50, 6122, 5611, 2312, 6259, 6712, 1168, 5788, 4619, 5867, 1592, 3114, 4185, 4814, 3812, 3998, 6538, 4700, 1403, 3521, 2406, 1237, 4523, 3313, 4763, 4894, 6547, 4060, 6870, 6378, 6123, 6625, 74, 4977, 926, 6436, 203, 3218, 4443, 131, 4975, 3844, 1758, 1190, 4579, 5577, 5391, 6413, 823, 7007, 2892, 3766, 4868, 3843, 1765, 1949, 2869, 563, 418, 2884, 6818, 1311, 1739, 720, 4715, 1470, 1578, 5136, 1809, 6950, 6201, 6714, 6279, 56, 1166, 1447, 2993, 5151, 4227, 4144, 2783, 4880, 6466, 1627, 6287, 5823, 1733, 6197, 526, 4141, 649, 6183, 5204, 2990, 6184, 670, 4181, 5451, 1547, 4001, 6738, 600, 3047, 6941, 653, 5610, 2286, 4493, 95, 111, 2304, 5775, 6867, 3646, 2086, 810, 6830, 5145, 4953, 3612, 5705, 5955, 3160, 3733, 1960, 7284, 4650, 2775, 341, 3767, 7110, 4365, 4712, 6461, 6339, 6763, 5297, 7150, 219, 4197, 2002, 6617, 4378, 5939, 3569, 7030, 4967, 1120, 7341, 2757, 4506, 1625, 2407, 4412, 733, 2270, 2372, 3148, 2822, 3479, 5065, 535, 2963, 3668, 3836, 5527, 2828, 3672, 5812, 3473, 270, 6234, 2727, 1231, 7349, 1557, 1543, 3144, 799, 372, 3897, 980, 4008, 2315, 5388, 7299, 4383, 627, 6506, 4280, 5926, 5879, 7191, 777, 4413, 4240, 3962, 1400, 3032, 4810, 387, 6092, 7139, 221, 4971, 4221, 4295, 3132, 4956, 3083, 592, 1656, 3064, 6729, 3045, 5216, 1028, 3763, 1764, 3556, 5749, 6891, 4928, 6434, 192, 4759, 6256, 2473, 2736, 6352, 4855, 3085, 7263, 1586, 5991, 6895, 3063, 6989, 6573, 1006, 4594, 2544, 3137, 3044, 1830, 4945, 5845, 2428, 5409, 2658, 5923, 2465, 2034, 2636, 4475, 7163, 3261, 4385, 894, 5394, 6586, 5770, 2357, 6564, 1813, 207, 6518, 1564, 3666, 3374, 36, 1055, 299, 6728, 389, 3877, 4944, 7003, 224, 2302, 426, 6226, 2750, 6105, 756, 7384, 5578, 5302, 2007, 2185, 4277, 3402, 7364, 6873, 1178, 2177, 4244, 2818, 617, 4757, 7187, 645, 5291, 855, 798, 2730, 5729, 5617, 1777, 1382, 3524, 2561, 4735, 2522, 223, 338, 790, 4851, 6089, 1096, 1677, 3275, 6578, 2483, 2629, 3347, 6314, 2125, 4293, 7125, 901, 3335, 6947, 774, 1506, 6869, 2815, 7039, 4673, 3046, 3252, 82, 3764, 6111, 3394, 5986, 2536, 3315, 7101, 5140, 3236, 6241, 4920, 2672, 3868, 3746, 5983, 5925, 7363, 6170, 3013, 2972, 6329, 5046, 4969, 1046, 694, 5735, 3913, 3431, 7211, 6778, 2257, 2553, 3187, 3778, 6020, 1735, 3175, 4884, 1161, 3544, 6131, 6377, 3540, 4096, 3715, 1953, 4752, 5671, 5482, 5331, 7193, 1811, 5476, 3002, 1071, 6819, 2093, 4406, 1620, 4926, 3693, 3206, 5572, 762, 2048, 1115, 5795, 1408, 3344, 996, 2480, 7023, 6386, 4634, 4282, 4355, 5256, 4976, 5460, 847, 4024, 2769, 988, 5251, 4734, 5128, 7338, 4211, 682, 6443, 1551, 3161, 5196, 558, 1689, 6693, 3416, 226, 5791, 6963, 1061, 1453, 3276, 2124, 2881, 6639, 3421, 4899, 4161, 2203, 356, 6834, 6791, 6455, 7327, 6003, 4722, 7182, 3675, 1905, 6595, 4693, 4330, 5801, 6962, 6159, 909, 128, 2608, 6096, 6021, 1997, 3866, 1054, 7377, 1742, 2432, 2571, 391, 2937, 2944, 110, 2932, 602, 2106, 4457, 2572, 921, 4198, 1655, 2460, 3615, 5787, 2560, 2615, 5614, 3814, 2269, 849, 523, 2905, 23, 1474, 534, 6376, 1435, 1399, 6736, 4983, 6679, 5075, 1579, 865, 5576, 3041, 3026, 1184, 5574, 5005, 2159, 2401, 3750, 2006, 5423, 4298, 5942, 6263, 6286, 4801, 2557, 5761, 5219, 6653, 7346, 4342, 5403, 4628, 4947, 1165, 5916, 4083, 6271, 3095, 2880, 4587, 2194, 2075, 2609, 5733, 672, 6924, 5643, 4253, 614, 6635, 4555, 4256, 6132, 1110, 5132, 278, 6349, 4833, 4194, 1024, 2046, 2163, 2630, 5718, 259, 5981, 7115, 1922, 49, 2021, 2121, 880, 5855, 7237, 1529, 1114, 373, 4200, 5077, 1361, 4446, 4800, 5696, 6772, 6207, 2201, 7147, 1208, 40, 3290, 6038, 5946, 2969, 2032, 5557, 167, 5424, 425, 5354, 1844, 6710, 532, 2700, 3802, 3225, 1270, 7176, 2446, 852, 2569, 6943, 6274, 494, 6292, 3015, 465, 4180, 7218, 858, 4845, 5657, 7262, 2329, 2254, 5493, 4006, 5036, 68, 4656, 6748, 3280, 3585, 6689, 5061, 4084, 3570, 2319, 107, 2392, 3087, 4514, 972, 5704, 7383, 4991, 5416, 7360, 6230, 3401, 3659, 528, 1414, 5864, 776, 3017, 2118, 6886, 1451, 2364, 1175, 2725, 2729, 4685, 2242, 6527, 2784, 5905, 4589, 2442, 6239, 4609, 3899, 3034, 1941, 3758, 5449, 5175, 6429, 3937, 5982, 2919, 4529, 3531, 119, 5135, 6799, 326, 5223, 5999, 1049, 3691, 2984, 2668, 3204, 2883, 1824, 594, 3377, 2405, 2950, 4813, 978, 3765, 1556, 5087, 1961, 3384, 2084, 2155, 1630, 634, 3262, 3984, 5701, 6833, 5780, 4517, 1852, 2662, 1378, 6607, 6273, 122, 6599, 5979, 1766, 6309, 5642, 7223, 7040, 1789, 5494, 419, 1143, 4730, 6666, 3723, 4430, 5101, 4353, 5564, 3986, 7113, 4600, 1512, 6465, 316, 6422, 4050, 7255, 2148, 3755, 281, 3092, 4554, 39, 444, 915, 5726, 3055, 6674, 2799, 181, 1994, 1144, 2272, 1323, 6120, 1657, 6192, 4452, 3069, 4177, 5692, 5639, 5861, 3490, 5322, 1223, 1476, 3761, 4570, 6699, 164, 5154, 4972, 1602, 3960, 2844, 2240, 2247, 1222, 414, 3528, 3019, 4395, 2103, 3024, 971, 4819, 5922, 1575, 3961, 5267, 3510, 6592, 4490, 1428, 4105, 6375, 3718, 4059, 3198, 1366, 275, 3126, 2055, 1601, 6872, 4477, 717, 2589, 27, 6294, 5507, 1582, 3016, 5958, 5893, 9, 2383, 1329, 483, 2476, 876, 366, 1614, 2008, 4816, 3796, 511, 7171, 5491, 758, 3606, 1174, 151, 1041, 7050, 405, 1386, 6182, 1759, 656, 2277, 6357, 6388, 4987, 3302, 946, 1781, 6597, 1204, 4449, 25, 2373, 5183, 6195, 6305, 3076, 5969, 3903, 887, 1315, 1599, 6570, 3005, 2350, 1294, 5621, 6404, 1681, 6704, 3759, 900, 6598, 3726, 454, 30, 5237, 7151, 2895, 4636, 6408, 6909, 2882, 7302, 4206, 2470, 6255, 6808, 506, 7038, 4271, 2182, 1038, 6425, 1066, 2673, 599, 3885, 1916, 447, 3925, 3240, 7087, 6660, 6231, 4042, 1583, 3859, 5324, 1615, 1925, 6446, 3427, 5431, 1790, 342, 3516, 6220, 6813, 7072, 2170, 3454, 1715, 4686, 6644, 1477, 431, 2964, 3655, 780, 5430, 5037, 1850, 7174, 3662, 6039, 536, 7357, 3827, 4583, 2090, 6225, 5533, 977, 280, 675, 766, 718, 2389, 1833, 4258, 1929, 5725, 4720, 4803, 4004, 1818, 5677, 4243, 5375, 2058, 1507, 3711, 6205, 4154, 1699, 4117, 7057, 4155, 4996, 6471, 4337, 490, 6611, 5948, 2368, 150, 4644, 6532, 3108, 59, 1033, 5734, 7054, 6877, 3115, 2734, 969, 1911, 5684, 6984, 3070, 3605, 94, 4946, 1271, 831, 5820, 3292, 3170, 214, 4082, 6497, 4925, 4922, 3239, 2751, 2534, 5014, 4902, 380, 5752, 1452, 510, 67, 2390, 6322, 2837, 3716, 3724, 1878, 6215, 5026, 4145, 6992, 3324, 3274, 951, 4387, 5634, 5008, 3408, 6495, 3579, 6308, 4170, 4223, 802, 3670, 4448, 5708, 5987, 1219, 5648, 1616, 4504, 44, 2827, 6291, 4935, 2864, 3919, 4635, 6196, 3601, 3285, 3550, 2901, 7119, 233, 4229, 651, 3014, 5585, 3629, 1722, 2083, 57, 3403, 3883, 2788, 833, 2275, 1821, 760, 6670, 4938, 2253, 5218, 3954, 3928, 6967, 5457, 1463, 2671, 4496, 3820, 4257, 4130, 108, 3697, 5047, 3190, 3395, 1122, 1194, 3637, 6435, 3793, 4876, 3036, 7052, 7131, 1030, 4527, 7367, 6961, 1211, 4241, 4861, 4640, 5532, 2234, 2986, 4525, 5436, 1884, 3360, 4248, 3183, 6764, 2463, 2050, 1073, 6042, 5228, 6360, 1798, 2126, 2061, 6845, 4012, 5699, 4433, 3009, 2863, 1988, 5741, 3314, 6648, 5350, 331, 3442, 583, 2335, 3174, 6082, 6804, 744, 1177, 2453, 3107, 1683, 1074, 7398, 573, 4575, 4883, 4307, 4094, 7082, 236, 4545, 5289, 6045, 138, 884, 5756, 4789, 550, 6832, 3955, 691, 215, 2410, 4869, 2271, 2712, 854, 6488, 565, 1422, 488, 4304, 4284, 4097, 5053, 6749, 7308, 6844, 2490, 5133, 661, 4230, 346, 801, 3917, 5521, 7028, 7046, 7226, 1370, 218, 5655, 587, 3153, 2958, 5707, 5567, 5902, 2052, 1965, 3872, 5885, 958, 5508, 2426, 7197, 4352, 4405, 6134, 2821, 5066, 4847, 3562, 2087, 3491, 3445, 1275, 1360, 2313, 7145, 1043, 4660, 5645, 5715, 4333, 5831, 1970, 6762, 7217, 3343, 1611, 1446, 5390, 2150, 5453, 4035, 3186, 2722, 5242, 1328, 7070, 6326, 3270, 4104, 124, 5019, 2181, 2025, 1866, 3508, 7060, 4431, 3818, 1127, 3596, 4133, 1286, 4259, 7000, 2584, 2555, 2292, 333, 6929, 6932, 2064, 4750, 3502, 2778, 3142, 3357, 7382, 2968, 1130, 6905, 2665, 3891, 5688, 5998, 6593, 5798, 182, 196, 3178, 7332, 3910, 5445, 2009, 4839, 2259, 928, 2151, 4674, 3362, 437, 5150, 152, 1500, 2680, 3952, 7222, 1448, 6723, 33, 545, 3944, 1233, 7290, 1413, 1346, 6577, 2916, 3035, 6035, 3208, 1437, 5446, 5914, 2683, 3889, 5060, 1731, 7221, 2107, 6994, 6944, 6459, 2261, 2308, 6146, 1934, 3389, 4340, 339, 3627, 590, 6350, 3493, 4968, 5563, 1920, 3434, 1576, 3301, 2825, 5450, 1449, 3882, 2754, 2870, 5137, 358, 1521, 4237, 7172, 6433, 6219, 2289, 6999, 2739, 3436, 3471, 4710, 2634, 4458, 4470, 2795, 2685, 2076, 6156, 4093, 1472, 2430, 5941, 6015, 5653, 6987, 1596, 2000, 3452, 6078, 2497, 2625, 6863, 6469, 32, 2934, 715, 737, 5067, 1224, 5272, 520, 2820, 4098, 7195, 2613, 5512, 2262, 1348, 4897, 4544, 4690, 6885, 4564, 2726, 407, 6632, 386, 5584, 3879, 2236, 3597, 1432, 5009, 3487, 4255, 4328, 4708, 755, 1062, 4601, 2941, 249, 2995, 127, 1931, 2213, 2491, 46, 2651, 1390, 6463, 3684, 3640, 2982, 5896, 2387, 3105, 5325, 6354, 3333, 1842, 370, 4586, 3432, 6064, 2719, 3671, 5001, 6911, 4862, 3429, 4114, 5345, 3390, 5543, 2758, 795, 6720, 378, 1019, 6124, 4537, 2771, 2100, 1395, 2845, 3232, 1309, 3340, 6024, 650, 4824, 3418, 3610, 3028, 5241, 6198, 47, 2597, 2846, 6848, 4131, 743, 1746, 6718, 6204, 1076, 1288, 7229, 2244, 6077, 481, 293, 1443, 209, 2981, 3555, 7343, 6324, 5763, 7036, 6175, 6817, 325, 5198, 1089, 4701, 6490, 433, 6282, 1116, 613, 6493, 2760, 2409, 2998, 81, 4602, 1828, 1417, 4189, 4013, 5963, 5486, 2132, 910, 5700, 4687, 4645, 2018, 660, 629, 5881, 7031, 5033, 2858, 1261, 2085, 5501, 4124, 1788, 3756, 3781, 7401, 3103, 4106, 517, 4658, 1870, 1078, 1156, 3717, 5841, 469, 2456, 2640, 350, 2325, 6061, 3081, 1903, 697, 1619, 979, 5971, 4959, 1434, 2037, 3785, 2809, 1641, 6568, 1436, 1553, 1756, 2381, 5007, 4765, 5022, 726, 6472, 365, 1180, 2513, 767, 5826, 2556, 1736, 4210, 6780, 260, 7025, 2141, 7315, 778, 7390, 6726, 3519, 4999, 7265, 5587, 5260, 7024, 345, 5410, 7059, 2819, 2065, 5794, 357, 1279, 2028, 540, 3212, 4311, 2898, 3780, 1697, 2063, 5471, 305, 5920, 6796, 5559, 6178, 7286, 2891, 1901, 2767, 2001, 1103, 6432, 4745, 7369, 1221, 1375, 4054, 1827, 566, 5049, 3209, 3966, 382, 3728, 623, 1273, 2841, 6667, 4359, 2573, 4262, 3523, 6711, 1587, 7248, 3678, 3565, 2317, 2703, 3441, 1662, 55, 3298, 6802, 4807, 2278, 3288, 5281, 5627, 1671, 4440, 4482, 3769, 698, 2737, 4381, 2521, 3574, 80, 1668, 1525, 2189, 1172, 936, 4573, 2164, 5393, 2089, 6920, 6571, 6525, 6397, 2520, 4403, 5258, 4392, 279, 4918, 154, 3530, 4691, 6407, 3328, 1696, 7285, 6866, 3003, 3272, 897, 5143, 835, 1021, 6824, 467, 5637, 6623, 6029, 1982, 4427, 3535, 3104, 2191, 3306, 6106, 6629, 5638, 5144, 114, 5167, 1092, 1306, 463, 702, 6059, 4334, 497, 5850, 6529, 3168, 3307, 3391, 1475, 492, 3392, 1387, 5628, 1139, 605, 2701, 5619, 1439, 6851, 1499, 4236, 1070, 2593, 7225, 3964, 804, 4995, 3134, 1232, 1056, 3419, 3451, 4692, 228, 542, 710, 3078, 5825, 1081, 5130, 5608, 6018, 6541, 4842, 4943, 1290, 4796, 826, 1283, 5437, 1757, 185, 3289, 1606, 1825, 5539, 190, 5807, 1015, 7092, 220, 4762, 6744, 6665, 3536, 5747, 5020, 4132, 5952, 7404, 1702, 5829, 3974, 2763, 2351, 1268, 5949, 585, 6163, 2228, 853, 7309, 1738, 7085, 4019, 2045, 243, 1332, 5848, 143, 1540, 5720, 6624, 4160, 3624, 5853, 6843, 3072, 4341, 579, 5516, 3687, 7076, 581, 1679, 502, 6921, 2399, 5702, 1331, 2158, 5340, 3138, 6253, 4727, 3439, 5830, 5303, 6479, 3682, 5929, 1213, 3902, 3010, 6936, 3346, 412, 1815, 113, 3246, 398, 6341, 3215, 6504, 5537, 1855, 4014, 7055, 1752, 2960, 4643, 5528, 4038, 142, 430, 5098, 2078, 2036, 10, 5153, 4929, 4065, 3770, 1372, 163, 3805, 3799, 4875, 6505, 3030, 7213, 1900, 5384, 3414, 2417, 4468, 5822, 5443, 327, 3366, 3146, 5750, 5317, 2297, 3185, 3165, 7098, 135, 4384, 3526, 3456, 5100, 796, 2229, 7166, 1992, 4016, 4442, 1485, 4756, 6067, 4140, 1787, 5418, 2408, 646, 1710, 4454, 7365, 1498, 575, 561, 750, 6925, 3893, 5063, 2646, 2738, 4841, 1519, 3604, 3623, 1593, 5104, 5573, 696, 6894, 6954, 1186, 4460, 1202, 5915, 7118, 3323, 235, 267, 3537, 1896, 5305, 6467, 4518, 2115, 2733, 1973, 87, 4565, 3553, 2451, 6966, 1379, 422, 7376, 2957, 3481, 6899, 6481, 5571, 2872, 1720, 4778, 2326, 4409, 3406, 5236, 1317, 4320, 5368, 5849, 4606, 4612, 584, 3399, 3091, 6202, 3712, 1628, 1912, 2104, 4483, 420, 2276, 6613, 200, 2397, 6216, 1253, 7200, 4815, 5803, 343, 693, 5549, 1398, 4668, 5740, 889, 4162, 5961, 1546, 779, 2287, 3784, 2190, 2015, 553, 6790, 589, 217, 4103, 1636, 193, 3926, 6637, 3117, 402, 5919, 5266, 3054, 3122, 3989, 7288, 4461, 5731, 2647, 6190, 626, 3654, 5489, 2938, 7077, 1276, 5298, 6934, 3121, 6031, 2056, 1684, 1867, 3855, 6244, 7330, 1084, 1425, 4581, 315, 5309, 4267, 2943, 392, 3089, 4864, 975, 3309, 5459, 2581, 2909, 1926, 1964, 3133, 51, 1027, 1040, 3699, 6074, 3080, 6520, 1969, 7264, 6437, 3857, 4751, 7095, 3341, 4274, 6126, 668, 1347, 3299, 5474, 5736, 6685, 3870, 4000, 1318, 3050, 1393, 6138, 383, 5721, 5117, 2596, 3567, 3000, 5156, 5884, 291, 4742, 2669, 7257, 1977, 3149, 2024, 3752, 6734, 4533, 5605, 724, 3990, 1464, 5956, 4122, 5131, 5495, 6892, 5586, 4301, 4761, 1923, 7097, 998, 6491, 29, 6771, 4297, 6327, 974, 5525, 714, 7199, 5329, 4005, 6050, 4159, 17, 3166, 4965, 211, 4041, 197, 4099, 194, 7251, 4087, 2945, 3520, 2724, 4410, 4526, 5968, 5148, 4678, 4522, 4377, 4358, 1569, 4166, 2478, 7159, 3372, 3424, 3387, 1753, 2951, 4414, 478, 4605, 1806, 5161, 5408, 6276, 5843, 1971, 970, 816, 1839, 4113, 4254, 7091, 2419, 878, 7207, 7205, 6605, 6838, 805, 4985, 5364, 4239, 621, 6460, 7345, 3927, 6249, 1527, 4325, 1698, 5590, 4306, 6441, 5917, 6985, 1581, 4076, 6079, 3774, 4749, 1300, 7004, 7088, 3748, 1950, 7108, 3527, 4729, 1678, 63, 6991, 6265, 4675, 6127, 3181, 2199, 4934, 6161, 6211, 7268, 1327, 3787, 3242, 5680, 3708, 3378, 5842, 5327, 1445, 2702, 6816, 2591, 4071, 5758, 6296, 837, 5012, 6839, 2077, 2380, 3217, 5698, 3539, 6371, 4225, 3534, 3370, 4805, 5518, 6676, 266, 7049, 4744, 5017, 7061, 2279, 518, 5666, 4436, 2030, 1162, 1928, 2454, 4879, 3916, 3677, 3155, 6703, 3477, 4479, 3830, 2129, 6881, 1287, 6615, 7037, 2320, 1053, 208, 741, 803, 7100, 1234, 1280, 3476, 2939, 6661, 1659, 2471, 2666, 6535, 4557, 1704, 1930, 5502, 1721, 3714, 5514, 4394, 5625, 1894, 7393, 986, 3052, 5962, 2718, 1450, 2994, 2345, 2241, 5207, 3053, 3971, 2582, 5945, 846, 2924, 2764, 2027, 2060, 117, 5937, 4889, 4499, 1570, 4455, 866, 3494, 2396, 3911, 1444, 3489, 2282, 1034, 5901, 3953, 7096, 4064, 2468, 7136, 3996, 4891, 4820, 6691, 1407, 246, 3482, 3790, 522, 2603, 4498, 2386, 2657, 4047, 258, 5594, 7194, 5766, 7202, 5429, 5497, 6191, 625, 1740, 485, 3622, 4852, 2206, 6953, 6534, 3022, 101, 6708, 4652, 2902, 1369, 7361, 5553, 923, 6119, 7156, 5295, 6337, 5779, 3786, 6680, 7006, 5871, 4463, 1808, 1955, 6502, 2747, 3650, 6682, 5630, 5623, 3816, 5875, 5601, 5439, 319, 5892, 6785, 3251, 2580, 6782, 5529, 4484, 4794, 6009, 2486, 1688, 2804, 3829, 6634, 684, 321, 52, 179, 4184, 2391, 5974, 1154, 4178, 6054, 1091, 4321, 678, 5990, 6566, 2427, 4397, 5300, 5097, 1293, 2492, 730, 6696, 5362, 2663, 4196, 5074, 2437, 4592, 6058, 54, 1119, 1189, 4108, 7026, 3443, 516, 6811, 7245, 622, 4039, 5772, 2165, 2402, 7259, 2746, 5773, 5248, 3506, 773, 1077, 2732, 4283, 5622, 298, 706, 493, 474, 7314, 808, 3256, 3935, 400, 4954, 5560, 6512, 6235, 3260, 388, 2578, 6384, 562, 2617, 1017, 3957, 1075, 4850, 6116, 109, 294, 5307, 1281, 2101, 268, 6618, 598, 2933, 3545, 340, 5120, 6800, 5199, 6214, 2708, 4386, 1171, 1518, 6946, 721, 2154, 771, 1968, 2040, 3073, 3738, 2989, 983, 1424, 882, 7374, 4680, 3890, 3831, 1691, 6753, 3837, 1986, 7019, 4630, 91, 2856, 1624, 2794, 362, 3295, 6840, 4721, 1892, 3474, 3568, 3558, 5545, 6117, 429, 5935, 7228, 274, 3632, 1989, 560, 2670, 364, 1067, 4174, 2133, 7258, 3976, 1163, 7249, 5669, 2187, 461, 1047, 4077, 4931, 5002, 1589, 2612, 5165, 5603, 1730, 5222, 5591, 7272, 5479, 6185, 872, 6781, 5536, 4467, 6344, 2445, 713, 7386, 4924, 2655, 2171, 7321, 5276, 1316, 184, 5096, 6558, 449, 3351, 1761, 7073, 6275, 2108, 3707, 544, 5279, 273, 6494, 5996, 3850, 6278, 1460, 4090, 3557, 1741, 5765, 6167, 2416, 355, 6928, 1520, 5124, 2361, 1979, 5425, 2686, 3169, 3321, 2210, 6323, 2681, 6102, 657, 1985, 1857, 1548, 3679, 1567, 2947, 1197, 1555, 711, 5685, 6188, 5105, 2466, 2855, 1719, 1868, 7044, 5665, 7215, 867, 3226, 1660, 806, 1732, 2053, 1734, 477, 5076, 3200, 738, 2874, 7048, 13, 3968, 3552, 952, 2899, 1167, 6567, 6110, 4560, 6980, 5540, 2238, 7358, 1035, 2120, 4391, 1690, 5084, 3241, 567, 4214, 7106, 64, 2696, 5500, 1984, 1706, 4345, 5379, 3329, 6027, 4286, 6391, 4990, 1153, 4447, 5334, 2996, 2660, 6044, 1865, 5401, 7320, 5176, 3642, 2614, 6871, 5857, 2773, 6510, 21, 1577, 1117, 361, 7291, 41, 1060, 2291, 1142, 238, 871, 4882, 5155, 5355, 2010, 1729, 5924, 3008, 4418, 5274, 5054, 4183, 6147, 7326, 4667, 5048, 1195, 7395, 5887, 7170, 1297, 4588, 2197, 3847, 6559, 2323, 6060, 6919, 6874, 5262, 7307, 1207, 3300, 3417, 5209, 344, 4272, 2359, 4142, 1241, 6645, 4372, 3607, 1775, 3788, 6898, 6319, 6406, 907, 6630, 5467, 464, 1292, 4932, 6978, 121, 4202, 7234, 2283, 7018, 6427, 3266, 3641, 1339, 3705, 3791, 5663, 2448, 4528, 3522, 5618, 6484, 48, 6417, 2628, 1396, 7134, 4290, 3918, 5021, 6300, 1881, 4033, 2779, 2627, 6668, 6616, 1124, 6773, 4118, 3173, 5706, 4444, 7117, 4422, 6960, 2305, 1838, 5226, 1544, 6133, 4011, 72, 7391, 4139, 6825, 6090, 4848, 6821, 6542, 5697, 2676, 4607, 1007, 4148, 1106, 5319, 1051, 1537, 5041, 4585, 1693, 4679, 1957, 5742, 6707, 3143, 3245, 5542, 6048, 6626, 1674, 4102, 1200, 4079, 3368, 6750, 6658, 4112, 1488, 891, 6519, 6754, 1550, 2109, 4472, 5091, 4782, 2954, 289, 5583, 5646, 6955, 24, 5078, 886, 821, 3771, 2412, 5440, 4153, 97, 7282, 2927, 2388, 4401, 4973, 4706, 5044, 6270, 3745, 2633, 2393, 4906, 3124, 6330, 1633, 2731, 5913, 120, 3651, 761, 2849, 2690, 415, 6746, 3156, 3007, 3011, 3783, 6419, 3264, 6343, 2802, 363, 1692, 1217, 3600, 4532, 3062, 1000, 5339, 368, 5897, 966, 4425, 6331, 3652, 570, 2123, 7165, 541, 673, 2105, 3287, 5335, 917, 7252, 6747, 883, 3982, 1978, 7359, 789, 2745, 5513, 6684, 6473, 6557, 7351, 2239, 1853, 1158, 1192, 7342, 685, 7318, 4684, 6887, 4173, 5157, 1510, 5900, 6880, 4597, 4439, 6619, 2130, 191, 3312, 3278, 5221, 5613, 2072, 3216, 4832, 2999, 2327, 5768, 4951, 2219, 6721, 3305, 5710, 3348, 1322, 480, 3277, 4233, 2583, 6002, 6402, 5134, 5517, 1059, 4354, 1769, 1320, 2341, 5004, 79, 489, 6614, 5869, 5776, 7016, 838, 1205, 3956, 5336, 2530, 6650, 84, 3547, 3141, 5739, 7283, 4326, 620, 3018, 5444, 5693, 5174, 1179, 962, 6789, 4829, 552, 3193, 5475, 4657, 3303, 1296, 4308, 198, 6186, 4970, 137, 1001, 3267, 3184, 4608, 2862, 3863, 4476, 5837, 5268, 5631, 5282, 1388, 4617, 7021, 2516, 3853, 4292, 6025, 3744, 3430, 6129, 7280, 6081, 548, 5598, 1155, 3653, 1016, 788, 2890, 3577, 4030, 4316, 3978, 7042, 2894, 2575, 6317, 631, 5103, 2830, 1880, 5570, 1561, 1228, 3422, 5834, 468, 1312, 303, 4212, 4531, 3191, 7366, 26, 5907, 4949, 6083, 18, 6022, 666, 3339, 3515, 6982, 2966, 7210, 877, 3789, 7122, 4604, 5013, 1987, 5, 6036, 2814, 2562, 3231, 2306, 827, 1023, 141, 2157, 1899, 3268, 1079, 5870, 5606, 5626, 5886, 5349, 7065, 3109, 1133, 2458, 832, 5435, 665, 4260, 38, 1664, 436, 5908, 6565, 5432, 1330, 2447, 4134, 4863, 3060, 432, 2542, 783, 2904, 4136, 6774, 133, 7169, 1225, 3469, 5534, 3354, 2976, 5277, 71, 2160, 2474, 7354, 1362, 1626, 5016, 912, 5468, 1255, 201, 6137, 4167, 2092, 3102, 2857, 1409, 4702, 3310, 3129, 6258, 2074, 4075, 3449, 3505, 6135, 5712, 5561, 3676, 2167, 2352, 6364, 3795, 2768, 2102, 360, 3858, 3674, 504, 5667, 4538, 1259, 6649, 5802, 3683, 5727, 817, 4653, 4709, 2772, 5652, 4315, 254, 848, 4915, 3498, 1392, 4556, 3182, 2677, 4164, 7235, 1711, 3618, 3988, 5244, 1107, 70, 6221, 722, 1612, 2706, 3037, 7271, 2394, 6347, 3199, 1966, 253, 2545, 4524, 530, 3316, 6526, 2587, 3929, 4018, 1102, 4450, 100, 4492, 1864, 2265, 2911, 4215, 4957, 6688, 3749, 652, 2579, 7266, 2179, 6028, 669, 4521, 1461, 248, 1607, 4966, 2599, 687, 2653, 5116, 582, 4500, 1762, 1487, 8, 6859, 6531, 2433, 4056, 1011, 3594, 250, 5817, 6631, 1773, 6298, 1181, 2175, 3804, 2362, 7001, 6468, 3777, 6150, 5716, 6522, 3327, 336, 2955, 1367, 2434, 3518, 5876, 6907, 4120, 4743, 2068, 734, 1501, 5265, 3779, 2871, 1148, 2626, 2574, 5197, 4288, 4603, 5254, 5686, 6896, 5358, 6579, 5415, 4788, 1462, 3203, 3065, 5683, 2826, 2336, 935, 5382, 3071, 1782, 3338, 6130, 4007, 1650, 2812, 5178, 3448, 89, 1037, 4804, 4346, 4213, 5744, 2285, 981, 662, 5030, 3583, 3118, 399, 5682, 104, 5668, 4669, 3033, 1980, 5880, 6478, 6930, 1635, 4963, 3101, 1680, 7335, 6320, 5709, 7389, 2384, 6189, 1459, 2592, 2246, 3852, 460, 4508, 3912, 3698, 4191, 1723, 3807, 2887, 6223, 4952, 2288, 812, 692, 2096, 3760, 4711, 5185, 1605, 4854, 4417, 6937, 5172, 69, 5243, 2859, 3269, 5810, 4312, 2495, 4441, 1252, 6850, 6369, 2043, 3318, 2033, 6164, 4222, 3342, 1522, 88, 5899, 7033, 4811, 4942, 1648, 6257, 1786, 7178, 354, 3068, 158, 2082, 5743, 6046, 4220, 1748, 4866, 1258, 1188, 4892, 3412, 927, 2567, 1913, 3809, 6442, 4055, 1313, 3621, 2195, 6514, 2099, 4726, 3233, 7047, 7066, 1887, 144, 4584, 1726, 2482, 4404, 2790, 3729, 5099, 4299, 4818, 1164, 3700, 7167, 5851, 11, 116, 4519, 1220, 913, 1917, 2959, 1014, 115, 5888, 3775, 4494, 5392, 5928, 6040, 2829, 5031, 689, 2113, 2029, 1490, 2062, 4567, 3580, 4914, 5246, 752, 984, 3933, 4694, 7141, 7388, 6149, 2330, 43, 3112, 1647, 3029, 6852, 6306, 4663, 3082, 1908, 4125, 5064, 6390, 4911, 4649, 7071, 2873, 2554, 1946, 7317, 2501, 3259, 4085, 5122, 6563, 3630, 770, 3727, 1250, 6758, 1713, 1068, 7107, 1565, 4092, 993, 6144, 296, 2912, 1039, 7010, 7161, 5596, 6069, 3048, 4073, 3720, 6155, 255, 374, 574, 6916, 5227, 2865, 7109, 4505, 212, 1792, 3061, 3257, 842, 6515, 5800, 3970, 5461, 5711, 5448, 1716, 6575, 6262, 4285, 2443, 6056, 4812, 5713, 2717, 2, 6793, 5927, 1173, 4780, 940, 4733, 5607, 1245, 2705, 639, 2903, 4647, 7078, 1169, 1707, 3331, 6540, 4793, 3113, 5786, 4374, 139, 5127, 4732, 2743, 1025, 809, 4912, 3349, 4474, 6084, 3284, 5160, 5673, 732, 6171, 2539, 6636, 3876, 5032, 4923, 4246, 4462, 85, 1065, 6847, 4707, 4927, 4802, 5473, 2042, 3757, 6462, 3747, 7241, 2067, 2237, 1478, 112, 4275, 1639, 6445, 4107, 1239, 4654, 3202, 5094, 6801, 6815, 6739, 2080, 2263, 4775, 7313, 73, 4242, 317, 3542, 7081, 5792, 2623, 1598, 222, 5371, 4109, 6097, 4903, 169, 2699, 735, 3972, 863, 4979, 4540, 5315, 3177, 7020, 3669, 4034, 3908, 3854, 5454, 6602, 1993, 6237, 4172, 98, 5814, 4853, 3735, 1191, 2707, 4058, 3365, 5343, 3250, 2235, 424, 705, 4265, 5602, 1473, 2620, 2273, 6511, 2489, 6998, 323, 6724, 2091, 1416, 5238, 618, 1005, 5815, 2256, 5421, 2011, 4126, 5793, 6315, 818, 2400, 6594, 6555, 1026, 5039, 4263, 4590, 312, 3709, 3222, 3248, 4507, 3157, 4074, 3543, 5988, 5523, 3832, 1772, 2774, 5085, 4009, 6725, 3322, 6975, 704, 3945, 2664, 1269, 822, 1295, 1535, 1343, 28, 4309, 5361, 5568, 1214, 6609, 3470, 3617, 6229, 2137, 5333, 7236, 2847, 819, 1954, 1090, 4269, 997, 7075, 3461, 7177, 4821, 6338, 6901, 6582, 1744, 2366, 5784, 6108, 2450, 1227, 3587, 7135, 2110, 60, 4375, 5111, 1848, 1849, 2469, 3696, 6856, 4281, 1709, 4021, 1541, 6420, 6651, 6695, 242, 2114, 3188, 7243, 466, 3639, 2014, 2208, 2766, 3856, 2807, 6737, 5714, 4997, 4400, 2255, 4078, 4856, 4760, 2153, 2691, 840, 5813, 3282, 7214, 3943, 6995, 3754, 7149, 3320, 205, 6001, 359, 7303, 6251, 6969, 7105, 6213, 4026, 6439, 7084, 3189, 5378, 5068, 65, 4870, 941, 5531, 2780, 4974, 1105, 2301, 180, 455, 5661, 3283, 5465, 6974, 5777, 6179, 3794, 4562, 1771, 5250, 1568, 6295, 3382, 4795, 3725, 5745, 6562, 2176, 1663, 5672, 6858, 6047, 6181, 1807, 6301, 5316, 5816, 4057, 5164, 3253, 2710, 4149, 5320, 3352, 4032, 1538, 3367, 5102, 7189, 1355, 4344, 919, 5214, 719, 2639, 519, 5852, 1338, 6908, 2715, 6328, 2594, 1104, 2940, 1841, 1058, 1335, 2498, 7204, 5271, 1421, 1703, 3636, 4748, 129, 6498, 1701, 5112, 4798, 186, 3459, 5477, 2415, 5548, 2215, 4438, 5147, 4776, 1776, 2674, 6788, 4163, 7146, 5554, 22, 607, 6154, 3878, 5050, 644, 6805, 3098, 4676, 6252, 3644, 1876, 6139, 1814, 2886, 4516, 1147, 4, 2023, 925, 4651, 1099, 4566, 5463, 2249, 5866, 3846, 985, 5095, 6539, 3992, 3388, 5433, 4741, 1658, 2094, 6401, 5894, 5113, 1536, 4978, 5530, 7104, 2854, 2438, 2012, 1958, 5719, 2585, 955, 6633, 3194, 2920, 4632, 2546, 1394, 6589, 1004, 937, 2038, 1087, 262, 7231, 3294, 7256, 375, 5138, 2097, 1542, 102, 4147, 3874, 3192, 6588, 4100, 2528, 2472, 171, 6438, 5356, 288, 4204, 2374, 3369, 6803, 136, 7305, 2761, 5377, 2913, 2039, 851, 1642, 3332, 6897, 2980, 3385, 5580, 7133, 4843, 5404, 7230, 3731, 712, 5290, 2138, 991, 2413, 4992, 1572, 861, 6673, 2192, 7069, 3946, 2682, 175, 5921, 1924, 7029, 6604, 782, 1580, 4646, 1267, 6854, 1125, 2309, 5419, 3090, 5806, 2019, 736, 5509, 6959, 5737, 5311, 904, 4127, 4116, 3249, 3100, 768, 4520, 3875, 5018, 5764, 3838, 3549, 2258, 5359, 1212, 785, 1251, 6948, 987, 3334, 6949, 3120, 7198, 1206, 3475, 130, 3353, 4719, 335, 5347, 3559, 4572, 1669, 6906, 2711, 401, 1264, 4268, 4904, 2532, 1240, 615, 539, 2342, 3433, 1308, 1349, 3959, 2697, 4799, 1523, 3862, 4429, 791, 4828, 7254, 31, 6751, 709, 636, 5629, 3803, 6583, 2217, 7232, 950, 6423, 4893, 1608, 4909, 7239, 2606, 2047, 7227, 140, 4830, 3234, 939, 4165, 7387, 5312, 1640, 2204, 4379, 7336, 3772, 3734, 2810, 6669, 2971, 5992, 4783, 2225, 3941, 2178, 7002, 2667, 1383, 4753, 5562, 6795, 6797, 2421, 5538, 6393, 1353, 1467, 7009, 6543, 4823, 4887, 6507, 4998, 2088, 6280, 2921, 1816, 225, 1907, 2248, 7275, 982, 3904, 3379, 1022, 7142, 5397, 859, 2867, 4349, 438, 4368, 5863, 5633, 6094, 6879, 1310, 4620, 2749, 6254, 3228, 1257, 423, 6787, 5285, 895, 2079, 6971, 703, 498, 5283, 6203, 3179, 4088, 1112, 2139, 5624, 3398, 1291, 610, 2507, 989, 6227, 5380, 4027, 6297, 1861, 5171, 4111, 2973, 1042, 2069, 2295, 5510, 1902, 6318, 3619, 3888, 4110, 4080, 5650, 4302, 1249, 7353, 297, 2852, 5649, 153, 5310, 6381, 2016, 3969, 892, 7064, 4836, 3464, 6136, 6681, 5245, 965, 2523, 2741, 995, 6552, 2479, 6826, 7116, 3195, 4332, 1094, 2621, 6085, 2695, 7297, 6957, 5280, 699, 6938, 1157, 5552, 6697, 4238, 716, 6868, 5641, 6449, 538, 4485, 6356, 4849, 2455, 406, 601, 7281, 4916, 7196, 4599, 6713, 3529, 4834, 4025, 5833, 2622, 5847, 1129, 3914, 6366, 6836, 2059, 4980, 5220, 4201, 6109, 4831, 4251, 938, 2205, 3051, 417, 3075, 4786, 5783, 2245, 524, 4677, 4478, 6489, 1013, 4426, 6584, 5338, 5466, 1834, 486, 6334, 2337, 4629, 499, 4511, 7402, 7063, 2202, 349, 7381, 6200, 5644, 6095, 3541, 7112, 3271, 815, 2321, 3921, 176, 769, 1705, 4960, 6382, 6290, 4043, 4329, 7175, 6453, 421, 5202, 4543, 4790, 7405, 148, 126, 4784, 106, 6224, 1193, 1653, 2840, 7246, 5045, 1324, 6560, 1810, 6846, 503, 6166, 83, 6741, 6828, 1665, 2813, 5455, 6243, 2755, 6755, 3975, 4648, 3894, 6063, 311, 5411, 7127, 4343, 6030, 5107, 5670, 6087, 4561, 3140, 3483, 6675, 2805, 973, 3428, 1009, 2552, 2200, 5547, 2607, 3056, 5555, 6355, 6576, 6608, 2564, 1340, 757, 529, 3896, 1247, 5057, 5976, 7162, 450, 4878, 4048, 4552, 3965, 7287, 7216, 395, 6700, 1072, 4907, 5003, 329, 7045, 3861, 5478, 7289, 5351, 2420, 5212, 1136, 5062, 2135, 3220, 1591, 3949, 1134, 1651, 2519, 4361, 1974, 2543, 3942, 324, 4910, 5809, 4150, 3263, 2875, 3171, 957, 559, 3207, 603, 6141, 1405, 7090, 3940, 5177, 3116, 500, 896, 4867, 3400, 6363, 2709, 5306, 1673, 6965, 4168, 1737, 3864, 3560, 1610, 5458, 4671, 1303, 3930, 5755, 2371, 2704, 62, 3548, 1410, 6926, 6864, 1532, 3350, 4937, 1135, 263, 4865, 5363, 829, 6499, 2511, 1854, 2850, 6145, 6451, 836, 4591, 5544, 2923, 6931, 6165, 4578, 5993, 1151, 3297, 6716, 4838, 4121, 7298, 2346, 427, 90, 4703, 1804, 5511, 5828, 4874, 247, 7212, 1915, 103, 3551, 4837, 1196, 4123, 3396, 5616, 6996, 2558, 1644, 2298, 4781, 4473, 204, 2221, 6620, 5717, 942, 1285, 3538, 3967, 6483, 5234, 6503, 7279, 3304, 118, 6247, 700, 5200, 5261, 4704, 4232, 6209, 3106, 6100, 313, 6654, 5748, 569, 6358, 663, 4003, 1381, 5485, 146, 2131, 3578, 2602, 2162, 4716, 948, 7034, 5970, 6365, 4791, 4633, 4901, 6153, 6194, 1700, 3164, 6289, 1326, 2499, 1831, 5168, 3027, 482, 6238, 5836, 1694, 5000, 2174, 2926, 4188, 195, 6313, 6798, 6385, 5360, 1418, 6888, 5273, 6470, 3598, 5365, 1126, 14, 2568, 961, 2142, 7397, 6023, 5304, 2477, 2525, 5462, 6642, 2355, 7099, 4846, 1376, 5595, 276, 4247, 1401, 5229, 6007, 6193, 591, 2041, 3973, 2116, 4774, 6414, 5889, 6591, 6732, 2942, 5824, 1337, 6523, 5940, 1263, 3839, 7224, 6410, 3376, 2098, 231, 3279, 5936, 6508, 505, 3647, 1002, 295, 1952, 2781, 6398, 7260, 5052, 4252, 3093, 1937, 2915, 5984, 1918, 1469, 501, 3845, 6766, 870, 664, 6810, 845, 1010, 2250, 1438, 3546, 5751, 5387, 2876, 4028, 6551, 898, 5505, 1482, 2073, 1282, 6646, 5840, 3450, 6062, 556, 397, 2688, 7304, 5093, 3871, 4129, 3938, 2641, 2066, 6657, 2931, 2801, 3842, 7347, 2687, 4574, 929, 2877, 5015, 1095, 7017, 2299, 5313, 173, 1936, 2462, 893, 3462, 7012, 3661, 6752, 727, 6051, 1199, 1890, 6722, 1927, 5407, 4626, 2435, 676, 2935, 7312, 765, 6627, 1305, 1685, 1149, 3265, 2953, 4347, 471, 2314, 2537, 3564, 1406, 4806, 6284, 725, 3420, 5903, 2533, 7173, 1595, 1779, 881, 5072, 953, 1513, 6053, 6951, 1860, 35, 1996, 5746, 5909, 2649, 2209, 1822, 3572, 330, 77, 6655, 4886, 1321, 2311, 6411, 6792, 1509, 4364, 5566, 491, 376, 6266, 2436, 4826, 384, 2786, 6380, 1341, 4950, 3255, 3293, 6444, 3447, 161, 5490, 4146, 731, 61, 2411, 5034, 5233, 4896, 1364, 5042, 3455, 6208, 166, 6180, 1585, 2049, 6878, 1100, 6353, 3643, 1064, 527, 1682, 2839, 4045, 3058, 7293, 2503, 390, 2481, 3503, 4380, 1933, 1431, 7331, 667, 4023, 5504, 4615, 1101, 4993, 12, 740, 2679, 1649, 4797, 6990, 1210, 409, 2694, 1008, 1229, 1433, 2922, 2835, 6121, 2316, 4495, 3006, 6701, 5839, 3817, 1962, 5499, 1426, 4809, 5080, 2363, 6849, 1530, 6890, 4547, 3732, 2967, 4773, 4941, 5180, 2375, 2811, 1272, 2328, 2149, 6860, 4754, 686, 304, 7051, 4509, 1803, 4664, 6409, 3710, 911, 6395, 5125, 6418, 1622, 781, 2347, 1412, 3635, 6070, 6862, 2605, 1141, 4469, 6546, 5328, 6601, 5170, 1289, 6671, 1088, 3136, 1063, 2656, 5470, 3397, 6107, 5213, 2588, 2936, 92, 5860, 2183, 7053, 2977, 4388, 3484, 6248, 4988, 3077, 435, 1423, 2910, 5275, 7148, 6218, 5182, 5357, 5071, 5308, 2218, 533, 1799, 6010, 5051, 4017, 187, 571, 4037, 4351, 5035, 1458, 347, 2356, 2547, 708, 6979, 4512, 1137, 7011, 2637, 1230, 1623, 3741, 6142, 1914, 5878, 2172, 5181, 640, 5944, 2889, 6694, 3689, 3375, 2216, 514, 3213, 873, 3833, 7352, 5790, 1573, 2127, 3509, 287, 5582, 3851, 6550, 5681, 2494, 3059, 7129, 6719, 1976, 6302, 442, 4550, 371, 839, 1643, 3576, 2978, 1389, 2577, 3131, 6548, 1411, 3453, 512, 6735, 156, 3223, 2853, 1505, 2268, 1170, 202, 5367, 5332, 5838, 7022, 7068, 2524, 1940, 5997, 5951, 3407, 5070, 2425, 1837, 5158, 2485, 2721, 3892, 1571, 5159, 6513, 3049, 1262, 3869, 6206, 6997, 385, 7, 914, 6756, 3721, 3355, 5398, 6775, 2893, 5024, 5732, 4002, 4551, 2527, 1938, 66, 310, 6585, 4182, 2267, 5330, 3437, 7126, 3286, 6293, 6841, 123, 2211, 2648, 3663, 4642, 5931, 4817, 4187, 6351, 6055, 5278, 811, 284, 2379, 4399, 5934, 759, 547, 1909, 3584, 1695, 3633, 4070, 5898, 307, 879, 2054, 5129, 6093, 4294, 3840, 6415, 4772, 4193, 3380, 1495, 7244, 3492, 6968, 1795, 5252, 5369, 473, 34, 2866, 6476, 170, 4356, 688, 2610, 1717, 7188, 739, 6553, 6914, 6458, 6517, 381, 3074, 5600, 2214, 6939, 271, 2962, 19, 1471, 2035, 3613, 1350, 5321, 3782, 1995, 515, 6099, 4873, 4451, 6875, 3243, 2360, 6893, 4989, 1728, 3457, 3211, 4208, 3688, 2398, 5738, 2952, 638, 1888, 825, 3690, 5257, 5056, 162, 2559, 4051, 1708, 2600, 1780, 5249, 105, 3021, 2720, 6041, 4559, 3444, 1517, 924, 747, 5386, 3667, 5296, 2220, 5964, 2997, 5498, 1796, 1491, 6678, 5456, 572, 245, 797, 6348, 6332, 5723, 1846, 3465, 7203, 2444, 3446, 2026, 1963, 2917, 3797, 5496, 899, 4948, 2004, 6842, 4135, 3979, 3737, 7123, 2300, 6835, 4339, 2897, 6603, 3873, 3898, 3084, 6509, 6545, 4618, 2212, 4571, 4069, 1128, 183, 2207, 1097, 745, 1800, 794, 2334, 6379, 5664, 4497, 5142, 2488, 4872, 300, 2742, 1661, 4314, 593, 3097, 6487, 5994, 7375, 6080, 7032, 1778, 1457, 3500, 3229, 4250, 4595, 6160, 1981, 1018, 1873, 1465, 4627, 2860, 2632, 2500, 4731, 4179, 328, 3614, 7300, 3205, 1083, 5286, 1533, 6917, 6496, 6372, 1357, 4623, 5656, 4596, 4411, 1562, 4576, 1182, 6745, 6474, 5728, 2322, 4068, 3079, 4542, 1823, 1235, 4871, 746, 3358, 4491, 4044, 3631, 658, 5995, 2618, 1145, 5190, 4226, 6177, 1466, 6652, 3810, 922, 6596, 7140, 4513, 4382, 1784, 5769, 4331, 3485, 6572, 6475, 6727, 3094, 1600, 4203, 7015, 1003, 4639, 7144, 5959, 3657, 1882, 1504, 4713, 462, 5522, 4670, 5592, 4062, 188, 5040, 3139, 679, 3410, 1515, 5575, 3951, 6743, 2803, 5083, 2496, 5255, 7316, 1109, 479, 6740, 234, 3983, 6769, 1146, 5414, 2529, 4101, 3247, 6172, 577, 1132, 1351, 3865, 5932, 404, 472, 7394, 874, 5835, 4827, 2601, 3625, 2988, 1983, 2512, 277, 4768, 2526, 6861, 3219, 947, 4224, 4577, 6169, 6806, 3713, 6561, 5640, 5348, 264, 5231, 3824, 5187, 5118, 1138, 3980, 206, 3145, 6396, 3197, 4278, 377, 7201, 6416, 5288, 4434, 6098, 394, 990, 2348, 635, 4548, 754, 6043, 6492, 5762, 2349, 2296, 6450, 4930, 4350, 659, 4994, 334, 1819, 5954, 628, 6006, 3638, 3238, 5546, 695, 2590, 2422, 6091, 4755, 6426, 5314, 1948, 637, 6162, 4822, 4089, 213, 3162, 4063, 2461, 2293, 6933, 3841, 1187, 2635, 1845, 5194, 434, 5341, 1566, 7183, 4156, 6128, 6477, 1176, 5730, 5722, 1877, 2798, 6325, 6837, 1749, 7295, 3648, 6883, 6983, 7128, 6757, 3163, 6912, 7261, 5918, 5422, 7080, 5372, 3486, 1236, 5597, 4319, 3497, 4323, 4373, 3906, 1012, 4245, 4502, 241, 6903, 960, 1889, 1050, 286, 37, 2332, 1325, 2310, 875, 555, 5028, 1108, 6176, 6304, 6152, 2531, 496, 6076, 5604, 3435, 4407, 1676, 729, 786, 3950, 5192, 6115, 5059, 5811, 4036, 680, 1113, 2005, 6779, 2831, 5868, 3776, 5447, 7400, 2652, 2429, 6013, 5911, 1942, 3415, 3588, 6367, 4981, 2991, 2303, 1632, 5767, 6940, 5166, 2619, 2928, 1384, 6399, 443, 4428, 6333, 3994, 4913, 6236, 6662, 6088, 6068, 2538, 578, 5581, 4456, 4370, 1998, 1559, 5503, 1430, 2044, 690, 5883, 1554, 7103, 1847, 508, 3154, 4747, 1266, 1098, 2353, 5687, 1397, 6733, 4218, 6730, 742, 3001, 3478, 5264, 2808, 4955, 2714, 2948, 174, 2431, 3230, 439, 5079, 4022, 4291, 476, 5520, 261, 683, 507, 7181, 5804, 5163, 5225, 2243, 5906, 1921, 6904, 1871, 4728, 4015, 1020, 5569, 2914, 6768, 3042, 1524, 2020, 5301, 1374, 3336, 2475, 3762, 4688, 7372, 7121, 3886, 251, 3922, 4389, 888, 7043, 451, 6910, 445, 6544, 1215, 6922, 5989, 764, 1508, 551, 5038, 470, 3813, 379, 2147, 1031, 632, 6412, 3963, 3701, 6240, 2642, 5960, 3201, 7143, 3881, 2231, 3517, 5206, 3147, 4503, 6454, 5259, 1480, 6361, 6261, 6019, 2134, 1967, 4888, 4338, 244, 6945, 2227, 2378, 4143, 4421, 3821, 1333, 4961, 4264, 3413, 6500, 2678, 2457, 7155, 1990, 3224, 3999, 3180, 6767, 7041, 5480, 869, 6776, 5179, 5211, 6394, 7403, 2173, 6071, 1754, 3158, 5027, 3214, 4362, 5006, 1959, 6516, 6374, 4877, 7005, 3981, 3819, 5352, 5203, 772, 903, 5526, 2782, 3123, 3176, 7322, 6310, 2604, 6037, 1712, 351, 5799, 7296, 6915, 1483, 456, 2145, 5115, 6882, 6345, 4435, 7102, 5662, 1455, 1385, 2508, 2358, 549, 7368, 4771, 459, 4176, 5524, 647, 5966, 3825, 7168, 2796, 320, 6336, 4371, 5844, 4040, 4279, 4723, 3128, 4487, 4736, 4453, 1898, 6113, 4289, 3127, 5452, 4432, 1747, 4270, 7154, 5342, 3673, 3743, 2382, 2540, 6383, 6554, 2740, 4714, 3806, 3235, 5146, 2851, 4348, 2484, 3254, 5370, 2385, 6913, 2365, 6370, 2684, 1334, 2979, 6993, 2834, 3210, 4471, 4464, 5872, 2369, 2563, 1274, 2631, 6677, 3463, 6705, 78, 3934, 6486, 5294, 3326, 2140, 1203, 7220, 6233, 210, 0, 4228, 2611, 5376, 2166, 2624, 2504, 2395, 4936, 6643, 6000, 5385, 4825, 604, 4777, 1768, 2487, 3811, 7356, 820, 7179, 5785, 3739, 3110, 1503, 5089, 3438, 3004, 2992, 2929, 6066, 5086, 6715, 4327, 3603, 3296, 5464, 963, 4151, 1442, 4305, 1082, 6528, 1637, 3495, 6918, 3645, 611, 2198, 1359, 3514, 6647, 5366, 6125, 6321, 4683, 3915, 2744, 3828, 1391, 3694, 5654, 5565, 6760, 5808, 2765, 6659, 1427, 3909, 96, 272, 5344, 2112, 6260, 1972, 1767, 5353, 5141, 2232, 807, 5588, 5472, 2918, 4207, 4273, 58, 6073, 5406, 4157, 1947, 3730, 5263, 6600, 2949, 4138, 3159, 580, 3591, 232, 2789, 1725, 7240, 4593, 4488, 285, 6656, 5635, 5753, 3130, 6823, 3834, 1454, 6034, 6742, 2965, 2793, 45, 6448, 1069, 458, 830, 3023, 5239, 5846, 6765, 5235, 5615, 5073, 6, 189, 6610, 5318, 6986, 1835, 5381, 3111, 6556, 1314, 793, 1718, 4699, 1121, 749, 642, 1301, 3308, 3880, 7219, 2111, 2735, 7350, 7371, 7192, 6005, 6972, 5930, 753, 4199, 3685, 3031, 3592, 3905, 3458, 4895, 6228, 890, 3409, 7344, 7027, 2616, 546, 2752, 6312, 7392, 5873, 1944, 6640, 787, 6299, 3822, 1874, 7267, 1875, 1516, 5195, 4357, 5985, 2440, 1260, 7340, 3901, 6569, 3096, 7014, 6501, 4249, 6829, 6389, 4840, 4276, 172, 2051, 1879, 6430, 5678, 1872, 6952, 2823, 403, 1943, 125, 1086, 4336, 5980, 5789, 2541, 1588, 1352, 2698, 2274, 2650, 3472, 4128, 7152, 4844, 5877, 4530, 856, 2566, 2659, 3595, 1299, 3405, 4235, 6456, 5874, 4376, 3291, 5862, 7120, 4737, 5323, 3356, 6405, 4740, 2266, 784, 1140, 3823, 2970, 5110, 2449, 902, 1590, 5299, 5208, 1885, 4415, 5972, 6530, 2136, 3581, 554, 7013, 6288, 1484, 6217, 920, 4614, 2324, 5043, 1336, 5797, 850, 1956, 7301, 4764, 6075, 2196, 1052, 6902, 6480, 2693, 6065, 306, 586, 3987, 1805, 237, 4858, 4535, 3703, 2570, 322, 4613, 1, 3900, 2868, 4481, 1344, 3319, 2535, 5933, 6052, 5399, 2144, 3626, 5292, 6447, 2169, 5660, 1891, 1365, 3511, 2418, 5658, 367, 1785, 2168, 6964, 5188, 302, 5674, 1618, 4313, 2956, 1774, 1080, 1672, 3907, 4624, 5108, 1613, 413, 5247, 6664, 1895, 5832, 3499, 6104, 5405, 860, 2595, 6140, 6698, 1486, 5189, 655, 4779, 6212, 2975, 292, 1277, 4396, 3702, 564, 3135, 6424, 2797, 1670, 4175, 1840, 1479, 157, 6011, 3386, 3345, 1123, 1243, 2128, 6976, 1502, 5854, 4917, 7276, 7270, 3507, 6316, 930, 428, 177, 4466, 2156, 3038, 2186, 396, 3525, 4610, 6706, 7074, 3634, 3849, 5205, 5023, 5169, 4933, 3411, 7306, 6174, 1760, 229, 3440, 1284, 2280, 4621, 5609, 6876, 5346, 2340, 53, 4010, 2791, 4360, 5659, 5724, 2439, 3991, 5796, 1843, 4666, 1666, 2644, 2716, 3501, 337, 3496, 6392, 1829, 1254, 6114, 1609, 2146, 2908, 2193, 6590, 5011, 4724, 6335, 440, 2119, 1763, 4900, 7380, 1793, 7294, 5126, 410, 6690, 943, 597, 949, 1481, 6250, 2095, 5943, 6958, 4209, 6158, 7058, 1152, 5469, 2817, 5691, 6857, 6581, 6033, 6340, 2338, 1044, 4695, 6855, 5215, 3736, 1858, 2467, 5119, 5413, 5270, 1441, 7083, 4169, 3742, 3167, 6853, 6014, 2756, 5819, 1492, 3939, 6403, 7153, 3977, 290, 3426, 1617, 1802, 6400, 6687, 933, 1545, 931, 6812, 3125, 3923, 4665, 1531, 5782, 576, 1783, 1770, 4486, 1218, 6822, 7158, 7086, 6387, 369, 2022, 4553, 3364, 1750, 1975, 7008, 612, 3566, 6157, 1951, 5859, 5612, 3589, 2318, 701, 4641, 5123, 834, 457, 348, 1862, 2515, 800, 3172, 230, 5106, 1621, 7233, 1402, 2223, 1415, 2787, 7311, 862, 1085, 5232, 5676, 6988, 2987, 446, 1057, 4046, 5620, 6311, 4769, 5651, 1539, 5152, 6440, 671, 145, 4300, 6927, 4095, 1594, 4029, 2930, 5492, 5284, 2367, 5082, 5895, 2832, 1118, 5805, 3867, 2586, 1904, 1574, 4638, 4287, 3620, 4568, 643, 6809, 1646, 3997, 5891, 4885, 509, 3848, 1456, 2576, 2518, 4705, 484, 5173, 5224, 3066, 748, 1687, 3586, 7325, 2307, 4682, 4859, 5541, 7337, 75, 6536, 5092, 2517, 976, 4217, 4510, 42, 5139, 6101, 6709, 3792, 4808, 5441, 6663, 2224, 3258, 2509, 7250, 1209, 1248, 1939, 6973, 7094, 252, 4890, 1812, 6820, 5774, 5978, 4416, 6277, 2264, 3931, 7269, 2251, 934, 2776, 6151, 3835, 7355, 6246, 3196, 7056, 7132, 3480, 2284, 475, 3, 1404, 4860, 2377, 4558, 2452, 3311, 5689, 168, 7130, 6692, 2514, 4091, 1183, 5506, 6428, 7114, 6831, 1373, 3563, 2551, 134, 2645, 216, 5519, 1238, 1794, 4158, 6307, 4420, 4622, 3887, 6900, 4582, 5191, 7238, 595, 352, 1497, 5484, 2638, 1380, 2598, 5383, 5373, 3664, 994, 775, 1354, 6072, 239, 3468, 2785, 7111, 6143, 6683, 968, 4792, 3680, 6199, 6362, 2031, 7137, 4072, 4958, 4152, 5904, 5217, 2510, 1226, 4881, 1371, 2974, 4067, 7067, 5695, 4785, 1514, 6485, 3227, 6524, 5396, 2122, 4598, 2081, 3753, 4424, 1029, 4697, 2759, 2505, 6622, 7062, 619, 728, 3895, 6272, 1493, 1826, 2414, 908, 5535, 301, 4984, 6786, 5186, 6303, 5632, 1910, 7323, 7324, 6008, 568, 86, 654, 4857, 3425, 5240, 5114, 906, 2071, 2161, 3948, 5090, 5977, 932, 495, 606, 543, 1468, 4908, 7319, 5481, 1358, 4986, 1036, 3488, 2281, 2689, 1319, 2344, 1724, 3423, 5055, 4767, 6759, 3512, 1869, 3043, 6342, 681, 5950, 5400, 857, 3325, 1897, 227, 1528, 5973, 4489, 3582, 4190, 7292, 1419, 6452, 792, 2824, 6777, 630, 5389, 199, 1356, 2907, 2770, 5420, 2925, 5679, 5426, 2806, 5910, 2713, 4964, 5703, 3057, 6621, 4717, 5412, 6521, 4534, 4835, 6373, 1797, 1242, 525, 99, 2339, 7035, 3221, 6784, 4659, 6889, 2861, 1185, 7310, 5975, 3826, 7247, 6267, 2878, 314, 2879, 2354, 4616, 308, 4746, 6168, 596, 5647, 609, 5781, 452, 4539, 269, 6112, 7396, 5778, 4192, 6283, 265, 4061, 7253, 6103, 5865, 954, 7157, 1604, 1932, 868, 1494, 6865, 3665, 3533, 6981, 513, 4317, 4031, 132, 1652, 7180, 3361, 2961, 6232, 3513, 5882, 4363, 964, 633, 5069, 7079, 4231, 3593, 7209, 5938, 5210, 1755, 1906, 3656, 5579, 3920, 6761, 4898, 1743, 7362, 3561, 5821, 2836, 3609, 844, 6245, 6549, 4739, 2983, 5759, 6222, 165, 5589, 4939, 918, 3532, 3363, 3985, 2946, 6537, 2816, 3947, 4501, 1549, 2728, 3719, 4465, 160, 4369, 6827, 6770, 2493, 5675, 2748, 3554, 1836, 4408, 2117, 4580, 7277, 7089, 3317, 6977, 2896, 3151, 2654, 1552, 3371, 178, 5636, 1714, 2404, 3337, 3330, 5488, 393, 76, 2233, 5556, 6638, 1440, 3924, 814, 557, 3751, 4052, 7138, 5058, 2230, 5337, 1304, 2792, 3393, 257, 4119, 6368, 5818, 6612, 2143, 4662, 7093, 1201, 5029, 408, 1560, 3884, 674, 5690, 1558, 6783, 5149, 3467, 3602, 3958, 1999, 2464, 1265, 6628, 5957, 256, 1751, 4696, 608, 5551, 7124, 5184, 1534, 1368, 2885, 6956, 1859, 5010, 5081, 5550, 5326, 677, 1851, 2070, 2723, 2459, 93, 723, 4770, 4569, 3995, 4296, 3658, 4738, 1654, 5515, 309, 1302, 4195, 2252, 1727, 6017, 4631, 7385, 4919, 6086, 5434, 2333, 5442, 2906, 3025, 4081, 1246, 6794, 2675, 1945, 7208, 2013, 4219, 441, 4324, 5599, 6970, 3237, 1045, 945, 5856, 2423, 1645, 4445, 5428, 7373, 813, 4459, 5269, 2370, 2838, 6346, 1791, 4625, 956, 2502, 3695, 1496, 7370, 967, 4322, 5417, 4020, 6814, 6431, 4698, 5427, 4689, 4546, 1597, 885, 3993, 4303, 6210, 3692, 2403, 3119, 3860, 6923, 5438, 3067, 5558, 3040, 3815, 7206, 159, 1563, 6807, 1307, 1629, 5193, 1298, 7164, 5754, 4423, 2550, 3773, 3800, 5395, 2057, 1131, 4335, 5771, 1345, 4515, 843, 1278, 531, 624, 1893, 588, 6587, 4549, 4541, 1420, 487, 5402, 616, 1745, 5230, 6012, 5760, 944, 416, 521, 7274, 3611, 6032, 6686, 6482, 4390, 537, 4049, 5965, 1603, 7328, 5253, 4787, 4725, 240, 1935, 6269, 353, 2643, 3798, 1634, 3359, 4905, 6702, 4366, 1111, 905, 828, 5487, 2188, 2549, 2343, 763, 4066, 318, 2506, 282, 3573, 6672, 2777, 1093, 7334, 5162, 3281, 824, 1675, 3373, 3768, 5121, 2441, 4261, 3099, 7190, 2842, 5374, 4318, 7185, 1216, 6016, 3722, 6173, 2548, 841, 3706, 1048, 4681, 2800, 3244, 2424, 5593, 6004, 3504, 1584, 3628, 3686, 6264, 6464, 332, 155, 4758, 5293, 1667, 3150, 2260, 6148, 6187, 1832, 4480, 4234, 999, 4940, 4672, 2184, 5890, 4982, 4216, 1377, 1489, 4718, 2294, 2290, 6057, 4137, 2848, 4766, 6731, 1511, 4402, 15, 6884, 1526, 3039, 7339, 4205, 6359, 916, 4419, 4310, 4661, 5912, 7242, 2017, 7184, 1919, 3681, 3012, 6533, 6606, 6942, 3088, 3599, 5694, 6049, 411, 3571, 751, 6281, 2152, 2985, 5088, 959, 7399, 1032, 1198, 3936, 2222, 1817, 3740, 5757, 1244, 6026, 147, 3801, 6421, 2753, 448, 5109, 149, 7273, 7378, 3460, 1159, 4266, 4171, 4053, 2762, 3608, 3590, 5827, 5025, 453, 2376, 2003, 2843, 6717, 3273, 1363, 4637, 6457, 1883, 1160, 1429, 1863, 6580, 3466, 641, 1886, 7348, 3932, 3020, 3404, 4393, 4536, 3660, 7160, 16]
Shuffle3 = [5562, 135, 4019, 389, 5191, 507, 3315, 6734, 7364, 3250, 7223, 244, 4791, 5324, 1384, 6504, 209, 2520, 5511, 4426, 5467, 273, 2965, 3370, 6519, 4867, 4438, 1782, 1857, 2276, 2440, 811, 1178, 1798, 3655, 3858, 1115, 6498, 3456, 6194, 2, 3300, 5123, 6354, 79, 1154, 6492, 7395, 6895, 5108, 482, 4493, 174, 4171, 3155, 4599, 1710, 2488, 2158, 241, 7179, 2105, 4332, 7256, 4877, 4661, 4090, 7236, 749, 6841, 6579, 4912, 2303, 1290, 5122, 401, 7057, 2454, 910, 5508, 5104, 5468, 30, 2182, 6563, 2819, 6515, 496, 3355, 954, 4322, 5023, 3302, 5965, 6938, 5705, 1759, 1453, 75, 2154, 3008, 4759, 5296, 2590, 3387, 3327, 4745, 2171, 743, 5495, 6003, 2489, 2421, 3189, 1596, 347, 3433, 7177, 5620, 2071, 104, 3984, 1855, 289, 4698, 5087, 3377, 3567, 6414, 1998, 2407, 1459, 6454, 6259, 19, 5317, 643, 7394, 7045, 3487, 4016, 2971, 5640, 2416, 2928, 4946, 1563, 2081, 6387, 4316, 785, 2733, 682, 1469, 790, 7087, 121, 5935, 2000, 5278, 4956, 3547, 3458, 5366, 1943, 4680, 2419, 308, 7241, 562, 2784, 3381, 4089, 2660, 3198, 6633, 5885, 6954, 6449, 242, 3672, 5312, 6687, 4844, 2579, 227, 1825, 1343, 879, 2892, 5171, 1725, 4449, 114, 7110, 3500, 6763, 4836, 5961, 6908, 7226, 736, 1424, 686, 2895, 3231, 5567, 3837, 5982, 2442, 6904, 4181, 3265, 3925, 2921, 4985, 5175, 105, 6458, 5113, 2149, 6796, 981, 1480, 1726, 4936, 4964, 4690, 6166, 5436, 2640, 4635, 2116, 7113, 4404, 924, 4725, 186, 4083, 3, 769, 5341, 5239, 7135, 2628, 751, 2199, 3508, 4379, 3286, 3867, 6738, 3806, 4703, 3047, 1987, 6307, 6136, 1863, 4356, 6448, 2811, 3603, 6442, 109, 567, 5294, 1007, 1404, 5454, 1251, 7154, 5863, 6512, 6545, 3757, 1758, 4038, 4219, 7214, 5079, 3061, 7071, 1030, 2318, 3988, 1992, 3810, 7303, 6269, 6741, 1318, 6346, 3325, 660, 4798, 2188, 4021, 2482, 5614, 466, 4006, 3052, 5217, 5954, 434, 4548, 3513, 7076, 2198, 6123, 2402, 5061, 2409, 2465, 977, 3290, 1778, 752, 7091, 5295, 4856, 155, 1883, 3422, 3476, 2623, 672, 4372, 5974, 1340, 2736, 574, 331, 6465, 2207, 3307, 3159, 6222, 1110, 837, 978, 7151, 1331, 1933, 617, 889, 5078, 5189, 637, 595, 1065, 7337, 4500, 510, 7, 4643, 3057, 2739, 6948, 1000, 5822, 5998, 6306, 2108, 2540, 1667, 4896, 6246, 3652, 5406, 869, 3206, 3398, 4782, 2381, 4288, 4663, 5345, 6171, 5731, 5462, 6858, 7392, 6853, 4488, 2070, 2832, 5060, 3175, 7143, 3887, 6227, 464, 4435, 6205, 5951, 7095, 5856, 2422, 1460, 4739, 225, 4585, 6061, 7191, 6625, 6827, 7205, 6768, 2412, 3685, 1966, 2913, 6043, 1307, 374, 5127, 111, 3808, 3424, 6696, 4731, 856, 5416, 2873, 3963, 6389, 4341, 3042, 3675, 2150, 2723, 763, 3143, 7211, 3605, 6635, 6452, 1107, 2486, 6133, 1829, 6292, 6986, 1787, 2706, 2574, 3914, 4997, 5855, 4363, 5574, 5558, 1872, 4371, 1337, 7128, 2667, 23, 4484, 1466, 1995, 3862, 2250, 2854, 4689, 6457, 7096, 3015, 5244, 921, 49, 4079, 4934, 4882, 1241, 1273, 1051, 6995, 2998, 1455, 7164, 3851, 3368, 2032, 7006, 3773, 1184, 1997, 3928, 3940, 2610, 1365, 3246, 1347, 1215, 4313, 3196, 5254, 6244, 4413, 3692, 4270, 3409, 4832, 6467, 5743, 3180, 2429, 7149, 6225, 5299, 2755, 6724, 1686, 5613, 7313, 1624, 4749, 108, 3283, 1188, 6390, 1621, 4418, 6834, 3550, 2374, 5534, 2539, 2878, 4192, 1116, 7221, 6759, 618, 3363, 1496, 6471, 2470, 1073, 460, 4146, 2471, 6664, 1930, 6914, 7020, 2078, 3884, 6960, 7165, 2464, 720, 5335, 372, 3733, 3710, 7334, 237, 764, 4241, 2288, 1286, 519, 3777, 973, 3592, 710, 382, 944, 7304, 3488, 6732, 6845, 3593, 2929, 3030, 197, 2645, 6749, 6811, 4376, 5972, 1386, 2231, 205, 310, 4451, 5162, 2926, 6040, 2766, 3506, 5583, 6420, 5214, 1716, 1137, 6030, 1229, 6462, 3869, 6513, 3796, 5590, 4286, 3794, 2795, 128, 5660, 2537, 4989, 2329, 6160, 5043, 3560, 1834, 5514, 4521, 1078, 7290, 2663, 814, 4470, 6805, 7147, 7040, 2366, 943, 17, 2277, 4894, 334, 5196, 89, 3714, 3003, 7289, 1038, 5586, 4436, 1099, 475, 7010, 207, 2313, 3408, 1145, 4064, 31, 5332, 3142, 5168, 3428, 7263, 1173, 6702, 963, 6751, 1468, 4310, 6119, 4598, 3897, 3878, 3178, 894, 3744, 542, 6517, 1690, 5900, 3501, 1951, 3677, 4469, 7188, 6549, 276, 3604, 6289, 2233, 6672, 1821, 5519, 7332, 1591, 2659, 2252, 4169, 7024, 5684, 2789, 6267, 1430, 6407, 739, 599, 4240, 2311, 5075, 2849, 6889, 5090, 1248, 3555, 3418, 6875, 4454, 1976, 2605, 3915, 5210, 3207, 725, 4802, 5802, 5428, 144, 2901, 1336, 7000, 895, 5280, 1680, 98, 3162, 5397, 2954, 5096, 4741, 2514, 5737, 6441, 4080, 3043, 5915, 6970, 932, 989, 3918, 6338, 2846, 6199, 3682, 6870, 1788, 1659, 3085, 3965, 4963, 1773, 2571, 3769, 5103, 5538, 3801, 1247, 5285, 279, 1849, 1663, 1694, 1844, 7381, 2256, 5063, 313, 1608, 2010, 2204, 2135, 2483, 393, 6645, 728, 7036, 7204, 4214, 3229, 4491, 2053, 4895, 1324, 2535, 2335, 1245, 4211, 6840, 4879, 6070, 4326, 6859, 4980, 5463, 2585, 729, 6522, 5112, 4902, 6370, 6341, 3748, 3530, 6466, 5658, 2122, 3267, 6092, 5507, 1676, 4792, 6592, 3098, 5958, 5851, 6110, 457, 188, 183, 1326, 5910, 4942, 5208, 4981, 2232, 4243, 3073, 1524, 3938, 2668, 3610, 3349, 3357, 1671, 6671, 6978, 3163, 4881, 6638, 182, 5050, 483, 2779, 3855, 5738, 5959, 3788, 3597, 6024, 2322, 5377, 2050, 6606, 3103, 3676, 6365, 5308, 6432, 4178, 2432, 4007, 175, 7349, 7268, 283, 5594, 4294, 4455, 7338, 5402, 5138, 5255, 1171, 4162, 6816, 191, 882, 4837, 5380, 737, 6369, 2576, 3019, 4424, 4588, 5410, 1869, 6485, 4628, 1939, 5401, 0, 4468, 5707, 6258, 2220, 3469, 6463, 5438, 5831, 4364, 5323, 7101, 969, 458, 868, 3288, 1341, 2847, 3260, 528, 3921, 6496, 5386, 5202, 6335, 2459, 1516, 7025, 3807, 3473, 6113, 3543, 3627, 4968, 1304, 913, 890, 5530, 2376, 6934, 4552, 1562, 1205, 3148, 1436, 3122, 7201, 2463, 365, 4692, 294, 2529, 1187, 6684, 1756, 246, 97, 4073, 3356, 1601, 1522, 5197, 5337, 2817, 2572, 7359, 2682, 1346, 4222, 6533, 7138, 6641, 1196, 4846, 3310, 5166, 3062, 514, 2863, 5498, 6505, 3635, 3076, 5059, 5132, 4009, 2165, 3115, 2321, 7001, 7326, 5106, 2601, 4249, 4056, 2868, 7030, 64, 5693, 6257, 6933, 5372, 4074, 3681, 1358, 6362, 5821, 5119, 1227, 2734, 1096, 690, 4099, 1279, 3431, 4194, 3291, 848, 4613, 7028, 502, 6721, 6726, 2438, 4263, 5685, 5713, 1786, 6586, 4646, 6451, 6712, 4045, 7167, 5378, 923, 3849, 1354, 7237, 4561, 5114, 314, 4422, 3211, 829, 1105, 6682, 4015, 3628, 3622, 5107, 5971, 1363, 4247, 3645, 929, 904, 5200, 4177, 7017, 2569, 5143, 1604, 1890, 1085, 740, 2315, 5879, 2293, 1635, 5487, 2716, 5585, 2189, 2104, 4452, 4377, 4610, 5818, 6521, 4732, 6643, 1478, 5216, 5425, 6530, 4478, 873, 145, 2699, 3856, 2685, 3153, 7399, 1915, 2118, 6288, 642, 4625, 2632, 7180, 3798, 4524, 6809, 695, 3870, 4130, 984, 5829, 5722, 6777, 167, 734, 1905, 6611, 3022, 140, 1732, 7066, 2582, 4272, 439, 10, 5710, 2020, 5843, 3482, 1098, 1491, 353, 7294, 6725, 1766, 1445, 3412, 5701, 6929, 5896, 5730, 6566, 2968, 5928, 3564, 2724, 2797, 854, 4845, 1082, 4609, 7038, 5451, 1417, 646, 1925, 4176, 5578, 142, 5938, 7270, 6348, 654, 6941, 885, 7015, 2986, 1891, 4952, 2587, 1066, 3563, 1086, 3011, 236, 1830, 3044, 4535, 212, 168, 5413, 5368, 4287, 3590, 3959, 6298, 74, 2864, 2427, 1192, 148, 1438, 7021, 1682, 5880, 5646, 6140, 2430, 3872, 3016, 2533, 2206, 2737, 5579, 4415, 6765, 6019, 305, 1345, 4951, 1291, 1329, 6464, 6753, 2411, 445, 2103, 4227, 4485, 5398, 40, 78, 2240, 1396, 4236, 5882, 3697, 716, 6188, 552, 4289, 5732, 7299, 194, 6594, 960, 4403, 5865, 1224, 5794, 2113, 4037, 634, 92, 5221, 3451, 7272, 5528, 7137, 5940, 5032, 2877, 3784, 3886, 4518, 3889, 953, 3079, 1657, 5832, 6737, 3157, 3779, 2999, 3608, 645, 7123, 6017, 6359, 3772, 1761, 3845, 195, 3262, 463, 6523, 6839, 1741, 1921, 1151, 1210, 1630, 3495, 2034, 1674, 1093, 6041, 3328, 1572, 865, 675, 3648, 1822, 3557, 2048, 3762, 2831, 2570, 1658, 6807, 5901, 481, 2827, 2292, 5293, 6780, 6200, 4384, 223, 1653, 6425, 462, 3793, 6488, 7046, 3761, 6079, 4607, 2046, 1537, 6881, 14, 6029, 987, 4472, 7189, 7282, 1922, 6924, 1790, 1958, 2757, 1802, 3099, 3690, 5655, 5696, 1754, 5279, 4522, 1755, 4107, 2143, 2993, 976, 5857, 818, 5409, 6252, 1228, 1826, 755, 2461, 4885, 633, 706, 5475, 825, 2988, 6066, 2713, 33, 6743, 859, 1977, 125, 6290, 5450, 1202, 6439, 5269, 2544, 6272, 4230, 4110, 4039, 1664, 6865, 6971, 4355, 4711, 3089, 1264, 920, 351, 4317, 4966, 1338, 6882, 3127, 3248, 6640, 786, 2346, 6685, 6068, 5909, 1528, 589, 1767, 2441, 5376, 6109, 4402, 2008, 4734, 3730, 6838, 4619, 373, 3210, 5692, 4986, 6912, 6691, 6057, 999, 2092, 5608, 2490, 608, 2559, 1706, 4529, 3316, 2291, 1207, 1280, 1026, 965, 3169, 7397, 6388, 3613, 3621, 1514, 6139, 6187, 6610, 1382, 631, 3129, 2239, 8, 2183, 4979, 1818, 2555, 4603, 6215, 4947, 4127, 132, 5206, 958, 39, 1724, 6361, 5408, 1124, 3882, 619, 3782, 2620, 4213, 4543, 309, 1730, 3194, 7380, 383, 5159, 2022, 995, 419, 4226, 980, 4640, 4869, 817, 6770, 4187, 7039, 2468, 7343, 3791, 649, 1775, 3712, 6876, 795, 210, 2889, 1736, 612, 847, 2080, 5260, 555, 417, 6203, 3929, 1879, 5136, 1515, 7356, 6939, 5781, 5989, 6415, 3097, 120, 1040, 6964, 2279, 4199, 2519, 4669, 3472, 6614, 5680, 4331, 4848, 6823, 4348, 5637, 6869, 5477, 2392, 1876, 1900, 4608, 561, 2821, 3303, 4657, 5554, 5003, 2909, 270, 5444, 5561, 1412, 1432, 425, 2662, 4296, 5791, 2495, 1530, 5679, 5866, 5596, 1968, 2218, 6714, 5545, 2577, 5022, 1333, 6299, 5035, 1699, 1211, 5890, 240, 852, 7253, 1457, 1024, 4134, 5551, 4052, 3028, 508, 6650, 4004, 4797, 949, 1385, 5004, 1647, 516, 5178, 6891, 5147, 4246, 992, 2130, 2882, 1967, 6343, 6762, 5073, 1443, 3834, 6318, 3405, 3771, 7200, 6347, 5926, 5007, 1006, 6131, 1032, 5483, 3135, 5379, 3114, 6108, 1487, 4971, 3787, 6527, 3776, 6570, 2452, 7112, 7012, 2825, 6717, 5065, 5070, 4753, 3165, 4443, 1617, 1146, 2099, 497, 271, 4721, 4252, 6561, 1669, 6391, 2140, 5453, 2717, 826, 6510, 579, 4220, 4781, 1410, 1297, 1709, 6071, 4717, 3985, 494, 5037, 7312, 3233, 2977, 5473, 6138, 5868, 735, 2307, 6038, 704, 2900, 1605, 5301, 2879, 5211, 4431, 4299, 4949, 3785, 4223, 7193, 4020, 4084, 553, 6760, 3924, 7133, 4267, 2373, 1416, 657, 4163, 1226, 6867, 4969, 3611, 1423, 4779, 2853, 3065, 4057, 5828, 5246, 5190, 3449, 4406, 203, 4098, 3657, 4444, 2703, 1357, 6803, 2028, 4461, 1764, 1618, 1177, 5568, 6683, 5072, 7100, 7090, 1010, 1131, 6824, 6174, 5053, 4604, 3974, 3541, 1484, 3850, 4974, 363, 4027, 5265, 3331, 2487, 5055, 405, 3276, 6666, 149, 1753, 523, 4940, 5811, 7257, 3977, 5946, 1434, 6461, 6547, 1842, 4805, 4853, 3976, 3691, 7259, 24, 262, 3275, 2134, 7202, 3219, 6422, 1742, 6020, 1701, 5311, 1253, 5977, 4028, 298, 381, 4330, 5230, 1373, 213, 505, 1551, 47, 4706, 7305, 598, 5394, 5947, 1164, 3013, 1303, 3091, 855, 6050, 1832, 6754, 4440, 2024, 2456, 5503, 3546, 2674, 4840, 6617, 2478, 6676, 6740, 4408, 5198, 6854, 169, 3255, 5673, 2125, 880, 2750, 753, 3840, 816, 4151, 2735, 4274, 5034, 1062, 6956, 4234, 563, 3948, 6012, 1265, 6219, 6253, 1538, 5934, 5555, 76, 2858, 441, 3126, 5706, 652, 6965, 2327, 3614, 4439, 46, 5671, 45, 4914, 6255, 233, 2808, 315, 6735, 5827, 5891, 6937, 3669, 1101, 7169, 4429, 2595, 5862, 6372, 4473, 2055, 1141, 5027, 390, 2820, 3416, 3519, 1371, 2680, 4183, 250, 6206, 1156, 288, 6535, 3994, 293, 13, 5008, 3348, 5964, 991, 2475, 6619, 1578, 4839, 6542, 5307, 6946, 3372, 1784, 450, 5194, 6273, 5445, 3562, 7029, 7308, 2752, 2803, 5784, 1612, 1355, 938, 6601, 5767, 6630, 6234, 4995, 6944, 5158, 1955, 5588, 6894, 952, 4041, 614, 5464, 2423, 791, 3558, 3351, 7254, 4821, 6890, 3066, 4533, 684, 5302, 6400, 4318, 5945, 622, 4112, 7281, 4566, 4357, 6539, 3183, 2548, 115, 5559, 6985, 7199, 4434, 1719, 5017, 71, 6900, 3295, 4302, 3910, 6818, 6907, 1070, 1935, 6600, 5550, 2561, 6871, 651, 93, 1523, 2695, 5169, 1374, 1100, 2355, 2804, 6581, 1150, 4999, 1375, 451, 2649, 800, 7387, 1795, 5919, 5836, 4370, 7371, 2209, 4709, 5226, 5922, 7131, 4630, 6697, 5899, 1017, 2197, 3626, 6658, 520, 296, 7315, 721, 4265, 3877, 6376, 7050, 1856, 6179, 1928, 322, 6711, 6125, 4710, 84, 7235, 5544, 1080, 4883, 5282, 2702, 349, 5129, 4810, 1298, 133, 4487, 2839, 6268, 6700, 5181, 6855, 5440, 12, 3832, 4366, 6877, 1212, 4777, 3809, 5825, 6385, 5488, 5338, 177, 883, 5165, 620, 1750, 4634, 815, 3517, 565, 1421, 3931, 1792, 6728, 5356, 946, 4572, 6251, 6584, 3464, 4754, 5921, 1585, 3486, 7034, 1213, 3390, 1126, 3831, 4829, 1904, 2297, 6228, 7324, 7329, 55, 4769, 6371, 3632, 5092, 5597, 517, 1081, 4381, 6791, 2996, 2568, 2591, 2962, 4095, 2195, 6022, 3587, 3147, 4516, 1021, 1873, 1419, 2067, 4920, 2418, 956, 7348, 7271, 6218, 6245, 1654, 158, 2253, 156, 5364, 2630, 4656, 1748, 4691, 3077, 300, 5086, 5201, 6923, 3815, 5126, 5711, 321, 7152, 4953, 726, 5975, 3719, 345, 1558, 4652, 4315, 4929, 639, 7402, 3537, 3765, 6277, 866, 5789, 3957, 827, 3203, 1433, 1112, 3529, 138, 4695, 6305, 4719, 7274, 4577, 7280, 4482, 1944, 4050, 2542, 861, 4103, 881, 4787, 650, 1087, 4428, 7222, 6962, 2278, 2379, 6310, 5361, 2935, 6856, 5526, 602, 7011, 99, 1076, 1580, 4852, 6660, 378, 3411, 2865, 5319, 5920, 5243, 2740, 6744, 4872, 974, 7088, 2299, 1866, 4935, 6952, 2638, 3479, 2040, 6787, 5504, 3888, 1367, 6730, 6002, 6649, 159, 2643, 2353, 1687, 6571, 2936, 1220, 1734, 2552, 5955, 6789, 7292, 4152, 600, 5407, 3124, 1301, 4961, 6931, 1941, 5913, 1047, 5886, 6378, 6360, 476, 499, 7225, 4269, 1685, 1629, 6083, 789, 2851, 1233, 5702, 7355, 2152, 2251, 7363, 4463, 6548, 5176, 3442, 3134, 4101, 762, 5930, 5304, 4859, 3080, 5815, 6233, 4210, 1288, 1138, 1666, 2499, 4204, 433, 3090, 3731, 3360, 6281, 6345, 2684, 5010, 1611, 5833, 5986, 3150, 1835, 3706, 1791, 3958, 5841, 3186, 4017, 1708, 4058, 7014, 3054, 4540, 2612, 4825, 6294, 6983, 7317, 442, 6380, 559, 578, 4399, 2732, 569, 604, 1463, 2656, 5631, 6075, 2194, 4616, 1409, 2077, 7098, 3138, 3998, 32, 6264, 6604, 3277, 5716, 5455, 3404, 3440, 2626, 5389, 4809, 7273, 6536, 5374, 714, 6830, 1615, 379, 5348, 4496, 6781, 5120, 2621, 179, 758, 3374, 5041, 6352, 5042, 3903, 4264, 4136, 5373, 2604, 5077, 5795, 1182, 4755, 2593, 5362, 5800, 1878, 4354, 4321, 7319, 6800, 2031, 5411, 1278, 2985, 1999, 5723, 3223, 3936, 6936, 2185, 6, 1901, 6878, 5927, 5630, 5424, 6860, 3859, 2302, 4261, 1206, 1959, 5665, 2485, 1269, 5941, 6585, 4029, 2715, 4713, 5750, 5340, 5877, 2212, 1319, 2238, 6851, 7242, 5944, 4903, 6810, 6243, 1283, 3371, 6784, 3235, 6595, 448, 6104, 2496, 5183, 3624, 7283, 4196, 4078, 6064, 5669, 5999, 2657, 5134, 7159, 6196, 525, 2153, 6241, 5093, 2111, 5756, 1123, 4067, 7124, 2725, 2980, 3746, 777, 3278, 3786, 3923, 7357, 4768, 2622, 2462, 456, 1315, 7307, 1200, 6401, 3514, 4509, 4786, 5542, 6507, 1899, 792, 7178, 1316, 533, 6556, 6301, 3502, 2481, 1395, 1471, 7244, 6968, 1910, 6209, 5145, 4387, 5049, 2527, 3580, 4506, 3437, 3499, 5939, 6479, 6285, 886, 616, 6142, 2073, 7156, 979, 5506, 1665, 5912, 226, 3258, 5772, 2368, 5359, 3133, 1926, 1429, 1295, 3151, 1731, 4245, 5141, 4278, 6088, 6440, 3454, 2260, 4644, 6126, 3234, 3997, 3643, 7153, 6063, 2611, 6772, 5949, 6447, 5172, 3846, 5383, 5000, 2101, 3403, 3463, 6056, 3913, 6091, 3573, 5036, 6651, 181, 5291, 6282, 7278, 4704, 180, 3875, 18, 392, 3023, 3981, 3665, 3736, 3161, 656, 2924, 2114, 2546, 6065, 3026, 3296, 2458, 5704, 7378, 2927, 2498, 4921, 3735, 6304, 2448, 1937, 1927, 6826, 3616, 1812, 2170, 5521, 5639, 1009, 5496, 2506, 4069, 375, 1681, 5790, 5199, 3311, 4036, 387, 2637, 1050, 4930, 1616, 5746, 6848, 2023, 5297, 3494, 3799, 1805, 607, 7158, 5968, 4618, 4335, 2365, 7049, 670, 5573, 2338, 3239, 7186, 6211, 189, 1949, 4783, 4367, 7246, 5382, 2747, 1259, 7333, 6195, 5435, 3309, 5024, 7127, 87, 5098, 5, 3571, 5884, 5385, 1769, 3082, 4389, 1898, 2690, 1828, 3535, 513, 1381, 4565, 1704, 1155, 3520, 4795, 6051, 469, 5801, 4113, 697, 4622, 2769, 4195, 4584, 3518, 3819, 3545, 2721, 4314, 808, 1431, 673, 1990, 5193, 3430, 6861, 112, 2446, 5472, 2193, 568, 312, 4893, 304, 222, 377, 3666, 3441, 2271, 2493, 5329, 3881, 6866, 127, 7145, 925, 1012, 3551, 5031, 152, 2955, 4775, 5417, 4292, 4636, 4614, 5155, 5369, 4830, 6033, 1566, 696, 664, 6524, 888, 5501, 1475, 202, 34, 5315, 3492, 5204, 7197, 1368, 2354, 6027, 1763, 4857, 5048, 674, 2083, 5325, 760, 6698, 3803, 4421, 2781, 4620, 871, 5985, 2317, 1118, 1643, 2320, 6669, 6475, 4337, 6201, 2009, 139, 6248, 4032, 4932, 3493, 988, 41, 1364, 4875, 5747, 3136, 5140, 2760, 3332, 6785, 399, 2241, 2799, 6431, 5661, 2990, 2890, 118, 4319, 7360, 88, 6474, 6302, 4092, 4586, 3213, 1571, 5734, 806, 4202, 2905, 6082, 5157, 3182, 2505, 891, 1397, 4445, 1765, 5541, 4349, 6093, 4471, 5274, 2348, 6095, 648, 5799, 3257, 6639, 6183, 7311, 119, 1353, 427, 6706, 2919, 4862, 1042, 3972, 7074, 1002, 994, 3394, 224, 4551, 1208, 5849, 6076, 2097, 3035, 3342, 3358, 5942, 3362, 2714, 7344, 5058, 3130, 2939, 86, 2181, 2479, 7117, 4124, 4131, 1221, 1237, 6145, 4716, 2575, 2115, 3072, 3586, 7322, 4659, 5002, 2946, 6957, 2658, 3892, 2457, 4580, 1481, 4991, 6828, 5753, 7120, 6804, 775, 5816, 986, 6551, 6723, 661, 6455, 1511, 1403, 6122, 3623, 5663, 3294, 6383, 1845, 6239, 550, 1045, 4767, 4026, 1679, 3732, 6308, 2436, 6424, 2216, 700, 6694, 2012, 7002, 5493, 3243, 732, 1636, 5700, 4395, 6621, 1060, 1203, 2991, 2782, 1476, 1061, 3491, 6193, 3350, 4075, 5548, 5918, 788, 3216, 3964, 6500, 3572, 7175, 1983, 6332, 2145, 962, 3650, 655, 2243, 4581, 624, 5653, 6863, 1533, 1747, 6254, 1048, 1170, 4900, 6326, 6226, 2683, 7298, 1675, 2351, 1352, 2586, 3427, 1592, 2284, 5656, 2259, 5943, 570, 4526, 3084, 6115, 7239, 1519, 6688, 4253, 3922, 5992, 5621, 5441, 3245, 3661, 7266, 3767, 1851, 2906, 2258, 4283, 2785, 5779, 1195, 4699, 5575, 982, 359, 1540, 2404, 3070, 6247, 6191, 4684, 1723, 968, 6236, 731, 6568, 2064, 535, 4546, 3836, 1411, 1287, 6528, 972, 4945, 903, 2534, 1960, 3971, 2987, 916, 2756, 1545, 6101, 5607, 2401, 2886, 5363, 2857, 2956, 5531, 6574, 11, 5929, 683, 3620, 2500, 4801, 4166, 1310, 5358, 6653, 7229, 3829, 7279, 1811, 896, 6825, 430, 1232, 386, 5423, 1104, 1255, 5745, 5759, 1919, 1275, 2205, 4150, 3658, 5565, 3108, 2410, 5546, 384, 2608, 3259, 5400, 6596, 6715, 2161, 3688, 3010, 5350, 6790, 6502, 2718, 2581, 805, 2597, 3055, 3443, 1282, 1673, 1472, 5644, 6007, 3236, 6977, 5916, 801, 4714, 4513, 5442, 4378, 779, 3745, 5712, 1913, 6576, 350, 3663, 556, 4937, 1334, 4992, 4654, 5682, 2504, 1503, 3179, 2306, 3438, 2316, 3410, 2362, 3594, 1956, 7132, 3333, 65, 2573, 1691, 6910, 6778, 6729, 4139, 5670, 1513, 1637, 217, 5384, 5258, 3885, 4025, 680, 2017, 4023, 2155, 2547, 4639, 5739, 4720, 1167, 549, 3305, 658, 5522, 6602, 3118, 6997, 7336, 5804, 2861, 1106, 4398, 5494, 4391, 2822, 3040, 1281, 1276, 2897, 7287, 5537, 7094, 3074, 4104, 4746, 4778, 3651, 2202, 2995, 4686, 4818, 3195, 1266, 306, 5355, 5595, 3354, 1902, 4365, 1656, 2696, 4696, 4705, 470, 1819, 5478, 3401, 6906, 2931, 4897, 571, 4476, 6146, 4978, 534, 3599, 7297, 2875, 3237, 4638, 3199, 3753, 7391, 2480, 6722, 4606, 3039, 4161, 4456, 436, 275, 3825, 1166, 4140, 3051, 3498, 6922, 5736, 1077, 2085, 1191, 5005, 424, 6157, 5981, 5626, 3025, 3145, 1988, 4329, 733, 5517, 6654, 4873, 2076, 2688, 5309, 2600, 7080, 6392, 6309, 4642, 6411, 6544, 5924, 5186, 5809, 4617, 1485, 4943, 1454, 4501, 7013, 272, 2625, 6117, 1804, 3552, 501, 5749, 6413, 2382, 2580, 3839, 6339, 922, 2541, 961, 742, 7215, 4887, 6213, 6813, 4295, 2176, 628, 4182, 1797, 1985, 7252, 4681, 6320, 6386, 1405, 400, 3181, 6540, 3874, 7043, 7182, 1041, 524, 3852, 6034, 339, 154, 3989, 6284, 1046, 1579, 4173, 4208, 1507, 6364, 5185, 4154, 5142, 398, 1962, 2705, 1122, 998, 2312, 6004, 1947, 5095, 2257, 2553, 1796, 6000, 2137, 2280, 5419, 5628, 5482, 4291, 3336, 5589, 6927, 765, 2223, 326, 6468, 4651, 506, 2836, 2588, 4285, 5458, 2228, 1268, 1064, 2650, 6757, 5353, 3367, 7044, 5662, 5742, 547, 2973, 1963, 840, 7335, 6437, 2300, 3217, 2619, 2036, 7393, 7079, 3749, 1323, 1274, 7351, 278, 899, 812, 691, 6655, 2558, 2056, 5263, 1668, 5601, 3818, 3848, 2560, 5580, 705, 5251, 3722, 1483, 5600, 5466, 316, 3797, 4983, 778, 5993, 1435, 5447, 1550, 5047, 2513, 2860, 2994, 2536, 3615, 3253, 7072, 6659, 7148, 2075, 6107, 6930, 2884, 4993, 1029, 2777, 3576, 1420, 452, 2671, 2200, 336, 7258, 2960, 1662, 1285, 4400, 5728, 698, 3056, 2835, 6567, 5668, 5888, 3842, 802, 2666, 3185, 6925, 3005, 1258, 2806, 931, 1143, 6393, 1509, 966, 2127, 7250, 1260, 2615, 428, 4244, 6059, 6883, 2144, 2525, 3718, 4138, 4188, 1380, 1996, 768, 4412, 2460, 4571, 4510, 1556, 1109, 6788, 2967, 606, 6039, 6085, 5164, 892, 7285, 4925, 3591, 2187, 3701, 5334, 942, 7372, 1039, 5719, 4156, 3467, 4030, 7276, 830, 2235, 5533, 5582, 1670, 4761, 1415, 1770, 5819, 3094, 5735, 3224, 2631, 846, 1272, 2477, 6408, 1634, 455, 6525, 1092, 228, 2234, 2969, 162, 5932, 307, 4583, 2888, 4987, 6210, 3298, 4514, 6532, 4819, 3230, 3943, 1079, 4309, 3565, 5976, 2880, 1559, 5850, 887, 3528, 3826, 566, 5803, 681, 5933, 6919, 15, 2844, 3905, 2129, 2908, 2138, 1978, 2364, 3251, 3999, 3932, 1394, 1069, 1871, 877, 4568, 2363, 2636, 5180, 2584, 4833, 4891, 6833, 1903, 3496, 6147, 823, 7190, 344, 3524, 6137, 1950, 707, 1482, 822, 36, 6569, 1470, 3187, 4811, 503, 1813, 4417, 295, 2872, 5076, 5893, 2948, 3783, 4590, 1328, 6170, 6001, 1588, 504, 1738, 5250, 6951, 5967, 876, 6644, 5213, 1532, 3505, 6774, 53, 1193, 2842, 659, 7035, 5459, 2166, 6868, 3287, 7361, 6888, 54, 3607, 5470, 3533, 5192, 1549, 3459, 5135, 2057, 1752, 4460, 1467, 2902, 5089, 4542, 2058, 5518, 7146, 3220, 280, 6128, 1965, 4148, 5225, 5071, 3894, 2746, 6974, 3326, 676, 1650, 5978, 7126, 7227, 4212, 5709, 6356, 5652, 1351, 4411, 443, 5414, 3982, 7121, 1838, 6802, 5326, 5388, 893, 5844, 4160, 6652, 1186, 3917, 5310, 1033, 218, 1456, 2598, 4878, 2106, 2943, 6699, 884, 332, 1448, 6295, 6132, 3457, 6344, 1068, 1964, 4707, 3532, 1972, 2947, 2646, 7248, 4382, 6727, 5209, 4119, 1117, 3883, 6325, 1552, 3720, 2812, 1543, 4742, 2915, 1932, 498, 2692, 2557, 5532, 1517, 5448, 4167, 7162, 2543, 5490, 5874, 5923, 1535, 587, 5552, 2791, 2063, 5969, 6739, 3838, 95, 3049, 5188, 2830, 1864, 5187, 3173, 2328, 2810, 7352, 303, 2469, 2675, 4111, 472, 6656, 4888, 4589, 38, 380, 256, 268, 4001, 996, 4420, 5128, 5664, 7261, 7027, 7053, 5569, 7388, 687, 1722, 6668, 2976, 2767, 898, 4116, 6509, 5500, 1581, 5173, 3926, 2871, 6487, 1084, 3800, 6849, 3811, 625, 1814, 5609, 6748, 4189, 6495, 2229, 2450, 7187, 1953, 3901, 5342, 2932, 6990, 323, 5152, 6837, 3304, 1688, 5741, 1957, 1158, 4427, 2974, 6098, 4916, 2893, 2629, 2334, 7370, 689, 4517, 1031, 7144, 199, 7366, 5163, 3232, 5691, 7220, 3092, 6421, 4254, 2304, 5733, 572, 7033, 69, 1088, 4922, 2268, 5988, 1036, 6608, 4172, 2042, 3218, 6459, 7377, 5205, 4576, 2644, 2131, 4405, 1441, 1589, 406, 1325, 4303, 5418, 1914, 5261, 245, 5045, 2337, 1217, 7219, 4868, 2285, 4082, 440, 6377, 4715, 4120, 7340, 4352, 57, 745, 5911, 3088, 29, 2773, 4982, 4423, 1768, 2269, 2266, 7354, 557, 5115, 878, 2190, 3507, 4960, 2191, 6819, 6575, 6172, 5587, 50, 3739, 781, 7369, 3934, 1001, 1005, 621, 4626, 5549, 4736, 3317, 3271, 1450, 6152, 2731, 6967, 603, 4350, 971, 2763, 5481, 5754, 2722, 2413, 232, 5714, 907, 5830, 1597, 5798, 3760, 4554, 4145, 1893, 4685, 3249, 1620, 3009, 2214, 627, 7321, 836, 2175, 1809, 348, 6756, 4910, 831, 1677, 2123, 2787, 2066, 3366, 845, 2758, 3835, 70, 269, 1598, 5807, 2959, 211, 103, 3843, 5557, 3727, 4908, 4874, 7234, 2255, 2704, 6175, 290, 7166, 6913, 6353, 1779, 1089, 6593, 5161, 5645, 688, 5994, 4538, 6628, 5284, 701, 7375, 5184, 3726, 2380, 934, 5837, 409, 5298, 918, 3612, 3912, 5592, 2526, 1270, 7210, 4708, 2310, 5074, 4889, 2503, 3952, 6363, 875, 5698, 5019, 6163, 6016, 3927, 857, 5721, 6961, 2673, 2095, 291, 1016, 190, 3375, 2726, 4047, 6497, 1175, 5412, 5222, 5786, 6657, 1739, 1225, 1013, 4764, 677, 4122, 1810, 107, 357, 3033, 3434, 4262, 5084, 4401, 1531, 5014, 3700, 3602, 3477, 4049, 5480, 4109, 4750, 412, 4351, 6419, 5434, 143, 693, 5724, 2107, 2174, 5275, 2011, 4752, 192, 4871, 3240, 4560, 1129, 5286, 2837, 2859, 3792, 5581, 5393, 6423, 993, 2970, 4919, 3439, 2248, 6917, 3649, 51, 6490, 4532, 2424, 5314, 6560, 863, 5881, 2492, 2972, 1827, 6355, 2635, 5937, 6736, 4774, 4257, 4324, 7233, 6550, 6167, 6947, 7277, 2523, 1209, 5281, 5215, 601, 2162, 2521, 1623, 4788, 4144, 7009, 5443, 5233, 3673, 4071, 4397, 761, 5097, 6279, 6164, 6052, 4266, 2823, 7331, 766, 3158, 2400, 7115, 983, 3596, 1148, 2501, 1986, 3407, 3978, 5765, 4841, 7389, 2711, 4290, 3531, 5694, 1721, 2217, 4293, 5038, 4593, 1609, 3445, 4990, 3516, 6135, 746, 1512, 4573, 6263, 1881, 6134, 251, 3270, 509, 414, 141, 7325, 1808, 4693, 4344, 6080, 4679, 7342, 255, 3032, 3841, 3876, 1582, 480, 91, 1284, 170, 4392, 1833, 2794, 4530, 3202, 1242, 5755, 4336, 3937, 7181, 970, 3383, 1837, 2230, 5983, 1083, 5904, 4605, 2655, 5253, 6418, 2340, 6814, 3101, 6935, 6943, 6015, 1144, 5987, 5729, 6538, 2319, 1330, 4563, 6559, 1794, 5015, 5813, 5752, 5612, 6886, 6616, 1495, 3973, 7003, 838, 413, 6632, 5392, 6328, 5547, 7383, 3391, 5651, 849, 234, 4822, 1263, 4918, 6558, 6270, 3382, 3781, 6329, 4823, 259, 623, 1383, 4507, 7350, 930, 4063, 4396, 5179, 7102, 6552, 3629, 4480, 2982, 2069, 2433, 6988, 1072, 5774, 42, 6434, 1504, 1683, 1705, 713, 4547, 2341, 1003, 5962, 7075, 3740, 1880, 3156, 2168, 1103, 7398, 6905, 3512, 5535, 6847, 4970, 6909, 7328, 4239, 6950, 2332, 1020, 1548, 7082, 7054, 90, 4077, 3879, 7183, 5020, 6249, 3568, 6793, 7306, 4164, 2678, 2343, 5479, 6090, 3510, 4641, 6945, 3725, 7185, 3154, 492, 4892, 2033, 327, 1874, 5046, 1292, 3329, 6557, 6397, 1645, 489, 1693, 3111, 1063, 4216, 2754, 772, 2686, 2157, 5039, 7212, 1114, 6217, 3475, 3747, 4129, 2648, 4203, 2136, 2016, 2538, 6926, 5028, 5099, 1714, 474, 3164, 5643, 5835, 2814, 3549, 6398, 3137, 2019, 4185, 1004, 2146, 5399, 6162, 820, 3833, 3891, 3105, 2509, 610, 1982, 5957, 6769, 330, 2786, 6529, 6615, 5808, 1640, 5030, 1728, 5667, 5787, 4224, 2159, 1306, 5403, 2387, 1497, 4369, 3683, 5513, 6844, 583, 3579, 6381, 1644, 4835, 2301, 6334, 6719, 2856, 2221, 4860, 1824, 4100, 4737, 7396, 1075, 5100, 4495, 2287, 4531, 7171, 2979, 1888, 6588, 718, 3397, 5212, 1140, 1918, 842, 3379, 7016, 5422, 7139, 5271, 1729, 5300, 1800, 2497, 3642, 1534, 3067, 6716, 6885, 3466, 3113, 4453, 1906, 4284, 7176, 4815, 4594, 6118, 3995, 2981, 5339, 3452, 6627, 5952, 5290, 3559, 5599, 2862, 4312, 3539, 5322, 201, 1231, 5852, 2336, 6099, 3254, 6054, 4666, 4728, 2911, 6311, 2945, 936, 355, 4430, 4024, 3619, 1576, 4008, 3962, 5082, 4676, 5124, 6428, 3264, 335, 3703, 4308, 787, 780, 5247, 3201, 4687, 2466, 7231, 5688, 853, 4773, 6046, 5869, 215, 1477, 453, 2088, 3192, 2930, 5553, 844, 1707, 1911, 1884, 4034, 919, 4615, 3830, 2015, 1369, 172, 2314, 5793, 1142, 3228, 6690, 6518, 6078, 1055, 5861, 6605, 4060, 629, 543, 6537, 4437, 2415, 4555, 2790, 6597, 6920, 5248, 1250, 2989, 5242, 2283, 3970, 2944, 35, 6402, 2156, 1121, 1499, 3261, 3269, 3824, 362, 2289, 7109, 7081, 3774, 2642, 2840, 3415, 2957, 5788, 5848, 6582, 7116, 3212, 3343, 208, 6275, 3110, 2618, 7218, 3168, 4855, 641, 4880, 4854, 6481, 5433, 5907, 1853, 2941, 7078, 3521, 2997, 3674, 1702, 3659, 7365, 4915, 5011, 1994, 5177, 2964, 6106, 6996, 4432, 1975, 518, 6351, 3086, 1757, 3462, 5056, 5013, 6141, 4539, 2815, 1896, 6516, 3093, 6798, 5105, 6864, 7108, 5666, 4816, 5875, 4300, 2350, 5824, 2326, 2606, 2282, 5576, 7042, 1222, 3193, 1240, 2290, 1277, 3509, 6266, 1971, 2550, 6979, 3585, 584, 2511, 1781, 2451, 5956, 6116, 2783, 7302, 5289, 1527, 2759, 6143, 6379, 7092, 1309, 4508, 3338, 5903, 2567, 699, 6340, 6048, 4898, 3069, 3704, 7058, 6506, 6114, 5948, 5894, 5025, 2391, 2384, 5812, 2141, 2041, 4197, 3583, 7249, 5371, 3024, 2809, 6915, 4142, 5842, 3847, 2694, 3721, 258, 4175, 2035, 1223, 2942, 2516, 3857, 1426, 5486, 7026, 1360, 1389, 193, 2082, 4126, 6375, 4002, 580, 1817, 6291, 4054, 4962, 5437, 2922, 4796, 2751, 4649, 3252, 4743, 2524, 2087, 2090, 6018, 6021, 277, 6077, 4475, 3282, 6998, 2439, 2074, 3990, 178, 318, 3188, 5288, 1587, 1071, 5603, 6642, 3144, 1130, 3935, 5883, 7232, 832, 5838, 447, 5931, 2178, 337, 342, 117, 3987, 1349, 6271, 165, 4325, 1505, 1165, 4338, 5610, 81, 4072, 1568, 6486, 2172, 7084, 3911, 5527, 678, 4653, 6815, 6648, 4388, 1458, 5405, 3002, 3308, 7269, 7224, 4591, 5769, 4820, 68, 324, 3525, 4155, 835, 7323, 2834, 813, 1034, 4849, 5390, 7405, 6262, 6100, 2938, 4541, 1400, 4115, 3109, 4655, 130, 1793, 4562, 7130, 6433, 3041, 151, 5207, 2870, 1398, 7107, 2445, 1803, 2061, 2331, 6168, 3540, 4901, 6349, 1180, 3429, 5785, 3152, 7170, 6151, 6973, 4467, 2476, 712, 605, 2494, 3485, 1831, 3444, 4726, 1388, 4497, 6124, 3723, 1577, 2848, 4567, 2201, 319, 5029, 1595, 1799, 4086, 7368, 4650, 1108, 2709, 1028, 4864, 4068, 3301, 632, 6752, 5149, 3647, 2712, 214, 6221, 4106, 2512, 4927, 6324, 4941, 1256, 6096, 4000, 2425, 6049, 5259, 1442, 4955, 6189, 727, 5619, 4179, 1044, 4977, 415, 897, 6127, 6342, 3095, 3435, 1554, 3695, 2530, 2609, 1934, 2473, 166, 3341, 2594, 4612, 7067, 2305, 5895, 6150, 5902, 5009, 3174, 3075, 4632, 7255, 5676, 843, 6181, 4180, 4611, 4827, 5489, 5131, 2236, 4360, 6102, 5867, 4858, 4055, 3330, 1867, 27, 1584, 626, 5111, 6444, 5563, 5116, 361, 5139, 5054, 6899, 5305, 6317, 6435, 914, 1886, 2774, 738, 4975, 2294, 6709, 6278, 3140, 4280, 3631, 4123, 2788, 287, 2614, 4536, 1035, 5980, 5627, 6011, 2883, 7403, 3526, 4784, 239, 6953, 3421, 292, 465, 2453, 1136, 6918, 3578, 5757, 2813, 7111, 3598, 1865, 3503, 5088, 2180, 4457, 4954, 6893, 1289, 4333, 5859, 3393, 106, 5306, 2148, 1625, 6081, 1652, 4697, 1058, 4081, 2578, 402, 5241, 5512, 685, 3420, 285, 358, 3083, 4416, 2227, 1462, 870, 7243, 418, 6161, 2052, 2339, 1639, 3534, 3979, 4523, 2816, 5584, 3320, 2215, 1870, 6436, 2151, 3916, 6312, 2394, 3167, 6405, 3104, 2653, 4228, 231, 2003, 1493, 4700, 5536, 667, 6008, 3515, 6186, 4780, 1547, 1529, 6511, 5391, 3171, 6031, 3425, 4647, 3256, 3956, 5118, 6562, 3778, 3548, 3480, 2005, 2039, 6911, 3471, 1655, 2894, 6647, 1961, 3820, 6565, 1909, 3996, 2549, 2599, 1498, 1376, 5854, 3268, 2887, 990, 6321, 1648, 6966, 3955, 545, 5491, 161, 851, 4200, 4558, 5431, 5365, 5858, 665, 2112, 1751, 493, 2426, 2738, 3947, 3828, 2531, 4682, 2898, 611, 1262, 4051, 3898, 927, 5081, 5624, 3639, 7301, 7213, 100, 6314, 2502, 7103, 5847, 5672, 3601, 2771, 3993, 6230, 2904, 7063, 6705, 1152, 346, 4847, 1302, 2109, 1613, 3204, 3523, 1711, 3293, 7073, 6238, 2390, 1317, 3146, 2843, 4462, 4674, 2142, 3141, 5354, 6665, 3724, 4447, 1440, 247, 4481, 6185, 1361, 2768, 2110, 4834, 1428, 928, 6229, 3453, 3027, 5995, 4545, 4407, 2378, 1243, 1198, 1940, 526, 1603, 3166, 6208, 6224, 1780, 3708, 500, 3292, 4466, 1633, 6120, 1053, 3640, 5485, 2375, 5457, 3364, 5695, 2086, 5720, 4557, 2753, 5892, 1854, 4186, 6180, 426, 2925, 123, 2912, 6949, 1882, 5283, 3461, 7318, 1649, 4158, 364, 2992, 1622, 4793, 4206, 5367, 2798, 7085, 5085, 5109, 4794, 1372, 4723, 3031, 4624, 6155, 265, 6892, 7373, 4459, 4237, 2408, 2963, 4442, 85, 1744, 5764, 1885, 6747, 3012, 101, 926, 5970, 3575, 6416, 4353, 2697, 5615, 7345, 6036, 4944, 2007, 6607, 5820, 6598, 3618, 1600, 1546, 6010, 3821, 3900, 5823, 6097, 163, 1892, 5227, 5814, 4926, 3986, 4757, 2399, 7118, 4070, 1989, 2434, 5509, 4520, 4564, 841, 719, 955, 6677, 4528, 3866, 546, 1920, 2617, 957, 4091, 7136, 6330, 5634, 6013, 131, 3686, 1565, 947, 3000, 2961, 5370, 4011, 4950, 3314, 1715, 3953, 1606, 7194, 6493, 7022, 3018, 2164, 6322, 1239, 1236, 2743, 6297, 4965, 5137, 551, 333, 5677, 3678, 6144, 1703, 3595, 3664, 2177, 6087, 1542, 7163, 7374, 2388, 7161, 490, 1772, 2377, 6112, 1095, 4670, 340, 4498, 582, 2261, 2051, 5771, 4988, 585, 3112, 4118, 4232, 7129, 4744, 3606, 1924, 902, 860, 4959, 1564, 4, 1969, 5629, 2242, 6776, 6473, 4766, 6872, 7379, 1379, 948, 6242, 5604, 4094, 4494, 7018, 521, 6916, 1861, 2589, 6761, 5960, 4629, 2874, 200, 403, 5770, 4727, 1097, 5101, 5497, 5873, 5160, 4271, 3078, 2273, 7251, 694, 2371, 7064, 1160, 2093, 368, 3556, 4297, 3385, 2383, 7346, 2742, 3058, 5905, 1774, 4574, 959, 1197, 3335, 2006, 6636, 6207, 153, 96, 6204, 2059, 941, 7047, 4225, 2049, 1875, 3707, 1553, 2841, 6055, 2265, 3802, 5780, 874, 6590, 6750, 5151, 3389, 1632, 1348, 1586, 394, 6842, 5839, 807, 80, 4861, 4458, 1917, 2805, 7119, 6942, 1344, 6037, 4343, 1401, 1619, 4170, 7089, 5633, 1660, 7059, 1718, 6843, 531, 2247, 66, 1406, 3484, 6503, 3751, 3699, 2360, 6182, 3504, 2914, 1646, 6130, 4906, 137, 2730, 1214, 1801, 7367, 4712, 5026, 1862, 157, 997, 594, 6693, 3617, 3205, 3670, 3656, 2222, 3790, 4259, 6674, 1128, 7041, 5605, 2001, 6313, 4066, 6731, 7401, 6514, 724, 2245, 3906, 4678, 4320, 1984, 5235, 4061, 2389, 6148, 558, 1159, 1452, 4733, 3071, 6675, 7019, 7083, 438, 3775, 2344, 4812, 6622, 395, 3893, 7206, 2885, 206, 4255, 5167, 522, 7173, 2002, 647, 5021, 221, 3226, 4569, 1335, 4121, 5346, 2770, 325, 7291, 7310, 3116, 4804, 3120, 4967, 7104, 3034, 3345, 3750, 3323, 1120, 6993, 2098, 6223, 2349, 6250, 6755, 485, 1238, 446, 6483, 3322, 4597, 3789, 2691, 530, 4527, 3960, 1816, 3662, 2467, 6014, 538, 3312, 3037, 1019, 2710, 4760, 4850, 4087, 1887, 799, 5760, 5460, 864, 253, 4141, 4675, 945, 1074, 7060, 1506, 1359, 3490, 4770, 5870, 5234, 6105, 6501, 5715, 3638, 2213, 1234, 5347, 5352, 2744, 484, 3908, 4730, 6159, 2596, 444, 4114, 1169, 6572, 3337, 1135, 7228, 3465, 421, 4053, 6190, 7330, 3340, 6084, 388, 3361, 4928, 301, 757, 7048, 3170, 5725, 1823, 2443, 1370, 5763, 4258, 536, 668, 4105, 7376, 3967, 6713, 4093, 3804, 3059, 6232, 6430, 329, 1377, 4174, 6121, 6880, 391, 176, 1560, 1320, 2054, 3930, 264, 1974, 1836, 1194, 5094, 6023, 4096, 3768, 5678, 5871, 4911, 122, 2124, 6932, 4688, 797, 754, 2323, 3123, 6887, 4931, 3693, 1541, 6484, 2367, 1407, 2211, 4799, 6984, 4298, 6073, 6896, 6032, 2084, 6631, 3414, 5001, 5598, 3200, 3944, 4132, 1492, 3222, 6176, 2045, 4948, 6577, 3538, 5439, 3238, 5697, 4327, 4342, 48, 2021, 1536, 3392, 2652, 2027, 3209, 5778, 935, 901, 6679, 794, 5170, 3527, 6836, 261, 4502, 7382, 5146, 1387, 5069, 6799, 4149, 723, 3208, 7004, 4137, 6296, 1305, 5344, 187, 1948, 4125, 286, 6047, 420, 1402, 4275, 1446, 793, 5429, 1465, 1486, 5117, 4368, 6999, 3214, 4984, 3006, 6445, 5641, 2532, 4582, 7207, 5110, 3588, 4976, 2196, 2966, 7055, 2918, 2449, 2800, 4701, 2037, 479, 2765, 3225, 110, 6184, 4998, 4973, 5984, 2275, 5650, 771, 4346, 1350, 6829, 6453, 828, 1014, 1510, 1218, 6480, 4601, 3106, 1502, 3689, 6612, 328, 6403, 2219, 1820, 6231, 2004, 4260, 7134, 5287, 2698, 679, 4375, 4059, 5540, 5997, 431, 4660, 3634, 1299, 2978, 2672, 3242, 2975, 6450, 3266, 6395, 4662, 4917, 6494, 2298, 6197, 3577, 5762, 5846, 6006, 6733, 581, 3961, 2634, 915, 7384, 6555, 5606, 6173, 1641, 2281, 2745, 7157, 7400, 4553, 408, 4637, 4972, 7032, 1294, 5404, 22, 1771, 5052, 6795, 6035, 2592, 6873, 5973, 5144, 6623, 5223, 4808, 1296, 630, 7198, 7097, 2167, 5953, 7192, 3297, 5649, 2267, 2398, 2186, 1418, 60, 821, 6846, 260, 6822, 6192, 2852, 7140, 7286, 4515, 5792, 1037, 7314, 3474, 3448, 5316, 3729, 6794, 3698, 317, 4359, 7320, 3584, 1689, 5153, 4022, 5768, 1815, 1147, 2121, 1427, 3281, 4279, 6543, 3741, 4345, 2910, 6429, 7077, 1672, 4242, 5766, 1852, 6879, 3827, 2357, 4477, 3636, 477, 2117, 5321, 666, 4996, 3968, 6009, 6293, 1916, 320, 5456, 6940, 2132, 1712, 5492, 4157, 6350, 7390, 4939, 1785, 5817, 4117, 636, 6446, 6583, 3696, 6599, 2385, 3197, 4005, 4683, 2563, 2472, 7005, 6028, 4276, 4537, 3566, 810, 6771, 4776, 249, 281, 1952, 750, 6476, 1176, 6214, 3021, 3087, 2719, 5236, 6624, 1366, 3365, 4215, 216, 564, 6903, 2406, 3581, 2639, 62, 2749, 3388, 5387, 3702, 3029, 576, 6303, 6045, 1574, 1181, 1839, 4256, 5577, 6646, 909, 3754, 3497, 1575, 867, 2359, 6069, 411, 2179, 26, 3121, 7208, 6274, 908, 3063, 4380, 6663, 371, 3284, 1494, 1859, 5718, 1599, 1293, 5102, 3378, 2951, 6821, 5357, 6783, 4448, 5617, 1425, 3896, 4383, 7105, 5066, 2102, 6662, 1760, 2428, 5266, 1052, 3263, 7267, 7142, 1570, 6779, 5057, 5516, 5331, 4250, 6564, 3752, 1841, 5476, 4097, 1321, 2564, 4817, 1737, 905, 1945, 6333, 6852, 2203, 2507, 4668, 449, 1185, 6898, 7216, 6198, 5872, 5375, 4277, 5761, 2263, 2192, 7065, 4505, 803, 4600, 5560, 1149, 5220, 6661, 2147, 2661, 257, 6053, 511, 4362, 5150, 6478, 4789, 1720, 1056, 6634, 5006, 3460, 586, 4304, 6129, 134, 2370, 4876, 2953, 7209, 774, 2210, 5343, 3289, 819, 146, 1661, 937, 1439, 5689, 5687, 6394, 3950, 5996, 5571, 703, 3907, 6025, 2309, 1, 6928, 1011, 6589, 1696, 2565, 2089, 352, 5062, 5797, 6074, 4824, 3470, 6300, 798, 5539, 6044, 1981, 4446, 5068, 1783, 5237, 3046, 804, 1931, 3716, 473, 1111, 1189, 4012, 6060, 3483, 6989, 5864, 3432, 1980, 366, 3890, 5623, 3582, 1938, 6874, 2474, 2517, 7275, 4043, 4740, 4410, 2062, 3873, 940, 7262, 3805, 432, 4735, 592, 5593, 3247, 1332, 6955, 4677, 2940, 20, 1022, 468, 63, 7245, 1746, 3942, 6689, 6975, 5572, 748, 3399, 3014, 4595, 7196, 4492, 6901, 5195, 6286, 2139, 3426, 862, 7150, 593, 7316, 2254, 6460, 454, 1157, 1437, 4570, 2602, 5897, 1590, 3321, 3346, 6773, 1362, 1413, 299, 2068, 4147, 5262, 3880, 4790, 2169, 1023, 4231, 4843, 912, 2845, 4102, 4221, 5018, 3983, 702, 1583, 5515, 58, 5484, 6202, 6897, 1593, 2431, 2272, 6520, 747, 7341, 1520, 635, 767, 7056, 575, 5963, 6154, 2681, 6158, 1698, 1907, 6094, 4311, 4419, 5966, 370, 1261, 3048, 3569, 5461, 2867, 529, 3991, 4913, 6489, 6384, 2444, 4489, 1254, 4623, 4807, 2647, 1569, 1201, 338, 662, 4665, 461, 3359, 5257, 3969, 7062, 2916, 6554, 185, 2435, 3419, 3764, 2748, 939, 5224, 2295, 4339, 6089, 539, 5635, 25, 1479, 2518, 1846, 2802, 6534, 2881, 1733, 3671, 1125, 653, 4826, 3036, 2026, 4596, 3353, 235, 644, 5616, 5979, 5625, 3241, 3050, 1776, 2060, 367, 263, 6443, 5520, 1461, 1946, 6573, 4803, 6553, 5471, 3468, 487, 3780, 906, 3128, 5591, 6808, 1628, 5690, 4866, 7288, 1850, 6426, 6980, 5083, 1057, 4534, 967, 6587, 4671, 7184, 4347, 6921, 2094, 4738, 3853, 6701, 1249, 6438, 975, 3939, 4490, 3191, 5252, 5465, 486, 3770, 6613, 3705, 2508, 7099, 4450, 6817, 2792, 1091, 3369, 5659, 6319, 1526, 4464, 2072, 1161, 4031, 4085, 3920, 4233, 3737, 2664, 1059, 609, 6235, 7061, 2264, 3861, 7309, 7155, 1378, 5303, 4153, 5648, 6991, 1489, 1642, 2208, 7051, 4040, 2342, 2778, 4672, 2801, 5889, 2772, 5887, 3455, 669, 3100, 5505, 423, 3600, 1132, 2669, 407, 6710, 2950, 6758, 3344, 1692, 2761, 6992, 2866, 3446, 5328, 4667, 2700, 6508, 219, 2025, 6609, 5708, 3544, 2566, 7086, 3589, 4088, 4800, 6976, 3096, 369, 6323, 4909, 4575, 4724, 1847, 2044, 1252, 5502, 2899, 1488, 2793, 4512, 467, 6667, 2850, 2361, 2764, 1314, 3279, 4550, 6835, 773, 37, 4306, 7265, 28, 964, 6831, 254, 2824, 3017, 5432, 3160, 204, 2603, 5449, 4465, 2556, 3711, 2616, 3280, 6086, 5426, 3709, 1018, 1271, 1858, 6366, 3633, 6412, 3904, 6764, 3395, 6368, 5040, 3625, 4340, 5268, 6832, 5657, 5674, 834, 5320, 3902, 2855, 5683, 3436, 3975, 6775, 4511, 59, 7284, 6149, 5775, 3713, 4831, 1246, 5570, 1127, 6337, 2096, 3396, 2551, 4765, 2728, 6767, 6578, 5064, 1607, 3450, 94, 2933, 5427, 2274, 4307, 7203, 3653, 7195, 6994, 147, 1311, 5805, 7404, 5396, 3992, 3871, 850, 5845, 5917, 3285, 4248, 2607, 21, 3795, 3347, 4886, 1153, 1204, 1094, 5203, 6801, 4994, 5908, 4771, 5906, 1235, 3756, 784, 1308, 2324, 495, 573, 429, 4722, 1500, 4390, 1954, 4358, 7174, 5219, 4729, 6276, 6618, 4592, 4191, 6316, 532, 6745, 4014, 5834, 3854, 950, 5564, 5148, 770, 1199, 4579, 4627, 2651, 2776, 6396, 3272, 5228, 3273, 3542, 3068, 2796, 7031, 4282, 129, 416, 7008, 4474, 3423, 6153, 741, 7037, 2393, 5276, 2807, 4190, 1339, 3007, 72, 6410, 410, 2780, 1244, 1414, 1216, 2917, 7362, 6958, 2896, 3172, 4658, 3817, 6720, 1090, 1015, 2633, 7106, 3980, 5681, 7339, 1133, 2417, 1322, 3919, 1183, 2333, 4128, 1762, 6482, 5777, 6042, 3844, 4433, 3478, 7122, 2372, 3570, 3522, 3738, 4924, 3687, 471, 4838, 4374, 1390, 6280, 1162, 2869, 4648, 4044, 6812, 6374, 160, 2491, 43, 3945, 2296, 2160, 5182, 2545, 5925, 4544, 2065, 4217, 663, 4035, 5776, 1638, 297, 1868, 4207, 4193, 5395, 5602, 1993, 4957, 4621, 4441, 5717, 1894, 343, 7007, 2891, 5826, 2775, 783, 5840, 3319, 671, 4238, 1054, 3654, 715, 2119, 6409, 4633, 5991, 3380, 4361, 2952, 5420, 6237, 5524, 3324, 4235, 1113, 591, 1567, 1713, 3644, 537, 7293, 5133, 248, 6072, 77, 6806, 730, 5430, 2414, 6686, 1327, 3630, 385, 2676, 4813, 2554, 4218, 3306, 3863, 5229, 5249, 1392, 1179, 1312, 722, 1444, 5796, 5091, 4938, 1163, 3384, 2262, 1908, 196, 2689, 2226, 6169, 3417, 3117, 4756, 759, 1342, 2345, 3215, 7386, 6287, 1877, 5330, 5751, 5806, 2876, 404, 3053, 3868, 5647, 6820, 4559, 4165, 67, 126, 6531, 5525, 52, 2984, 3177, 459, 5240, 1391, 437, 2397, 4483, 83, 1393, 2762, 4323, 6260, 1267, 1684, 2437, 5277, 2030, 2133, 3481, 5566, 2224, 3609, 3816, 4763, 638, 2934, 5499, 6406, 6005, 2455, 5336, 1257, 6797, 4747, 7023, 4065, 184, 5381, 6637, 488, 354, 5618, 6491, 2677, 360, 3132, 2958, 6857, 302, 1027, 2949, 577, 2079, 3812, 5529, 6240, 4143, 6062, 2184, 2907, 1610, 2613, 1695, 1464, 1594, 1806, 1139, 5080, 220, 7353, 6336, 5256, 56, 1626, 4751, 3274, 5421, 4905, 2937, 73, 2687, 1743, 3402, 541, 1627, 2286, 16, 1979, 6673, 2246, 708, 7172, 2818, 872, 1521, 6367, 6792, 1474, 243, 5703, 597, 1848, 3951, 5125, 5044, 1889, 2249, 951, 2727, 341, 5758, 1740, 2562, 6058, 6165, 6499, 4386, 4394, 1678, 4519, 1447, 3376, 5272, 1408, 4281, 782, 6742, 3386, 5950, 858, 5232, 1555, 1102, 2358, 2225, 3299, 3004, 1473, 1049, 4486, 2395, 7217, 5452, 4899, 548, 5675, 2627, 3038, 5543, 5727, 4645, 5878, 1422, 6982, 2708, 3684, 5016, 5740, 711, 2741, 3221, 2583, 6212, 356, 4168, 756, 1449, 6718, 4806, 2405, 512, 1614, 2693, 3536, 596, 5313, 2173, 1025, 1168, 2126, 5033, 4578, 2352, 1518, 173, 3184, 2325, 3680, 2920, 3715, 4373, 717, 4133, 5726, 3406, 2447, 1561, 776, 2163, 3763, 796, 6470, 6969, 230, 2420, 933, 3694, 809, 5632, 7093, 6626, 4033, 1300, 4393, 3814, 4556, 1313, 4842, 6746, 7327, 6707, 5218, 6357, 6884, 5318, 985, 1843, 6959, 4870, 4863, 7347, 2043, 6902, 4385, 2396, 5611, 3660, 6315, 527, 2641, 6265, 7295, 3176, 640, 7247, 3107, 1501, 1973, 6177, 1525, 6373, 3895, 164, 3489, 4503, 2903, 544, 3064, 491, 6417, 3667, 4334, 113, 7264, 1929, 6382, 4013, 1727, 3553, 2707, 2983, 5936, 900, 4328, 1970, 1602, 3946, 4923, 4201, 4718, 3102, 5556, 6704, 1807, 1942, 1897, 560, 3742, 3899, 2308, 1631, 5327, 7296, 229, 7240, 1174, 1067, 839, 4814, 267, 4414, 1840, 2047, 5783, 2679, 6695, 5638, 2128, 6670, 1789, 4135, 5748, 7052, 2729, 5273, 540, 124, 6987, 6283, 3743, 396, 3244, 3045, 6850, 4062, 3637, 5174, 1134, 5012, 4108, 3400, 4184, 4602, 7230, 4958, 3734, 5360, 3823, 7068, 1399, 5245, 4048, 44, 3001, 2654, 5333, 3966, 435, 3954, 590, 515, 1936, 3511, 6404, 422, 4772, 6629, 5270, 3860, 5446, 311, 4076, 2347, 1490, 1230, 3909, 7168, 3190, 7260, 3081, 2670, 6358, 2510, 4042, 4305, 6678, 1860, 1043, 4209, 4010, 6680, 4762, 3755, 3949, 4904, 4631, 1777, 6766, 4702, 5469, 7160, 2828, 7125, 1539, 5686, 615, 5067, 4425, 6782, 4198, 1717, 7358, 3554, 588, 3139, 1735, 4046, 7069, 3352, 4499, 171, 2244, 1451, 3933, 3060, 4673, 6620, 2833, 252, 6472, 4694, 1190, 3864, 2515, 198, 1356, 5898, 5654, 3641, 3339, 3766, 2038, 4907, 6261, 2522, 2665, 82, 554, 4159, 6591, 5510, 150, 5642, 6156, 5231, 917, 4933, 6681, 4273, 2237, 2624, 5130, 2701, 3574, 1697, 5636, 7300, 3679, 6981, 5238, 2369, 3717, 4890, 2330, 5853, 6546, 5810, 4785, 6178, 4268, 5051, 6862, 6477, 4205, 6427, 4851, 6526, 3728, 2403, 6026, 5699, 2018, 4301, 1923, 2826, 1895, 6786, 238, 6603, 3646, 6580, 911, 6067, 7238, 2013, 1219, 266, 6469, 5860, 5156, 6220, 6703, 2029, 5744, 2720, 6541, 4828, 6708, 3413, 2270, 1991, 5415, 7070, 1912, 3227, 5121, 5292, 7385, 6331, 2386, 1745, 116, 6103, 9, 5876, 4587, 5264, 1573, 4748, 3131, 2356, 613, 3758, 4479, 4549, 2838, 3865, 1749, 5782, 5523, 6256, 4664, 3822, 3813, 102, 3561, 6963, 833, 1544, 1557, 692, 5914, 1172, 61, 709, 3941, 274, 7114, 3447, 4409, 136, 6399, 4003, 1008, 4229, 1119, 3373, 2923, 3125, 3020, 744, 6692, 397, 2528, 3334, 478, 4018, 4504, 6111, 5349, 284, 824, 6327, 5622, 1700, 2100, 2829, 5154, 6216, 5474, 4865, 3313, 3668, 6456, 5990, 4758, 5773, 4884, 4251, 5351, 6972, 1508, 3759, 2484, 3149, 282, 2091, 2014, 4525, 1651, 5267, 376, 3318, 7141, 3119, 2120]
Shuffle4 = [4609, 7323, 6359, 4334, 399, 2677, 427, 4659, 2628, 3815, 2527, 6504, 7187, 4801, 2716, 5418, 2745, 2526, 3361, 6829, 1187, 5469, 1968, 3614, 195, 6340, 7003, 1227, 5621, 7141, 1791, 2782, 5911, 3471, 718, 3744, 4748, 2652, 5902, 2166, 1273, 5784, 7386, 3296, 6096, 2112, 1687, 6960, 4251, 5759, 2185, 4727, 5745, 4314, 5099, 4382, 3215, 6694, 4702, 724, 657, 4875, 2608, 6724, 3885, 5411, 4035, 5666, 1837, 5933, 4123, 1156, 3691, 566, 1148, 4396, 4631, 2773, 803, 1583, 5389, 5004, 4745, 932, 2847, 4574, 3763, 1824, 4765, 151, 4899, 2248, 6563, 2089, 6502, 1472, 1872, 7106, 2851, 4466, 2812, 1788, 5627, 503, 2615, 5679, 3968, 3160, 2866, 5913, 3058, 741, 2611, 3012, 7120, 5687, 4961, 4051, 6614, 5345, 3802, 4458, 1973, 2075, 6567, 4062, 3272, 5365, 3645, 3243, 3334, 6127, 5649, 7404, 6413, 5563, 326, 7046, 5534, 3795, 6211, 3344, 2163, 6363, 2740, 4431, 6781, 5054, 1464, 7232, 400, 5166, 6487, 2065, 5804, 4038, 6106, 1507, 6629, 3097, 1076, 6040, 5887, 4022, 4701, 4302, 2419, 6397, 4436, 2927, 1394, 605, 7129, 4118, 5126, 5877, 2497, 5796, 3120, 2746, 2823, 3322, 388, 409, 5335, 4552, 3663, 4294, 4629, 4161, 7359, 2321, 6331, 4782, 5021, 4393, 3969, 1278, 2613, 3973, 13, 3482, 1097, 5248, 6169, 2371, 4281, 737, 1599, 3617, 1053, 5629, 97, 34, 563, 2540, 5291, 3860, 3459, 4720, 6739, 230, 7013, 6792, 5719, 5573, 1570, 1532, 2938, 6636, 2515, 78, 4592, 1225, 2183, 4799, 1477, 306, 5003, 1442, 3184, 1333, 1636, 4000, 942, 6842, 4843, 2535, 1972, 4195, 1212, 5783, 5702, 6725, 4012, 2967, 760, 2092, 6591, 4255, 344, 2213, 1523, 4265, 4128, 2542, 3870, 5281, 2730, 4940, 1050, 6618, 5037, 836, 198, 7340, 6990, 3970, 6435, 2783, 1264, 1895, 3360, 7398, 4563, 5776, 4034, 5930, 1229, 6796, 5819, 4513, 5562, 2510, 4333, 5948, 582, 446, 6408, 1564, 1586, 6969, 1608, 4640, 6181, 3534, 2241, 3884, 7081, 3646, 3813, 5961, 338, 7382, 460, 1207, 2695, 4890, 6876, 1736, 2538, 345, 7034, 1421, 4994, 2029, 4829, 3131, 2769, 3435, 2653, 1351, 3139, 50, 204, 3385, 5142, 6719, 5140, 1071, 401, 1209, 913, 4968, 3851, 2103, 966, 6338, 5925, 7388, 130, 2864, 6953, 921, 6996, 639, 2959, 28, 6143, 6893, 6495, 3829, 2691, 5114, 5256, 4945, 3327, 5774, 1963, 1755, 2387, 2123, 1940, 18, 5448, 2152, 6975, 6452, 576, 4557, 1649, 5968, 7022, 5586, 3034, 6146, 5996, 1450, 4409, 6380, 4986, 894, 5831, 6068, 1010, 3544, 5181, 3071, 6251, 303, 6032, 1733, 6505, 2359, 6288, 4390, 6912, 959, 6616, 6964, 2283, 7400, 6693, 3577, 1082, 4554, 1326, 2363, 457, 1109, 685, 6797, 874, 617, 931, 6674, 6298, 1380, 3365, 2140, 1514, 4119, 2485, 2327, 4815, 4955, 855, 5722, 4593, 3779, 1676, 4186, 4301, 6696, 7102, 4115, 4083, 1238, 4426, 1308, 645, 7132, 3354, 6620, 6584, 1297, 2378, 5505, 1309, 732, 1111, 1373, 4218, 2037, 321, 5633, 1910, 3598, 4785, 4644, 729, 1902, 163, 1300, 4045, 2889, 5306, 5856, 5736, 37, 242, 7192, 2646, 1822, 3750, 7239, 4537, 6224, 7101, 1627, 5542, 7204, 5538, 5283, 390, 3098, 7274, 3798, 3391, 6633, 2750, 6711, 5117, 257, 1483, 5497, 4684, 6552, 1810, 7202, 4261, 2452, 4312, 209, 2467, 6976, 475, 7372, 1139, 5941, 3801, 4578, 302, 3975, 3747, 3569, 608, 2862, 2553, 5537, 5704, 1577, 1897, 79, 1942, 1112, 5214, 4854, 3505, 7159, 3276, 5929, 2724, 1376, 561, 1771, 5386, 2729, 2398, 3147, 612, 3422, 2270, 5484, 1067, 596, 7, 6028, 708, 2832, 4187, 5748, 2294, 1806, 956, 996, 3401, 4107, 2654, 6540, 4307, 4708, 7212, 6252, 3055, 5265, 7055, 7210, 7397, 781, 6230, 231, 3480, 6576, 1395, 3419, 2678, 6539, 6635, 1662, 4822, 4437, 3893, 484, 6043, 3141, 1606, 2203, 6077, 7219, 1288, 171, 7366, 4484, 5267, 7247, 872, 5405, 5995, 669, 3784, 1448, 406, 4297, 6806, 2200, 795, 190, 3629, 2508, 1637, 7020, 3825, 3410, 3835, 2177, 3299, 5168, 841, 6508, 5031, 2098, 5533, 1857, 5838, 3488, 2082, 5422, 3552, 2212, 2444, 3539, 2543, 5403, 1392, 2192, 534, 186, 4783, 48, 5789, 335, 6784, 2948, 5141, 7313, 6994, 2848, 4126, 3046, 6023, 798, 6079, 6699, 3698, 4011, 4482, 6955, 1410, 5187, 5866, 2813, 4247, 1248, 3660, 134, 2990, 5601, 1034, 4242, 2374, 2786, 6256, 2704, 5707, 3765, 4385, 3444, 3857, 6554, 1099, 6827, 317, 3018, 6645, 3442, 3313, 4526, 3273, 6510, 1349, 6888, 5544, 6386, 5227, 1282, 4270, 3464, 1566, 1237, 2365, 5963, 5956, 1381, 3915, 3083, 1122, 1213, 2310, 3821, 7189, 7259, 3547, 5654, 6482, 7229, 3266, 3030, 1689, 6892, 5202, 6257, 3670, 4792, 1903, 2781, 3912, 7017, 3628, 4868, 3376, 7309, 2802, 6161, 6024, 1234, 1340, 2372, 1075, 4071, 5262, 3878, 1271, 1668, 389, 2179, 6001, 3876, 2757, 1107, 2466, 6300, 4217, 31, 1716, 6080, 3164, 2518, 5402, 6188, 3720, 7240, 6378, 4326, 7355, 5289, 1573, 776, 2385, 367, 2520, 7178, 3817, 2521, 1795, 1429, 7000, 2446, 2357, 4099, 1247, 1096, 5067, 6603, 4366, 5441, 7073, 5229, 3727, 5007, 674, 2648, 5472, 5766, 6928, 337, 7138, 361, 2019, 3199, 5266, 2635, 2944, 7100, 7208, 5344, 1556, 3290, 5909, 4039, 5136, 2388, 4304, 6625, 929, 1752, 1784, 1947, 6774, 734, 191, 5555, 5665, 3479, 6617, 6214, 2002, 1459, 5896, 6820, 3023, 6547, 7364, 4534, 679, 4433, 5228, 2346, 3358, 4455, 1639, 656, 1252, 58, 3166, 4637, 3862, 5553, 334, 6136, 4909, 1854, 1969, 4349, 6530, 2855, 4744, 2441, 3377, 6250, 2536, 7115, 1079, 6021, 1030, 3268, 821, 1597, 7233, 2501, 4298, 4819, 7306, 4122, 5976, 2024, 5568, 6025, 7157, 2009, 6303, 5549, 3732, 465, 448, 6904, 4278, 565, 359, 4308, 2715, 4376, 4661, 1177, 2222, 354, 7257, 539, 3262, 2733, 1481, 4399, 6290, 830, 2431, 6698, 6551, 3595, 5508, 2256, 5333, 2649, 5164, 5754, 1084, 3908, 793, 4700, 234, 4158, 3254, 5254, 4077, 6016, 3352, 4569, 2531, 5161, 5939, 5047, 3472, 5864, 3169, 4299, 7044, 1116, 4575, 813, 1738, 2094, 5660, 420, 375, 5416, 2131, 5927, 5932, 4680, 6811, 1028, 4407, 3794, 216, 2734, 5703, 6559, 6172, 664, 945, 1589, 5371, 6451, 5685, 7198, 6267, 7176, 6313, 6493, 3298, 1466, 3935, 432, 6156, 4350, 1378, 6226, 7019, 491, 6390, 804, 2743, 5489, 5457, 3393, 4509, 26, 6414, 3757, 4666, 3269, 4845, 4053, 5967, 5039, 1706, 4777, 5447, 2586, 5153, 7267, 3793, 6291, 2059, 1709, 3514, 5215, 6381, 351, 2918, 3113, 3933, 4667, 4179, 3357, 5729, 2787, 2071, 6669, 693, 1845, 1993, 5500, 3059, 3115, 6197, 7396, 1792, 6921, 6481, 1016, 5330, 4611, 6177, 1846, 5445, 5326, 6309, 2896, 350, 4581, 2229, 2581, 4149, 602, 6428, 6868, 415, 7001, 5176, 3240, 1741, 6744, 2343, 700, 1623, 3311, 4234, 6328, 35, 5125, 1299, 6405, 7070, 3134, 72, 5511, 1727, 3931, 4082, 1653, 392, 330, 496, 2216, 1194, 4142, 4335, 5569, 5108, 2974, 4544, 2056, 6209, 4604, 6654, 5559, 4250, 719, 876, 5564, 549, 5332, 43, 2709, 858, 6604, 5873, 6006, 6322, 4280, 3799, 2876, 6672, 3654, 6880, 1770, 5119, 3587, 2841, 2301, 3998, 5056, 5006, 431, 5965, 5635, 3465, 412, 6294, 5363, 4095, 6478, 4995, 4394, 1327, 1557, 2620, 6703, 3861, 2225, 5213, 4789, 6720, 6145, 4511, 3701, 4774, 3087, 481, 1179, 5260, 3668, 4997, 404, 1700, 4001, 5420, 1348, 4418, 7160, 4966, 5585, 531, 5509, 2057, 20, 5263, 6583, 5282, 3053, 4565, 6915, 6843, 4494, 94, 7037, 5498, 4422, 4883, 7369, 5387, 3981, 111, 4154, 467, 2560, 1306, 2042, 7317, 1591, 4707, 4471, 6652, 7076, 7182, 7164, 5727, 7014, 5988, 6459, 3222, 493, 2368, 7242, 4381, 2758, 6716, 301, 4318, 6983, 1647, 2840, 483, 3549, 4363, 3541, 6870, 6307, 1580, 2360, 6304, 2426, 5514, 2016, 2330, 2665, 6883, 4306, 2570, 3833, 5520, 5816, 4269, 5584, 2277, 775, 5590, 1929, 2973, 2450, 7168, 7045, 6458, 5898, 5917, 2933, 1821, 3907, 6243, 7220, 1274, 547, 7093, 5393, 1401, 4373, 75, 6748, 928, 4020, 3640, 5991, 6930, 5072, 7134, 3609, 4603, 592, 6050, 2679, 2293, 84, 4330, 1866, 4014, 1073, 4898, 1619, 6687, 5093, 865, 4021, 910, 2379, 6105, 1923, 4170, 247, 5637, 2785, 408, 1939, 2148, 4412, 7299, 4689, 4404, 3260, 3085, 4488, 2151, 1904, 3627, 6794, 6371, 2058, 3894, 2945, 4800, 61, 5940, 463, 5230, 4688, 1679, 256, 2607, 3586, 1407, 1595, 1219, 5712, 6483, 365, 5220, 5764, 4013, 5546, 6557, 552, 4086, 570, 6659, 5752, 1317, 6149, 1014, 1290, 7280, 5185, 7389, 3452, 1612, 10, 307, 5870, 2572, 1242, 5836, 6044, 5833, 2442, 7111, 6366, 1000, 838, 1078, 1886, 4295, 6424, 2519, 1258, 1930, 6349, 3125, 1763, 2872, 6249, 6090, 117, 6647, 6460, 6775, 1559, 2124, 6052, 2556, 1228, 169, 2132, 4309, 4859, 3624, 2381, 5002, 6755, 6276, 6427, 5602, 2258, 4618, 6917, 5583, 6430, 5897, 2609, 3616, 1739, 816, 1129, 2817, 6664, 2514, 525, 6205, 4162, 2604, 7350, 2475, 7228, 2920, 3904, 3724, 2516, 262, 5096, 6914, 2953, 6449, 1766, 2235, 2991, 3231, 4776, 1989, 6734, 2975, 208, 59, 4675, 4809, 1476, 7356, 1411, 1230, 5381, 7119, 422, 6206, 1790, 6578, 2853, 3523, 238, 5426, 1310, 3685, 3735, 818, 464, 6476, 2940, 5478, 892, 7223, 30, 2097, 2481, 7260, 1063, 4812, 6088, 3812, 2631, 1173, 2622, 887, 6743, 4893, 953, 1047, 6196, 1491, 3080, 6383, 716, 3295, 5057, 2593, 2986, 7170, 3681, 1322, 6219, 3162, 1342, 1522, 4734, 1334, 5, 6997, 824, 4678, 6014, 5129, 1651, 3761, 7231, 2133, 1836, 157, 4057, 1412, 7173, 3982, 4442, 1144, 6756, 6972, 6231, 5410, 7394, 3451, 1953, 1932, 2013, 2721, 2931, 1710, 5429, 3355, 644, 4515, 6564, 5055, 7385, 4497, 6840, 588, 1474, 6418, 2636, 4889, 5557, 2161, 604, 1092, 7234, 1753, 790, 2291, 1855, 3457, 6800, 5453, 1115, 6468, 4470, 7227, 6676, 2039, 2445, 2110, 3242, 3606, 5697, 364, 7128, 992, 1723, 601, 6956, 3244, 1774, 7113, 2253, 2251, 1958, 1127, 407, 5962, 2629, 121, 2208, 232, 2978, 1730, 3101, 1106, 5527, 1501, 4991, 2427, 2906, 4219, 2818, 747, 3297, 3284, 7175, 4927, 3403, 3006, 6075, 4059, 4674, 6343, 5805, 2873, 3558, 9, 434, 2578, 1851, 5466, 1568, 2529, 4577, 3823, 7016, 6333, 2333, 1525, 6853, 6396, 7292, 4372, 6613, 6780, 7361, 779, 2155, 2068, 6519, 5945, 4207, 6245, 1152, 4033, 6479, 7086, 3069, 5955, 4383, 5539, 504, 3746, 2605, 6270, 342, 2219, 4952, 939, 1341, 127, 5234, 5241, 4660, 3230, 6384, 5916, 1414, 6689, 788, 1233, 1406, 5582, 5744, 1232, 2435, 1614, 1878, 5025, 7336, 5392, 6860, 4093, 122, 5526, 6650, 6729, 5799, 5329, 3439, 1748, 4358, 5307, 2484, 2675, 5091, 2230, 5803, 6838, 654, 4914, 6407, 4884, 4630, 3615, 3939, 1596, 4755, 2141, 1408, 6735, 2286, 3118, 1941, 5672, 4551, 1336, 7346, 1850, 2900, 6908, 5847, 2625, 2597, 6965, 1484, 4685, 2044, 139, 4725, 2456, 640, 2109, 3864, 2462, 889, 947, 5853, 6807, 1957, 1950, 160, 7027, 5828, 1286, 5032, 7401, 3171, 2693, 2651, 4897, 3536, 1203, 4606, 3205, 5481, 6713, 5688, 1201, 2766, 6142, 3106, 706, 3584, 2313, 551, 6441, 1397, 1750, 1359, 2673, 1995, 3094, 5033, 4976, 4181, 6027, 3528, 237, 4716, 1011, 4764, 2028, 1999, 3994, 98, 3062, 6835, 6982, 4024, 7108, 4070, 713, 3731, 5317, 2946, 165, 4934, 972, 521, 4501, 7261, 4623, 2666, 4803, 6550, 1546, 908, 3189, 5521, 7207, 2273, 7290, 655, 1032, 3575, 571, 6822, 977, 5276, 2157, 5179, 1561, 2645, 3170, 2956, 3366, 5189, 311, 3716, 6817, 598, 2926, 7311, 4040, 875, 7319, 3380, 3512, 5924, 4873, 6517, 7371, 174, 7378, 2671, 1876, 2227, 6675, 4237, 3237, 5613, 4240, 5741, 3156, 985, 6354, 980, 2574, 16, 2022, 3644, 1064, 1675, 2888, 2354, 3086, 55, 6018, 1272, 6350, 3089, 6627, 2640, 3348, 5532, 5451, 5530, 220, 2074, 6641, 2443, 5194, 3475, 1916, 3714, 5087, 3084, 1678, 3921, 3275, 688, 3206, 476, 5845, 950, 4100, 6611, 3966, 2759, 49, 3132, 6534, 4842, 7390, 7087, 6890, 5757, 7193, 3072, 4427, 3434, 3588, 2894, 1891, 7146, 5728, 7297, 4440, 5169, 4771, 6470, 2181, 5320, 2717, 6114, 2341, 1885, 3887, 4999, 6208, 3704, 1362, 148, 5551, 2492, 611, 6596, 6979, 2555, 6758, 2768, 4786, 1809, 4171, 5273, 6204, 3790, 5936, 4109, 3967, 249, 2748, 2361, 1782, 5116, 3641, 5571, 7041, 7271, 4441, 2668, 5139, 3157, 6885, 5372, 5203, 1070, 3369, 949, 2275, 964, 4620, 64, 7047, 3337, 5846, 4860, 3172, 5818, 6663, 3203, 560, 3168, 4293, 6003, 2502, 5382, 572, 984, 4260, 4639, 1473, 5755, 744, 3186, 4928, 6757, 4585, 106, 1140, 6282, 2487, 5290, 1652, 3149, 653, 791, 4016, 2193, 5821, 276, 6560, 5874, 4798, 4998, 1358, 1178, 3525, 7133, 7235, 6573, 1020, 1456, 6488, 29, 1542, 470, 1015, 2468, 6984, 2674, 1955, 5053, 1638, 1811, 6045, 4483, 5867, 698, 3176, 3163, 1578, 53, 3906, 4491, 4984, 1184, 3148, 3722, 1325, 5715, 3283, 3532, 3695, 2034, 6856, 3246, 979, 559, 3233, 3005, 3892, 1965, 1143, 4805, 4423, 5353, 5157, 329, 383, 7033, 3010, 217, 4414, 885, 2394, 2038, 628, 428, 2316, 742, 5669, 720, 726, 5989, 3972, 5645, 4504, 2590, 120, 4094, 4425, 7040, 2206, 4036, 6732, 2053, 7169, 11, 915, 2138, 6837, 5848, 2207, 6334, 3036, 3415, 1553, 6170, 4416, 5204, 1368, 6535, 223, 1852, 5338, 6461, 7068, 7109, 4043, 115, 1451, 5676, 7282, 5316, 2684, 542, 6085, 3382, 7310, 5643, 717, 3760, 3255, 6111, 5224, 4550, 5391, 6295, 6375, 863, 339, 6538, 1400, 4078, 5015, 6875, 2731, 4046, 4351, 3916, 5986, 4108, 355, 3025, 1223, 1269, 5548, 5407, 5698, 6839, 2086, 6751, 4709, 4028, 290, 2494, 911, 1660, 2060, 3599, 1183, 2856, 4201, 2955, 5993, 3572, 2910, 2844, 6035, 7050, 2113, 5863, 3252, 1193, 2496, 2197, 2114, 5098, 6553, 2669, 5865, 381, 7151, 3239, 5437, 5092, 5662, 3808, 1882, 2929, 7123, 5480, 4406, 3038, 967, 87, 2380, 6939, 5134, 6623, 2479, 5677, 4882, 6986, 1914, 2201, 7191, 132, 7221, 6217, 6060, 5103, 1554, 869, 3949, 7305, 536, 6894, 615, 7090, 5101, 4130, 941, 3096, 1833, 6881, 5547, 6558, 2296, 3020, 6923, 3281, 1235, 3742, 370, 4694, 768, 2793, 5813, 6155, 1509, 6235, 2064, 3061, 1779, 1549, 4371, 6404, 810, 1794, 2544, 6122, 6522, 2569, 5938, 4231, 2736, 2914, 5797, 4766, 5730, 7264, 3516, 4303, 2088, 4496, 4085, 2182, 3368, 997, 5367, 6601, 90, 5049, 1519, 7211, 5681, 6754, 4835, 7391, 5657, 21, 3288, 5630, 4804, 3636, 5636, 207, 5247, 485, 891, 194, 2336, 4726, 3574, 4856, 4536, 526, 1757, 2345, 4391, 6501, 5084, 3836, 82, 6736, 7273, 1241, 4264, 4032, 6950, 499, 3004, 5292, 4225, 4857, 660, 438, 199, 2779, 4243, 7112, 3285, 2231, 2474, 3103, 5693, 4953, 2995, 3306, 5823, 4498, 3999, 4698, 3774, 4155, 3426, 6938, 5121, 5413, 1344, 1100, 6012, 3341, 6051, 7326, 5716, 2085, 5952, 6269, 5010, 2102, 3713, 6139, 1692, 1323, 4075, 1931, 7024, 3639, 5312, 6993, 6804, 843, 3656, 1926, 681, 4841, 3191, 2917, 5696, 7127, 6940, 2512, 3650, 362, 5205, 5857, 1911, 4949, 6434, 1812, 1345, 1526, 1604, 4460, 1104, 5288, 176, 7367, 2010, 3844, 6132, 5709, 4220, 4395, 5156, 4989, 7251, 5190, 3003, 202, 6589, 1330, 2557, 5130, 6812, 3753, 5923, 3842, 4977, 4284, 3453, 3481, 119, 1260, 4252, 3985, 1444, 1823, 4664, 5279, 3180, 5884, 3458, 4205, 7249, 631, 6803, 5957, 3564, 3687, 1463, 2278, 4050, 5258, 937, 6154, 5556, 6342, 1419, 4773, 6308, 864, 298, 3484, 7238, 1279, 486, 5001, 1912, 5944, 6844, 4327, 1974, 3903, 3195, 5488, 4693, 4840, 3274, 982, 6325, 7152, 3546, 5112, 1263, 7153, 1480, 6899, 4872, 7057, 3406, 1357, 5458, 4907, 3129, 6926, 1879, 6207, 6274, 4429, 999, 4923, 1398, 510, 5493, 5321, 3495, 3592, 368, 2070, 4587, 5667, 2885, 4213, 4296, 3697, 5540, 2046, 4925, 5578, 600, 777, 6285, 396, 283, 6490, 4848, 166, 1155, 6456, 785, 4867, 4325, 4686, 3944, 520, 5732, 6980, 3832, 1386, 7348, 4182, 3044, 2305, 2935, 2650, 3107, 1762, 6579, 203, 5802, 3959, 4221, 1315, 6039, 3699, 6091, 6571, 696, 2749, 4763, 5787, 5341, 6903, 7085, 1575, 3509, 3347, 7298, 4617, 3423, 933, 2899, 3331, 2007, 4757, 281, 4461, 1908, 3319, 553, 764, 2903, 3476, 4826, 3026, 5648, 4355, 5997, 3846, 2386, 6355, 3304, 7195, 2561, 4830, 1920, 3212, 5625, 1350, 6678, 2912, 2104, 638, 5012, 3850, 898, 5706, 1004, 5311, 5045, 4462, 899, 7288, 3394, 3424, 3088, 6887, 2477, 4553, 848, 5343, 6802, 1659, 2892, 1186, 6417, 4027, 6813, 4735, 1198, 2774, 2962, 1555, 1937, 295, 5593, 3386, 304, 7117, 6336, 2199, 341, 7243, 2852, 4714, 6615, 1798, 4567, 5812, 2036, 7253, 6293, 5567, 3417, 4209, 4881, 2308, 4599, 3009, 6581, 4900, 1694, 6727, 3305, 1259, 1141, 5133, 3800, 1324, 138, 3594, 3729, 1364, 5209, 5222, 5908, 4980, 4188, 740, 4463, 5912, 2205, 5959, 2587, 4956, 1567, 6750, 3986, 1922, 4388, 6472, 2849, 2934, 1162, 214, 1605, 1883, 4816, 5377, 6013, 5242, 6662, 4787, 4902, 384, 7349, 919, 1208, 3112, 2190, 7301, 5631, 4117, 6094, 3710, 5801, 6323, 218, 2829, 4880, 272, 4054, 6769, 2236, 6515, 440, 995, 2202, 4029, 851, 4723, 3408, 5717, 2996, 6369, 315, 3845, 1114, 3318, 3551, 5278, 4413, 920, 6586, 4208, 4365, 6680, 5596, 513, 4742, 3277, 5268, 6019, 6465, 6228, 2242, 6113, 6284, 7110, 5855, 6937, 4924, 1994, 3899, 5624, 7217, 6438, 723, 7060, 6225, 6821, 3416, 4148, 4601, 5951, 6666, 312, 6439, 689, 5915, 1356, 530, 152, 3682, 6889, 4547, 2588, 3041, 3217, 1468, 2049, 1415, 3409, 1805, 990, 6925, 4514, 5609, 6959, 4056, 85, 2594, 5597, 4290, 5499, 4813, 2404, 2547, 1133, 2559, 5762, 5358, 4529, 7287, 7171, 695, 3402, 2875, 294, 4922, 2312, 7275, 5199, 305, 6463, 373, 2599, 479, 5990, 1506, 358, 711, 994, 3770, 4485, 3146, 248, 5886, 4447, 7225, 1584, 5587, 1175, 5861, 4176, 6316, 5216, 1387, 3350, 1768, 3943, 5197, 5269, 4891, 6967, 23, 2870, 4343, 6078, 5068, 754, 6332, 4074, 4080, 5286, 6910, 5076, 4266, 4018, 6382, 3866, 6362, 5504, 4865, 6445, 1890, 6991, 5743, 854, 5346, 5747, 4971, 5826, 2176, 3329, 411, 3902, 5675, 4004, 6162, 5024, 2396, 7066, 3450, 3362, 4681, 3259, 5646, 5463, 2513, 634, 751, 137, 3226, 2347, 973, 4432, 6852, 4663, 1083, 3709, 7393, 4978, 2142, 2254, 6401, 3108, 2861, 4638, 1384, 4677, 3992, 7351, 1715, 1785, 4338, 6346, 4548, 405, 6561, 6648, 2800, 7009, 1277, 5839, 105, 694, 1849, 2710, 1520, 5027, 5196, 1551, 1780, 7061, 6594, 658, 2552, 5030, 1138, 1495, 2299, 540, 1773, 4827, 3568, 287, 2172, 6444, 856, 630, 2534, 162, 5421, 7314, 4721, 4340, 3583, 1949, 6029, 5085, 5844, 1701, 4157, 3923, 4770, 6311, 6555, 1565, 4465, 5622, 2712, 1746, 3717, 4305, 4092, 62, 1842, 4546, 5972, 4249, 780, 4761, 3027, 6344, 2433, 1747, 3675, 6147, 3367, 4825, 1503, 3267, 5595, 3013, 2947, 2106, 1037, 5239, 1081, 5041, 2737, 2087, 897, 635, 2274, 1027, 6685, 340, 3708, 4067, 1166, 4543, 5255, 4464, 3869, 2814, 2815, 4238, 6015, 2370, 3683, 862, 4682, 5753, 594, 7196, 3608, 3531, 916, 2233, 4384, 755, 2660, 881, 1181, 7357, 5152, 1943, 1877, 6448, 4348, 1869, 4025, 1054, 710, 22, 5495, 4920, 1452, 4788, 6701, 3008, 3796, 3325, 5758, 2752, 5243, 95, 6906, 4405, 7303, 7194, 1707, 4026, 3078, 6041, 2532, 241, 1952, 2062, 2790, 12, 3093, 3623, 7104, 6651, 5906, 4943, 5485, 4853, 2921, 1385, 5825, 2913, 4656, 2355, 1031, 2081, 6992, 4566, 6805, 1101, 3941, 558, 385, 2362, 1102, 187, 2589, 909, 1366, 5947, 4811, 1437, 150, 1164, 1641, 1944, 5314, 6046, 6607, 2040, 5888, 5591, 1196, 4215, 6580, 3589, 1087, 3054, 7294, 4403, 1051, 4091, 5817, 4911, 4041, 4607, 6815, 5274, 2592, 6779, 2392, 110, 1983, 5503, 759, 2760, 1997, 5659, 5106, 6178, 4476, 5769, 5018, 158, 1295, 7256, 4065, 1176, 4331, 3978, 2344, 4320, 4202, 6653, 1971, 805, 3632, 5682, 6673, 4143, 704, 108, 5670, 5374, 2395, 4633, 7250, 1718, 2376, 2765, 4212, 6244, 4583, 2187, 5946, 4696, 2658, 847, 5475, 2720, 868, 1760, 3726, 4353, 3263, 2031, 1467, 2998, 6999, 1205, 888, 584, 6512, 4007, 69, 6905, 225, 5494, 1174, 172, 101, 2612, 3811, 6768, 1276, 606, 849, 5889, 6791, 3771, 538, 4938, 4632, 4913, 3064, 4417, 2262, 5978, 3390, 7307, 5388, 441, 4168, 224, 2924, 5383, 5895, 1465, 2869, 1804, 5294, 6631, 4430, 2260, 2171, 6123, 730, 1642, 2045, 1899, 6952, 4079, 91, 1667, 5598, 1893, 7186, 3065, 3950, 4877, 1829, 2503, 1409, 1220, 4754, 6985, 6183, 7302, 4964, 154, 296, 6639, 1529, 3803, 4313, 7333, 2601, 5261, 5619, 2713, 1055, 623, 6115, 7241, 6061, 6933, 7325, 489, 4762, 6086, 8, 6896, 6568, 3145, 1113, 2338, 1712, 2919, 4729, 449, 1905, 4533, 5070, 1335, 545, 1389, 1734, 7216, 57, 802, 3326, 6492, 88, 4864, 2413, 4767, 643, 4626, 6773, 3900, 6063, 6599, 1832, 1981, 6107, 4241, 2478, 1982, 6275, 1022, 6819, 2472, 7125, 3988, 4316, 1786, 6851, 3011, 1867, 5440, 5192, 6264, 96, 882, 3279, 492, 3383, 1363, 880, 770, 1157, 3734, 240, 2473, 6987, 1048, 179, 3828, 1343, 1423, 4467, 5415, 2119, 3421, 3289, 4439, 3566, 102, 1847, 25, 2877, 3119, 4378, 6523, 352, 4539, 1906, 60, 3241, 5770, 6998, 2319, 1853, 1987, 2602, 6234, 5610, 926, 7374, 2397, 2289, 3126, 3303, 6200, 1160, 346, 2904, 5742, 2767, 4362, 5029, 4992, 4669, 3743, 4068, 2985, 792, 7324, 3690, 2963, 3445, 1834, 6833, 6823, 3859, 2776, 3996, 471, 4113, 6776, 5219, 5565, 722, 3605, 1135, 4979, 2083, 4102, 1169, 5131, 327, 2952, 6120, 1541, 363, 3838, 3143, 6391, 1305, 6202, 5574, 4206, 5850, 922, 2859, 2984, 4887, 3232, 2830, 2483, 4259, 1761, 2810, 1443, 6128, 2093, 5860, 2794, 5882, 5150, 5943, 6981, 2909, 1370, 6134, 6575, 2630, 5097, 4870, 7183, 767, 4044, 6919, 1825, 6787, 6884, 292, 4885, 7031, 6067, 1059, 2965, 6232, 1640, 624, 4137, 2432, 2524, 3843, 715, 3336, 6329, 5942, 2306, 6436, 439, 1543, 6157, 4930, 2245, 253, 1432, 5120, 4292, 4064, 5512, 1572, 4820, 1873, 1927, 4950, 6834, 3396, 6688, 2434, 4341, 2091, 424, 1006, 6587, 3371, 5280, 5395, 3619, 1797, 2890, 3363, 5011, 336, 1486, 1732, 2682, 1915, 1815, 4410, 890, 4286, 3489, 1767, 6241, 3886, 2498, 1531, 1843, 3349, 5302, 3664, 2722, 3951, 4073, 284, 5446, 3315, 3144, 7124, 2005, 7140, 1620, 4374, 1457, 271, 907, 2522, 4717, 5308, 2982, 7012, 6055, 3647, 610, 5710, 2244, 1992, 6117, 67, 4347, 177, 3294, 3287, 6692, 2348, 1548, 5983, 1511, 4718, 7032, 136, 6475, 1280, 840, 6808, 5368, 4919, 756, 5397, 4759, 871, 6240, 5238, 1479, 554, 6239, 4282, 4097, 5375, 86, 5620, 6511, 5151, 221, 3700, 4134, 1289, 5824, 6700, 2168, 6988, 6598, 3282, 3773, 1740, 1558, 1538, 4780, 6911, 2882, 4951, 4614, 6066, 6403, 6995, 2052, 4781, 632, 1590, 5723, 5366, 421, 3082, 2780, 4586, 5251, 7381, 901, 3877, 1759, 5881, 4444, 6897, 5115, 6597, 5528, 5060, 2936, 1249, 3839, 5135, 1221, 6619, 2459, 3890, 3983, 1625, 133, 7199, 2063, 573, 4990, 3310, 2409, 5174, 4570, 125, 641, 1489, 1919, 1588, 6918, 957, 4888, 4114, 662, 3015, 3895, 3590, 6989, 1418, 5351, 4683, 1695, 5617, 3278, 3786, 6070, 2784, 5148, 5442, 3258, 3684, 867, 6566, 4379, 6499, 6125, 261, 3418, 1758, 3498, 5815, 4452, 6882, 4354, 1690, 4456, 6951, 808, 4713, 1552, 1967, 6008, 5671, 2250, 4652, 5376, 4946, 7074, 2276, 1245, 772, 6746, 3335, 6708, 3573, 4641, 7135, 4531, 4600, 7363, 1756, 4147, 1936, 4487, 6455, 6357, 5903, 1469, 5110, 497, 5064, 5201, 2880, 4262, 1888, 5462, 6138, 1123, 3494, 6848, 7262, 3187, 3543, 1918, 5100, 4975, 6164, 5603, 731, 3721, 4844, 5786, 5694, 4239, 319, 517, 6259, 2701, 3111, 2941, 6356, 3570, 6927, 4060, 1875, 3225, 6242, 324, 550, 3891, 6305, 4449, 5356, 3928, 3631, 5464, 2077, 5473, 2999, 5761, 2455, 5953, 2659, 625, 2156, 4160, 4222, 5642, 436, 506, 7284, 5354, 835, 3777, 2664, 3625, 201, 3946, 1035, 1544, 4061, 3179, 1019, 6491, 4710, 1800, 3856, 4752, 1405, 4510, 555, 4895, 4628, 4572, 4165, 92, 2905, 4204, 6943, 3427, 6733, 1697, 637, 369, 6033, 6166, 7222, 3398, 2465, 946, 2662, 7006, 580, 6324, 7392, 4047, 3527, 2637, 4769, 2834, 1132, 36, 712, 269, 800, 6621, 3035, 3524, 5177, 3404, 2364, 4279, 2703, 970, 5880, 1610, 6878, 263, 2545, 149, 5869, 672, 7069, 3740, 5347, 6135, 146, 2833, 1545, 1722, 3330, 5782, 2001, 3778, 2136, 3848, 3905, 6707, 1745, 5061, 4214, 4658, 4132, 1068, 4480, 6093, 2198, 6485, 4850, 5749, 5899, 4589, 7078, 4796, 5240, 757, 3293, 725, 2174, 629, 5094, 1091, 2188, 4747, 6514, 6809, 2149, 4657, 6433, 2302, 1547, 765, 2099, 3707, 3077, 3918, 4211, 4973, 4736, 5233, 6153, 3965, 3804, 3229, 1537, 331, 567, 5318, 6347, 2265, 678, 5652, 4288, 1239, 564, 4184, 4915, 3677, 1365, 6053, 3443, 2100, 3873, 2257, 7188, 4133, 2457, 4150, 7258, 6738, 6406, 2342, 2495, 5950, 7226, 1870, 6180, 2922, 1688, 6062, 1743, 735, 3462, 2373, 3448, 4905, 141, 736, 3301, 3603, 2596, 5683, 3152, 4523, 1346, 5718, 6489, 3930, 1945, 6764, 5611, 6737, 2335, 5449, 3429, 993, 2951, 3291, 2196, 5309, 7139, 6229, 1729, 5713, 6484, 5249, 2868, 2638, 4116, 3637, 5926, 5981, 5885, 5324, 1369, 6761, 6916, 2805, 5396, 981, 5720, 5107, 3578, 3200, 3769, 4268, 2228, 4377, 7174, 6453, 3694, 1039, 2558, 2657, 5529, 2838, 3810, 1171, 4197, 6640, 2403, 1056, 6824, 6549, 6327, 6258, 6266, 1103, 4072, 4650, 2600, 6671, 3270, 6144, 5050, 6152, 2809, 794, 4193, 322, 5891, 2237, 6236, 5705, 1643, 161, 6190, 2050, 6280, 6935, 1281, 3513, 1431, 5792, 3249, 6203, 3286, 3667, 3031, 4858, 1711, 3756, 3040, 372, 6660, 1302, 4862, 817, 4315, 6277, 5928, 6022, 27, 2238, 7038, 63, 2464, 300, 2105, 2417, 289, 3688, 2546, 3662, 934, 3696, 5364, 2857, 3314, 5692, 5987, 6223, 5779, 903, 4323, 533, 6936, 523, 1527, 4332, 7126, 2128, 1629, 529, 2908, 233, 83, 5298, 896, 707, 2911, 697, 5062, 2220, 2895, 7185, 6233, 3173, 273, 3500, 6042, 56, 4753, 5852, 6529, 4271, 2537, 3772, 5063, 2747, 2288, 5349, 1374, 5285, 2836, 4311, 6081, 7321, 2692, 3463, 6058, 978, 6638, 895, 7269, 5907, 6317, 4368, 5398, 3515, 4478, 3553, 3738, 5971, 6059, 6715, 5328, 3788, 4479, 2178, 3022, 4852, 7331, 5998, 2705, 3490, 1093, 1921, 6247, 5149, 2688, 2267, 752, 4042, 5127, 2461, 1807, 6929, 5075, 2079, 4230, 3079, 2511, 3224, 4101, 2280, 6771, 1735, 2255, 2504, 5960, 6353, 5118, 4151, 930, 4582, 478, 7136, 3425, 1803, 5588, 2826, 1488, 7383, 6174, 1808, 5756, 173, 1696, 454, 4731, 6103, 316, 6877, 2117, 5058, 514, 3977, 1154, 5159, 5208, 5461, 2145, 7137, 1984, 128, 461, 3339, 39, 2491, 1217, 4508, 81, 5579, 4768, 6786, 2323, 819, 6742, 3473, 2415, 1708, 19, 1303, 6084, 3702, 193, 2867, 6710, 6087, 40, 6005, 6469, 112, 4420, 3591, 4139, 309, 2623, 1199, 6352, 2891, 267, 1693, 7072, 6766, 5297, 6668, 2126, 2968, 5293, 562, 543, 5922, 1195, 1592, 825, 3612, 5554, 3961, 6957, 2797, 6931, 5434, 4397, 2027, 2806, 4520, 3807, 147, 7091, 2687, 5379, 6891, 7077, 7010, 288, 5700, 6160, 6385, 3400, 4591, 42, 4411, 1528, 6168, 5604, 1996, 416, 7011, 1951, 3831, 3593, 3958, 192, 620, 519, 2971, 6721, 7130, 3758, 4912, 2282, 1321, 2189, 3787, 6358, 3782, 7058, 5385, 6116, 3741, 3430, 4568, 143, 6198, 2055, 4257, 3165, 2014, 3733, 3940, 3780, 7056, 1602, 1058, 4724, 2656, 107, 395, 443, 459, 265, 5594, 17, 4926, 1314, 5043, 1502, 3990, 6709, 4676, 2551, 2411, 371, 6392, 4408, 7028, 4055, 1391, 2616, 3533, 2680, 6361, 4793, 3929, 5725, 969, 7345, 3114, 7380, 6622, 2159, 2353, 6845, 6496, 5433, 3428, 1438, 6871, 2116, 6010, 1615, 2264, 4699, 4398, 5428, 4719, 1632, 1749, 6065, 3643, 6367, 1060, 1582, 7201, 2528, 3511, 4445, 5430, 2047, 1751, 349, 3091, 5013, 6609, 6457, 54, 1441, 280, 417, 5019, 2619, 6947, 6, 6630, 4647, 7248, 5439, 578, 4866, 2252, 7015, 6447, 1182, 5560, 714, 3768, 4198, 2297, 1581, 5827, 2732, 2339, 5211, 2697, 3824, 2143, 2407, 3840, 3942, 650, 2408, 7254, 3922, 902, 4579, 4662, 15, 6364, 5301, 5223, 684, 1262, 4522, 4668, 5738, 1316, 1778, 2753, 1744, 6788, 2821, 5658, 5206, 5109, 3712, 7049, 5009, 2448, 3508, 6760, 4937, 3718, 4944, 1126, 206, 2772, 6163, 6130, 2134, 5394, 3865, 1702, 3477, 5221, 924, 5808, 2989, 4031, 6954, 7089, 2035, 3596, 2030, 820, 925, 2150, 4758, 5170, 3751, 2146, 2326, 4345, 7148, 5589, 3374, 211, 402, 3897, 7064, 5086, 1165, 1125, 2993, 3321, 4375, 1787, 1117, 1435, 4807, 1656, 210, 3565, 6556, 7092, 4256, 2626, 3073, 4730, 7230, 1946, 6351, 1835, 3076, 6443, 3739, 2191, 2121, 5931, 2246, 2032, 4760, 71, 5572, 3405, 2118, 646, 3837, 4728, 5814, 4935, 1985, 4369, 423, 226, 6064, 3130, 2643, 2122, 1848, 4103, 3783, 5628, 4824, 3378, 2147, 5750, 7084, 5065, 4277, 1041, 6031, 5810, 4364, 293, 2874, 3320, 6272, 6101, 6263, 1243, 1275, 3182, 180, 2402, 5245, 3455, 5935, 2069, 5081, 569, 1661, 5934, 7180, 2932, 3911, 721, 1674, 860, 4874, 1826, 7329, 5078, 5840, 3571, 2639, 5147, 3542, 6831, 5496, 7300, 3253, 2416, 5686, 1475, 5536, 3991, 3830, 4545, 4988, 5794, 5454, 1500, 2315, 4558, 299, 3257, 5080, 4818, 277, 1634, 1896, 4832, 5404, 6056, 516, 5304, 6525, 4559, 6218, 6932, 2614, 4105, 5623, 5327, 5380, 426, 4229, 6072, 333, 1304, 1036, 3621, 583, 3345, 5180, 7143, 4076, 2436, 1172, 2309, 5858, 4361, 6649, 5198, 5218, 3666, 5994, 3213, 452, 4794, 2883, 3221, 4493, 2685, 3048, 227, 4619, 3016, 430, 3247, 7005, 3538, 2563, 2751, 535, 3194, 2447, 4459, 917, 1433, 6867, 6826, 5868, 5522, 3208, 3387, 2382, 1189, 7341, 5044, 5482, 4317, 6731, 1403, 3501, 5035, 5543, 3692, 4138, 2453, 3190, 2154, 7330, 6377, 1934, 6422, 1446, 2562, 5095, 2726, 3227, 4703, 6296, 4336, 3686, 2500, 4738, 3456, 2340, 502, 5581, 5182, 1460, 1530, 6799, 3070, 1990, 413, 7039, 6129, 1161, 5034, 4741, 1146, 807, 2454, 2449, 4472, 6778, 2325, 3680, 5576, 5605, 6759, 6765, 1907, 1669, 1319, 2949, 3210, 5811, 2234, 3381, 5226, 1180, 1253, 2375, 850, 2130, 5763, 6902, 1737, 2541, 7304, 2576, 587, 4010, 1844, 589, 955, 2627, 7142, 2025, 2667, 2863, 6978, 1562, 1560, 3127, 7318, 1496, 4837, 5900, 3926, 2777, 7224, 7107, 6262, 7206, 2211, 1880, 1820, 4402, 6426, 3601, 4084, 3204, 1487, 7053, 1478, 4400, 6297, 5781, 6730, 3192, 6192, 4017, 5167, 5178, 3333, 278, 1052, 6924, 2762, 1598, 2725, 5195, 4588, 1074, 2451, 761, 3997, 2300, 774, 2021, 47, 7296, 683, 4216, 7379, 4272, 2349, 3555, 5535, 5319, 1043, 7328, 7079, 1150, 6215, 4136, 3671, 5894, 5701, 1817, 5541, 41, 5775, 1428, 7214, 5155, 2943, 4223, 4005, 2860, 6301, 4584, 4352, 4855, 758, 6253, 6818, 1769, 852, 3346, 3466, 2314, 3198, 796, 7276, 4490, 4173, 1862, 7335, 6968, 4562, 6624, 3110, 1307, 4081, 451, 1663, 1864, 4003, 4453, 1978, 6002, 6480, 6946, 7403, 2307, 2489, 2337, 275, 2356, 1814, 2111, 2523, 7088, 3181, 5082, 823, 5834, 455, 3043, 5479, 3749, 6193, 7354, 581, 6320, 948, 5186, 6518, 3917, 2186, 2438, 5876, 1089, 4654, 4435, 66, 6783, 886, 4129, 4613, 5339, 789, 2595, 2928, 5751, 6841, 2320, 557, 4386, 1796, 2533, 3633, 6104, 3413, 7065, 1505, 3955, 45, 76, 4921, 2571, 5639, 6546, 5964, 7353, 5089, 3000, 236, 3818, 3271, 5699, 5883, 1285, 6254, 7252, 2410, 259, 5122, 2139, 473, 3188, 5051, 616, 6798, 5038, 7402, 6195, 647, 1399, 2285, 487, 6520, 6321, 5632, 3883, 7018, 114, 38, 3007, 1574, 1960, 6718, 4446, 2641, 7067, 954, 750, 2764, 3759, 2224, 2000, 3947, 5456, 6526, 189, 5414, 3328, 1153, 6920, 7295, 7042, 6440, 787, 222, 4112, 7283, 3767, 1367, 4917, 2268, 4048, 2719, 3302, 52, 285, 3630, 2942, 3888, 5910, 3526, 1018, 197, 6934, 857, 4962, 3854, 2960, 2328, 5778, 7352, 5008, 1485, 1624, 6421, 5143, 5842, 1587, 3135, 1613, 3879, 2964, 3909, 6590, 3440, 2644, 6527, 1765, 927, 4321, 2988, 135, 5772, 6830, 196, 811, 3460, 2137, 2879, 778, 3384, 7043, 1434, 6866, 4697, 6966, 4876, 398, 3001, 1935, 5207, 1682, 2107, 5235, 702, 2633, 5807, 5793, 4246, 7291, 7360, 3661, 3292, 3755, 1120, 6047, 3889, 6148, 839, 4836, 4542, 2565, 4245, 4392, 6141, 6548, 175, 2980, 1540, 178, 1534, 3504, 4828, 2279, 4424, 3029, 3563, 6865, 5073, 6574, 1210, 3901, 4258, 5746, 170, 6216, 2795, 7205, 495, 5342, 6879, 5154, 2901, 6368, 4415, 3693, 5225, 4144, 51, 466, 4492, 5172, 3499, 1012, 2507, 1347, 2167, 379, 1240, 3995, 1894, 2358, 1137, 6312, 5507, 4087, 2332, 6372, 6112, 3486, 1065, 3021, 7071, 6872, 1977, 1338, 2043, 6644, 3610, 5313, 5918, 2573, 2261, 3449, 7266, 3309, 7145, 3251, 4009, 378, 3116, 6399, 6610, 989, 844, 4276, 4561, 3359, 3655, 2158, 627, 831, 3037, 4817, 3651, 477, 663, 2683, 6810, 1901, 1049, 4146, 5275, 2317, 1085, 940, 2135, 3649, 2549, 308, 2272, 5124, 4965, 5854, 4125, 1585, 7155, 3745, 2215, 4019, 4947, 2125, 5322, 5614, 6432, 4779, 2808, 609, 4541, 3993, 4135, 6442, 1371, 2939, 3308, 1913, 5173, 2819, 1167, 6900, 5958, 4421, 4110, 1057, 2440, 515, 4615, 270, 518, 3919, 3976, 4939, 5325, 6076, 3487, 3678, 2129, 6850, 1924, 952, 7281, 6677, 498, 4127, 5412, 3017, 1622, 3938, 1128, 4210, 3412, 5638, 5452, 5237, 3228, 7002, 2825, 6704, 1352, 4778, 4322, 3898, 353, 5777, 2811, 3529, 1024, 626, 5436, 3764, 1318, 1211, 5370, 5444, 5651, 904, 5400, 3081, 380, 3109, 4908, 5287, 2017, 3474, 6544, 1390, 6974, 188, 14, 4190, 2822, 2366, 3725, 7387, 6271, 4023, 2647, 2579, 6869, 167, 1670, 4145, 3133, 619, 3261, 728, 5460, 4235, 622, 310, 6545, 1188, 328, 5399, 6516, 3399, 1705, 4931, 6109, 5188, 7054, 5985, 6283, 1884, 7272, 5028, 3437, 682, 1976, 2153, 983, 6281, 4486, 1, 3806, 621, 5350, 4267, 4823, 5600, 914, 1445, 6248, 3467, 5515, 3447, 4797, 1191, 1159, 4273, 2420, 1698, 5303, 5373, 2020, 7059, 3642, 1928, 4287, 3737, 4329, 7343, 429, 1045, 6582, 2004, 4166, 2425, 2243, 806, 6415, 6665, 323, 6260, 2026, 4715, 3066, 4191, 3150, 3910, 544, 1088, 2799, 314, 474, 6184, 2916, 739, 3138, 912, 827, 6037, 4974, 1266, 2530, 509, 6816, 246, 7166, 3952, 1633, 4189, 1657, 642, 6395, 5843, 3468, 5785, 3819, 1621, 3634, 905, 318, 3235, 3814, 3234, 3984, 4653, 99, 274, 1515, 6602, 6419, 5992, 4929, 3585, 3248, 2577, 1185, 2807, 3858, 4248, 2789, 6265, 6246, 2681, 2284, 4605, 6092, 1892, 3161, 2421, 3562, 4058, 1267, 2827, 3962, 4185, 1704, 4506, 1436, 7278, 4691, 2011, 5217, 3178, 2744, 1783, 5862, 2061, 2839, 7285, 2755, 3493, 1072, 4627, 4253, 866, 2575, 3454, 6686, 6513, 6420, 4473, 6565, 6695, 6593, 3503, 2240, 4196, 3517, 403, 6095, 1776, 6973, 4610, 1925, 1618, 453, 6402, 2412, 3220, 5277, 2067, 5467, 1298, 4387, 1416, 5191, 4834, 6646, 183, 3047, 3039, 2506, 6909, 5438, 6963, 6473, 6100, 4948, 834, 1110, 7154, 2249, 2881, 2115, 5059, 5798, 5105, 113, 1339, 4784, 5455, 1413, 1353, 1563, 5820, 877, 3167, 6497, 6151, 6753, 1868, 3882, 3364, 5790, 1956, 3438, 4622, 595, 5465, 6345, 4111, 7237, 2700, 1222, 5020, 1672, 2655, 126, 1293, 2054, 1731, 2865, 4346, 6379, 6855, 4621, 3056, 960, 524, 4790, 2858, 574, 5734, 5979, 4339, 4648, 5966, 6777, 2554, 4164, 2754, 2486, 6789, 2884, 129, 1005, 6102, 1453, 5423, 1361, 2970, 1192, 1226, 4199, 5236, 5271, 2194, 5518, 3060, 3223, 5409, 5832, 614, 680, 6577, 3502, 2661, 2120, 6600, 676, 5724, 2221, 3789, 1202, 1462, 7177, 3896, 4419, 1372, 7384, 3317, 5417, 2983, 2144, 6186, 508, 3100, 6592, 4457, 2568, 4194, 5052, 2981, 5531, 494, 5369, 786, 5088, 5476, 6394, 4892, 873, 541, 2707, 5077, 4169, 5668, 3307, 5871, 5550, 5830, 7377, 5359, 6286, 4090, 2775, 116, 5244, 1396, 6187, 3492, 961, 4370, 2837, 5592, 5337, 4608, 943, 1830, 7246, 4983, 1170, 6089, 1781, 2127, 2816, 968, 4500, 5655, 3497, 4175, 7365, 5570, 4879, 5973, 3155, 2377, 7162, 6655, 6612, 6212, 5132, 842, 3099, 3852, 2329, 4739, 7121, 5893, 5890, 1287, 7118, 709, 258, 4530, 3153, 5331, 118, 5406, 2694, 3934, 3937, 3561, 579, 4291, 4228, 6118, 5859, 4743, 2073, 6588, 5765, 6572, 7080, 255, 5016, 673, 3979, 5299, 4580, 1379, 2584, 4131, 254, 5145, 6970, 2992, 5612, 4602, 252, 5272, 6477, 1244, 286, 343, 2642, 5210, 7255, 3948, 4030, 5066, 3014, 1671, 5315, 235, 33, 5492, 5069, 2961, 4434, 5984, 5102, 5653, 5474, 879, 4645, 6221, 2298, 482, 2739, 6681, 0, 5735, 746, 6315, 1975, 5920, 2390, 123, 4957, 5040, 3478, 3579, 7203, 3556, 1863, 2318, 3762, 6543, 5980, 5575, 3927, 2845, 3540, 2566, 6537, 951, 5733, 3960, 4918, 6740, 4474, 2217, 4833, 6814, 6585, 4495, 153, 998, 2210, 5171, 3953, 4443, 1061, 3338, 7158, 2714, 2033, 6498, 181, 4224, 4516, 1254, 5231, 3791, 1609, 1742, 2389, 4916, 3673, 5626, 6898, 2778, 4319, 1458, 3620, 3805, 762, 5310, 2214, 2482, 1461, 7167, 6782, 6570, 4524, 7161, 1524, 313, 5000, 6034, 3245, 6126, 4120, 5491, 6213, 1964, 1512, 6374, 936, 6365, 1251, 748, 1121, 6108, 3122, 7094, 5791, 2271, 5977, 1986, 142, 6194, 3963, 6847, 1988, 4967, 374, 3706, 4180, 2606, 6097, 6858, 1517, 1402, 2820, 4521, 1991, 5841, 3635, 3028, 4690, 6637, 893, 2480, 4344, 5137, 2923, 1724, 2263, 6454, 2173, 3672, 1021, 5490, 6306, 6785, 3324, 2632, 182, 5334, 3057, 4596, 2828, 2422, 3250, 2846, 2871, 6299, 3752, 2850, 251, 3665, 666, 783, 7320, 2761, 1900, 6310, 3576, 4808, 771, 815, 1635, 1440, 4795, 4851, 4527, 282, 5432, 5323, 797, 3214, 7021, 1124, 5048, 5767, 3945, 6133, 784, 4869, 4649, 472, 356, 1108, 6137, 837, 2915, 3063, 6326, 4806, 3507, 4141, 6562, 1838, 5165, 5193, 1134, 2048, 5901, 1404, 7265, 4519, 4006, 6793, 6073, 1535, 3659, 1539, 4469, 4450, 3711, 4477, 3613, 2902, 24, 2493, 5580, 2458, 2066, 2232, 3124, 6376, 3920, 7289, 1200, 4954, 68, 4254, 2711, 991, 5510, 6339, 4555, 6220, 6772, 394, 2567, 480, 3679, 2706, 4556, 5486, 2690, 3193, 3622, 2006, 4996, 3730, 832, 3881, 5608, 5113, 1772, 279, 4624, 6083, 1841, 6150, 2591, 607, 347, 3177, 3142, 7395, 5160, 878, 5252, 1516, 2663, 2804, 5674, 4594, 5212, 4357, 3052, 2096, 4153, 5525, 7327, 6171, 2539, 2405, 5378, 5561, 7156, 812, 4367, 291, 7114, 2699, 675, 5768, 974, 6854, 4886, 1360, 3652, 591, 6255, 2792, 1518, 3092, 77, 6657, 4104, 6238, 5892, 6747, 6175, 3597, 445, 659, 4360, 1270, 6895, 613, 3105, 4749, 769, 2499, 5673, 3980, 4098, 677, 7347, 7215, 2101, 3736, 6057, 2078, 4015, 4687, 4512, 7322, 6131, 6124, 586, 5104, 6398, 5517, 5046, 2525, 4499, 4590, 2728, 3775, 3522, 3653, 4468, 1959, 3723, 2170, 320, 7008, 5737, 185, 3332, 5616, 4981, 665, 1086, 1717, 7083, 2331, 1236, 5253, 918, 3264, 5708, 976, 4178, 2223, 2080, 4959, 7368, 692, 6521, 462, 2708, 3351, 7035, 6643, 1250, 2994, 1033, 1098, 2076, 2184, 6423, 6962, 870, 5487, 3123, 3392, 3521, 4737, 3201, 2897, 433, 1917, 2878, 3607, 2008, 1703, 6714, 6074, 3519, 2969, 1856, 1665, 2954, 1291, 4878, 3128, 3974, 1626, 5419, 6227, 3095, 6628, 3689, 2295, 3238, 6121, 6691, 5471, 2423, 2418, 3875, 245, 6314, 4535, 2476, 4706, 2580, 3356, 2164, 2488, 2770, 2259, 2756, 3469, 3045, 6159, 6767, 1206, 4328, 3375, 2281, 6531, 2383, 6569, 6302, 205, 3567, 4573, 7144, 6857, 3433, 7184, 5914, 4810, 4625, 4454, 4821, 1038, 6702, 1655, 1889, 2886, 6054, 1513, 7316, 3581, 6173, 799, 1136, 1017, 2400, 219, 1600, 1673, 5656, 1040, 7270, 6863, 6828, 325, 6862, 3611, 2798, 4389, 2369, 6437, 4167, 5523, 5552, 3185, 5577, 46, 1090, 1332, 3827, 7179, 1002, 5970, 3582, 1966, 3841, 393, 1066, 3151, 4906, 7236, 6425, 3820, 3019, 4705, 5519, 1130, 4528, 3600, 6004, 2517, 6446, 3868, 3340, 7315, 3849, 93, 3826, 4106, 3560, 4791, 6400, 1439, 5138, 1858, 5849, 3874, 4756, 1025, 4124, 6836, 2887, 501, 5647, 2437, 4679, 4263, 155, 883, 437, 5435, 348, 5650, 3867, 5663, 7097, 6179, 1828, 5680, 5780, 1311, 1686, 2239, 7096, 6011, 971, 5795, 1719, 5618, 5431, 131, 5731, 6431, 2391, 5506, 2311, 3847, 366, 4972, 5246, 1447, 3159, 2424, 1725, 1699, 397, 7338, 2303, 4236, 527, 2624, 444, 6000, 4069, 2401, 5969, 4052, 5022, 4156, 5361, 3748, 1069, 3280, 1860, 2907, 2209, 1909, 1819, 5760, 5975, 3872, 5026, 3431, 1536, 2854, 1493, 6723, 2490, 7197, 435, 749, 447, 1881, 5128, 6360, 4475, 1603, 6536, 6679, 1616, 2583, 4712, 1426, 738, 6608, 2463, 1816, 2957, 2689, 3033, 260, 690, 4359, 2460, 2414, 6942, 648, 7149, 2735, 500, 505, 2803, 2160, 5513, 5599, 4140, 4910, 7190, 4673, 4831, 5390, 6409, 100, 442, 1684, 1510, 2393, 5360, 3871, 3370, 1355, 3705, 5879, 3674, 2966, 1754, 1948, 1094, 6335, 4802, 938, 140, 1261, 743, 935, 6099, 668, 5905, 3618, 6832, 6337, 5470, 5183, 6165, 1726, 809, 6961, 5607, 1827, 4861, 386, 1008, 1013, 1026, 3117, 4595, 6722, 4772, 4936, 1430, 5074, 4958, 4227, 1393, 2893, 2937, 3989, 1840, 923, 822, 5042, 6020, 3050, 6656, 2550, 3353, 5872, 846, 1147, 5408, 3626, 168, 6176, 7165, 2324, 3776, 1257, 4665, 5296, 7147, 699, 5427, 6634, 4651, 3822, 5806, 4200, 3211, 184, 5644, 5036, 5146, 986, 5771, 2406, 1007, 7268, 590, 2439, 5904, 244, 2987, 5083, 603, 3987, 1042, 145, 6864, 1645, 4177, 7279, 5678, 2972, 7342, 3855, 2976, 266, 727, 2399, 1654, 3075, 1301, 6670, 2367, 1646, 6605, 745, 264, 5477, 6279, 3728, 3530, 6201, 2428, 4489, 1677, 4502, 7308, 6462, 7029, 2843, 2727, 773, 6387, 3196, 2290, 1628, 1375, 3924, 4740, 4985, 3834, 6642, 3121, 6464, 6846, 6506, 3032, 5919, 4969, 833, 4226, 6494, 2788, 70, 3420, 7163, 1497, 6524, 1898, 7075, 3389, 763, 2672, 3436, 5005, 6222, 2831, 5425, 2304, 6697, 4849, 6632, 6182, 1023, 5982, 3236, 4324, 450, 377, 6278, 2824, 1521, 853, 7026, 6977, 5459, 3715, 4203, 488, 1284, 200, 5684, 4337, 7362, 425, 4564, 6913, 2165, 1470, 1658, 419, 1764, 6237, 3559, 2292, 4636, 1312, 6007, 6210, 1224, 6370, 2072, 3219, 6945, 2041, 1328, 2670, 5664, 2564, 6606, 1504, 1420, 5851, 414, 733, 4560, 3136, 5878, 1630, 4289, 687, 6907, 1077, 5184, 701, 2617, 5829, 4163, 1644, 5144, 2603, 2471, 2162, 1601, 2796, 3971, 5158, 3853, 3781, 2925, 6749, 5162, 4970, 7286, 532, 1874, 6528, 6268, 6503, 3545, 213, 4722, 5090, 7181, 5634, 4894, 975, 3964, 6030, 332, 1268, 1163, 6741, 1802, 3506, 5163, 2763, 577, 3925, 4751, 6661, 6532, 1214, 1142, 6944, 6373, 4846, 7213, 6429, 360, 4838, 1003, 5384, 4356, 1131, 5443, 5545, 1231, 1685, 1151, 4901, 6098, 2676, 1218, 6801, 4598, 6949, 3535, 4612, 1607, 7312, 4933, 109, 5837, 4941, 1962, 6069, 5300, 4438, 4576, 6948, 65, 5689, 6626, 585, 357, 7062, 801, 2958, 766, 6971, 4503, 1329, 1471, 651, 1492, 3051, 3256, 1382, 5348, 900, 3218, 7277, 7099, 6389, 652, 7344, 2977, 2051, 1331, 3483, 636, 4963, 1680, 4942, 1666, 3648, 7334, 4172, 6595, 5726, 1292, 6706, 6705, 5999, 5809, 2247, 229, 5740, 3024, 1887, 4549, 1777, 1954, 1454, 5355, 6486, 1720, 7095, 507, 3216, 144, 2686, 1861, 4380, 6726, 1118, 1714, 2169, 6393, 2791, 4643, 4616, 3496, 3520, 4285, 1789, 3510, 1490, 3913, 4002, 6158, 537, 1980, 7116, 2, 4066, 6467, 4481, 3414, 6745, 387, 2226, 4274, 5079, 4233, 6412, 2175, 7098, 686, 2723, 3954, 5640, 4704, 884, 6886, 6388, 2598, 3432, 5111, 6717, 2950, 1425, 6348, 6541, 1283, 7200, 3658, 3068, 1631, 1494, 89, 845, 1550, 6542, 7036, 3461, 7063, 1354, 5921, 2505, 6341, 7004, 3265, 5401, 3154, 3343, 3137, 3766, 753, 5501, 4518, 4672, 861, 826, 5800, 6287, 1594, 6017, 6941, 3197, 4671, 7150, 1216, 511, 3604, 1424, 1320, 1593, 1650, 2470, 6009, 5352, 4733, 4244, 944, 6082, 7376, 6712, 1683, 1498, 4089, 3557, 469, 7172, 2218, 6795, 2015, 4192, 6273, 548, 6474, 2003, 2741, 4049, 4987, 1691, 5175, 1313, 3090, 5788, 2334, 1508, 1938, 212, 1265, 7332, 7293, 1337, 6667, 988, 1197, 1422, 4540, 7373, 5690, 5954, 7263, 3809, 1149, 5695, 1044, 2621, 3067, 6859, 3158, 2266, 2738, 103, 376, 4428, 6466, 3554, 5023, 1388, 4183, 3388, 649, 4634, 4159, 5017, 2084, 6318, 6684, 6825, 2322, 1246, 1215, 3816, 4, 6026, 5340, 418, 4982, 965, 3932, 5524, 2585, 4695, 512, 1569, 1865, 3407, 4538, 7339, 6507, 1871, 2696, 3754, 4635, 239, 5739, 1158, 3312, 1970, 1001, 5200, 7103, 1383, 5250, 3183, 1080, 5714, 1611, 4839, 859, 782, 575, 2742, 2350, 2610, 1046, 1775, 963, 250, 6874, 4096, 2018, 5949, 5502, 2384, 4283, 828, 6411, 7105, 3914, 3518, 7370, 5875, 4863, 468, 1190, 3880, 2702, 4670, 5014, 3602, 4342, 3491, 1664, 80, 3049, 391, 5450, 6450, 268, 4646, 124, 6849, 3102, 1255, 7007, 2835, 2582, 2801, 1427, 3202, 2287, 2095, 7030, 3580, 2771, 6189, 7052, 7245, 4507, 522, 4775, 6958, 2997, 1799, 7399, 2718, 1728, 5295, 1576, 2618, 3548, 104, 6509, 1294, 3537, 3792, 2634, 987, 4063, 633, 1961, 4711, 4896, 6410, 4532, 6922, 2898, 4571, 3316, 4871, 1571, 6289, 6762, 7131, 4300, 705, 4960, 4525, 1204, 6140, 6292, 6049, 1377, 1119, 661, 1095, 5615, 5711, 3797, 6330, 44, 7023, 6770, 4310, 2023, 243, 74, 156, 2195, 3104, 3676, 5259, 1998, 3002, 3174, 4847, 3395, 3936, 5123, 5357, 4008, 3638, 456, 7244, 4692, 1256, 4152, 5264, 3373, 1579, 6191, 7375, 6790, 4655, 4451, 1648, 2108, 5483, 2842, 1296, 5974, 7051, 3441, 3446, 3300, 5232, 5835, 6728, 4448, 4814, 597, 1482, 2430, 3175, 490, 3657, 6119, 1681, 5516, 3550, 667, 3485, 4732, 4275, 1009, 556, 4121, 5270, 6261, 5641, 6471, 159, 6861, 546, 3379, 4904, 6071, 3669, 703, 3140, 6658, 593, 6036, 4932, 5071, 618, 3397, 1818, 6048, 5284, 5257, 906, 6167, 458, 1145, 3074, 3411, 1801, 1499, 1105, 3372, 2090, 2269, 3207, 568, 5558, 5822, 2352, 4597, 6533, 1933, 7082, 5691, 2979, 5305, 4642, 3323, 1617, 7358, 4903, 1417, 6752, 7122, 2012, 1831, 2469, 2429, 2930, 3209, 3785, 73, 814, 6110, 6199, 7337, 5721, 382, 6185, 215, 7209, 4174, 2204, 7025, 3042, 6873, 3956, 6690, 32, 7048, 3470, 4517, 5773, 671, 4505, 6901, 599, 5424, 1029, 6319, 1721, 3703, 1859, 410, 1168, 691, 1793, 4750, 4037, 6500, 1713, 2180, 6763, 5336, 829, 6683, 6038, 5661, 5468, 164, 7405, 3863, 5606, 3, 528, 7218, 6682, 3342, 4232, 297, 6416, 4993, 5937, 1839, 3957, 4746, 228, 4088, 4401, 2509, 670, 1062, 1533, 1449, 2351, 5362, 5566, 1455, 1813, 962, 2548, 1979, 3719, 958, 2698]
Shuffle5 = [1796, 4758, 2662, 1519, 2309, 5172, 6785, 2874, 4136, 2705, 2405, 7105, 2211, 4208, 5440, 608, 3914, 6630, 2258, 1900, 111, 5649, 5522, 3482, 5328, 7073, 94, 6449, 2229, 7061, 3286, 7388, 5449, 1660, 3564, 7099, 4035, 6099, 732, 5431, 6102, 4031, 4741, 4547, 2122, 3429, 6999, 7362, 765, 1506, 6720, 1437, 4146, 2783, 3177, 4068, 1864, 7181, 6665, 6327, 7005, 7200, 2508, 5198, 445, 268, 756, 3150, 3836, 2948, 2279, 563, 5128, 5982, 5428, 716, 7380, 3663, 6256, 1754, 3499, 5734, 4748, 1554, 3794, 2949, 1689, 2466, 5177, 541, 6245, 7172, 4312, 5849, 1550, 2227, 785, 6125, 1945, 4158, 5729, 2972, 6704, 677, 5193, 4314, 6680, 4551, 2854, 4221, 2370, 1292, 502, 2147, 3596, 1774, 6166, 4992, 4893, 2413, 4611, 1995, 6742, 6134, 2993, 708, 5068, 4137, 7392, 2969, 7159, 6265, 4269, 6849, 6354, 1083, 3054, 4055, 1259, 4994, 6753, 1704, 178, 1183, 1063, 95, 802, 2770, 7231, 4088, 1355, 4678, 5178, 1951, 5622, 4453, 5826, 892, 820, 7140, 3000, 1531, 488, 485, 3962, 4556, 6993, 2496, 5259, 1196, 2016, 3978, 872, 3299, 7237, 2365, 4149, 5363, 3259, 4766, 436, 3267, 2714, 6211, 4751, 3008, 1851, 6032, 1749, 1369, 5024, 5349, 4874, 3475, 7228, 569, 6718, 1725, 3003, 32, 6248, 4185, 1353, 4634, 6450, 6988, 2464, 3491, 6800, 6611, 591, 6502, 4690, 5817, 702, 6505, 3747, 2487, 6631, 7075, 4336, 6390, 1152, 3170, 4305, 3023, 838, 6297, 5376, 3527, 4914, 5941, 5000, 527, 5648, 841, 2224, 2690, 5853, 6425, 1812, 6535, 3212, 671, 642, 1958, 5227, 2058, 4937, 6688, 855, 120, 7104, 2582, 6703, 3131, 508, 7133, 2072, 4451, 1162, 1717, 578, 5034, 68, 6855, 1987, 3049, 5201, 1785, 1007, 2836, 801, 5002, 2843, 4063, 7209, 4232, 6893, 2661, 2468, 2943, 3790, 6700, 4805, 6597, 5139, 4626, 2893, 687, 1315, 6043, 1464, 2097, 531, 3559, 4735, 4485, 5754, 7050, 2358, 2400, 165, 2301, 1780, 3497, 4443, 3532, 6177, 7040, 5766, 766, 4484, 6758, 5877, 5410, 2094, 3639, 2428, 758, 4274, 196, 5868, 7222, 757, 5043, 7036, 1600, 6107, 5763, 5436, 1532, 717, 3314, 6599, 7339, 960, 4911, 7350, 359, 4413, 3400, 4295, 4671, 5039, 3768, 5181, 7364, 2695, 2442, 6561, 5186, 4827, 4959, 4300, 3702, 6304, 4833, 7060, 3383, 3649, 2510, 1640, 3863, 6947, 5026, 2144, 2769, 2188, 3109, 3585, 3171, 212, 5262, 3645, 6031, 2438, 5662, 4181, 3986, 2967, 726, 2512, 3239, 6757, 6306, 3659, 6030, 609, 786, 3399, 6696, 5913, 5525, 4270, 5602, 2891, 1991, 351, 2509, 4278, 6023, 3743, 6782, 1380, 1013, 5257, 6769, 4376, 2000, 3824, 227, 623, 5084, 4272, 6329, 2896, 6141, 2349, 5563, 6907, 1087, 2282, 2607, 6965, 7399, 6145, 2668, 5908, 3464, 3577, 6805, 547, 4727, 7054, 691, 54, 4908, 6978, 64, 5797, 1418, 4050, 1006, 4921, 5105, 4880, 995, 3152, 1291, 6895, 3413, 401, 2913, 3869, 56, 5242, 5695, 1594, 1603, 2957, 3852, 5302, 310, 1178, 1069, 4053, 4564, 3875, 2596, 5142, 5161, 5209, 5046, 5776, 2849, 5607, 2197, 4930, 4083, 5465, 5321, 6567, 6172, 1565, 2, 2799, 2819, 1645, 1109, 2182, 220, 1893, 3428, 395, 5219, 7305, 648, 503, 5966, 5415, 2947, 1047, 5261, 727, 273, 266, 1115, 6228, 3587, 317, 5236, 2025, 4247, 6474, 2491, 3672, 41, 6111, 2143, 2244, 2372, 3677, 3443, 4515, 2848, 4001, 7302, 2199, 3959, 596, 5327, 863, 521, 7101, 3265, 2840, 7336, 6197, 5957, 7378, 2168, 2569, 5481, 1156, 1392, 2765, 174, 4977, 3710, 6370, 5851, 2454, 2515, 3561, 3271, 2114, 3729, 5324, 954, 4527, 6686, 551, 6636, 3457, 495, 4099, 1847, 121, 2292, 2397, 851, 354, 5317, 4523, 4846, 6090, 530, 1967, 1982, 1008, 176, 5413, 4617, 4161, 3483, 6655, 999, 4159, 1228, 682, 2586, 2610, 3760, 1433, 304, 1390, 6931, 5458, 1121, 6008, 5792, 873, 509, 7218, 3712, 7368, 5065, 2689, 4364, 2880, 4117, 6492, 3160, 5893, 6604, 4565, 2673, 5243, 6969, 4664, 3583, 4496, 6207, 167, 5472, 5626, 7256, 3249, 2248, 7311, 1041, 7023, 3263, 4123, 4839, 2822, 3492, 6454, 7269, 4075, 3436, 204, 255, 1625, 3555, 2968, 6302, 6787, 2026, 1414, 5901, 4583, 5790, 6823, 4022, 6372, 6321, 3368, 361, 303, 393, 5569, 6332, 4369, 6734, 7239, 2068, 2054, 249, 5258, 1036, 3165, 2280, 4358, 5902, 4811, 2922, 1694, 7148, 5862, 3830, 3966, 3266, 1602, 4697, 6587, 1700, 6004, 2547, 3519, 1939, 3745, 3490, 3633, 4076, 2001, 3939, 5119, 2890, 7203, 2488, 1113, 2251, 2046, 2045, 2961, 4007, 4886, 7062, 3462, 4021, 2901, 1860, 5293, 5041, 285, 84, 5023, 7290, 1606, 4960, 422, 4283, 783, 1705, 980, 1314, 3538, 1730, 964, 2775, 3007, 5997, 3598, 4486, 6888, 6644, 3488, 77, 1975, 6946, 2587, 5872, 3484, 4891, 2923, 5067, 1901, 1778, 3670, 6194, 6877, 1273, 6275, 2117, 5928, 559, 2206, 2126, 5194, 92, 2024, 2809, 5887, 6866, 3833, 7132, 7309, 6741, 7029, 5045, 5291, 1636, 4296, 3389, 3409, 5312, 6774, 2984, 2174, 5715, 6204, 2606, 2007, 754, 3256, 6713, 5967, 3625, 5576, 5060, 4744, 3021, 6158, 6641, 2721, 7, 3518, 1201, 7185, 5108, 90, 5148, 7223, 6839, 5104, 2504, 2536, 1673, 5764, 6662, 550, 6547, 186, 5294, 1632, 5164, 4686, 5151, 6404, 516, 363, 5779, 3548, 6127, 2022, 4989, 4420, 5329, 3489, 2665, 6975, 961, 6200, 1226, 6426, 500, 2767, 2028, 4604, 6697, 5497, 5451, 4426, 6971, 5699, 1517, 4115, 3591, 2820, 1542, 5787, 7338, 29, 6213, 968, 6488, 7353, 1538, 1921, 2787, 1710, 3347, 1793, 4749, 6984, 6258, 6251, 104, 4176, 336, 2521, 5283, 6339, 5270, 5863, 7294, 117, 2812, 5094, 2638, 5744, 6503, 2303, 967, 4348, 592, 4119, 6355, 4372, 7314, 4057, 5082, 4950, 2287, 6396, 2670, 5629, 1997, 730, 6120, 1514, 3311, 5500, 394, 4909, 7095, 6637, 5025, 4097, 1237, 2811, 3544, 377, 4435, 795, 6861, 4860, 27, 3377, 3805, 6047, 1511, 1138, 3751, 2259, 1746, 5789, 6448, 2198, 4996, 6676, 6216, 6949, 309, 6601, 2804, 4622, 4214, 2080, 1399, 2938, 3126, 6539, 3763, 589, 6873, 4062, 1763, 454, 405, 6285, 6932, 7002, 7123, 1240, 3813, 6135, 4375, 1460, 3391, 2773, 1816, 6001, 3667, 5583, 6673, 2492, 885, 4783, 850, 1070, 3930, 3102, 4550, 3580, 5316, 5063, 4156, 1616, 3689, 6815, 4282, 3522, 5560, 6724, 3594, 1188, 3036, 5891, 3237, 5821, 2129, 4813, 1524, 1458, 278, 4095, 2796, 5945, 4929, 4715, 3154, 6926, 2249, 911, 4078, 3326, 7106, 1179, 4079, 5462, 6921, 3841, 2444, 7098, 143, 5421, 3660, 902, 7205, 825, 5199, 7068, 5272, 1721, 2505, 7108, 7297, 1742, 1925, 805, 6525, 4144, 3404, 4870, 5434, 6740, 4635, 2123, 1042, 7334, 5044, 4822, 21, 2618, 5833, 1038, 2802, 913, 1415, 3078, 2145, 5273, 5793, 4998, 6427, 4498, 3904, 6155, 6068, 3096, 532, 6262, 2337, 5075, 6290, 5173, 5346, 567, 5355, 1288, 5630, 2342, 827, 2458, 7213, 1653, 3284, 4038, 1027, 729, 1426, 4338, 2329, 4069, 6555, 7279, 7124, 3371, 4507, 5904, 7329, 3840, 2327, 878, 6486, 5409, 1743, 4957, 1428, 571, 3228, 406, 3022, 3713, 4172, 6619, 6088, 6761, 1815, 6788, 5561, 3323, 2480, 1406, 4832, 6859, 3080, 595, 1324, 5088, 2700, 1758, 2346, 3250, 460, 7260, 2679, 1072, 4049, 4197, 2615, 3127, 3101, 1776, 3636, 4586, 2332, 1286, 5001, 5875, 420, 2978, 6104, 224, 5922, 3061, 2233, 4500, 5924, 833, 1802, 4614, 6418, 2500, 996, 367, 1018, 2257, 4444, 3373, 3137, 5646, 3764, 5753, 1150, 1185, 5459, 991, 1685, 1764, 2524, 4718, 799, 926, 4106, 2230, 3517, 4254, 3802, 6162, 5915, 1883, 1505, 4320, 4993, 3273, 7003, 5475, 3754, 1427, 1518, 4884, 5995, 6051, 512, 2987, 5083, 2420, 3897, 2622, 6165, 4034, 3901, 329, 5416, 5183, 442, 2578, 83, 4317, 6911, 4602, 193, 5781, 6958, 1733, 1372, 5050, 4478, 4649, 4243, 3589, 259, 7175, 6779, 5712, 6634, 7391, 4030, 5176, 5017, 6995, 7284, 5988, 7372, 3033, 4648, 428, 3560, 2768, 3579, 1233, 7377, 7179, 1100, 1334, 1628, 4266, 6024, 1947, 1333, 2396, 3445, 7092, 3354, 2646, 6017, 3116, 797, 887, 3734, 4425, 6392, 735, 6407, 2680, 3972, 7116, 2790, 1466, 3534, 3817, 2165, 5336, 238, 3006, 5305, 7220, 1736, 5655, 3348, 7142, 501, 2106, 5027, 2311, 1194, 6109, 3915, 2113, 7188, 3134, 1989, 6375, 2944, 1617, 7357, 5540, 3882, 741, 938, 3026, 6776, 4892, 4831, 750, 738, 3857, 3148, 1271, 5667, 5600, 5232, 6041, 4307, 2269, 2437, 1751, 4346, 7041, 4061, 3186, 4219, 4531, 985, 229, 6073, 3964, 5663, 522, 5691, 3394, 5443, 7047, 2798, 2887, 1028, 2347, 4277, 6249, 936, 5288, 1482, 2395, 7241, 1684, 528, 6052, 6887, 728, 211, 2088, 19, 5604, 1387, 5471, 5342, 2548, 3885, 4179, 4489, 6318, 7274, 2702, 4570, 52, 3027, 588, 6536, 3512, 2139, 4612, 6246, 1449, 6910, 5306, 7074, 2109, 2030, 4308, 3500, 1842, 374, 4145, 498, 3434, 2522, 3516, 745, 6957, 300, 7161, 87, 4501, 4791, 2724, 424, 2900, 1546, 4532, 5558, 109, 6512, 3597, 1697, 4962, 3187, 6388, 4771, 6173, 2738, 1631, 3547, 5011, 6014, 1806, 6419, 4260, 6528, 2876, 2232, 4640, 5665, 3653, 2675, 3611, 6588, 3253, 4427, 4549, 1050, 2351, 4997, 706, 1578, 7296, 6324, 3075, 5876, 6811, 14, 3838, 1770, 580, 6558, 6514, 3162, 3989, 7114, 2565, 6825, 3541, 2079, 1985, 23, 6447, 965, 5503, 6363, 1983, 2624, 504, 6564, 2824, 3199, 526, 2255, 3825, 3873, 1444, 1258, 3233, 35, 1326, 6953, 2762, 2052, 1629, 1114, 3441, 5502, 5903, 7351, 586, 2674, 5595, 5192, 6519, 2752, 579, 3247, 4047, 298, 553, 870, 147, 2595, 5692, 6596, 6533, 6108, 895, 453, 3967, 6795, 5836, 4752, 2694, 3816, 6138, 4627, 5544, 3704, 5224, 3651, 775, 5918, 6149, 2234, 1411, 1702, 384, 1954, 4842, 7356, 6117, 6981, 3353, 1984, 1933, 2930, 2945, 6884, 7342, 5714, 6917, 2034, 6784, 5985, 3549, 407, 2263, 6799, 6184, 2888, 1581, 4275, 6348, 1556, 6153, 4265, 2204, 5473, 348, 3570, 5961, 4854, 6919, 846, 282, 471, 557, 6846, 3082, 5239, 6730, 535, 5248, 6376, 7389, 1455, 5499, 1571, 12, 5936, 611, 2326, 6033, 5364, 3288, 1509, 1084, 5207, 1897, 130, 6707, 6660, 134, 6916, 6897, 5325, 1340, 1772, 1382, 5620, 4935, 5880, 1766, 6678, 4800, 6002, 5171, 214, 3494, 1213, 4388, 4374, 3141, 1726, 3613, 3844, 4770, 3812, 5138, 683, 119, 2917, 5950, 6832, 3554, 7055, 4418, 3854, 5864, 6236, 2245, 3658, 602, 1547, 824, 5723, 6429, 3819, 4367, 4952, 6890, 1025, 289, 1666, 1401, 1677, 4366, 5839, 4203, 5423, 539, 5673, 5004, 1175, 6027, 1814, 2933, 6867, 707, 5157, 2319, 2697, 3979, 4618, 5432, 6750, 6357, 3772, 3044, 3728, 2471, 1753, 6804, 4476, 3182, 6620, 2154, 2676, 5722, 976, 5594, 4169, 755, 7398, 4861, 6377, 4112, 243, 1722, 5313, 3752, 565, 194, 5211, 3123, 933, 3367, 3210, 5009, 159, 5234, 1510, 2621, 5506, 2870, 1405, 6394, 2829, 6269, 4901, 5182, 1424, 6308, 1911, 5666, 2707, 425, 4573, 3417, 6206, 2803, 4462, 7270, 4953, 2295, 6116, 4343, 443, 891, 5090, 91, 3687, 5759, 4009, 5953, 4167, 970, 759, 5315, 3804, 455, 2377, 5819, 7121, 6232, 6399, 463, 721, 6550, 1771, 1487, 1151, 414, 283, 4636, 6661, 4896, 3278, 7160, 5444, 816, 5709, 4130, 7354, 6283, 3, 1195, 6049, 725, 907, 6793, 2164, 6462, 5959, 6217, 6540, 5900, 293, 2990, 125, 4395, 1167, 3762, 6252, 2020, 1367, 6035, 1840, 3858, 6444, 7402, 6608, 381, 2931, 5838, 182, 4577, 2841, 326, 6233, 5057, 6373, 6523, 5076, 4700, 6412, 6010, 1880, 3553, 4355, 4696, 6690, 7233, 5260, 957, 1502, 2723, 353, 916, 4480, 977, 2692, 4560, 5574, 5978, 7027, 3118, 3581, 1719, 832, 4341, 767, 3630, 7115, 5528, 7207, 5488, 6934, 416, 2805, 2956, 3093, 4615, 2354, 762, 1906, 3403, 2777, 6405, 6074, 4189, 5071, 6069, 5429, 7313, 7245, 4073, 5703, 2626, 1918, 6307, 815, 265, 2350, 6361, 4768, 3780, 5340, 7117, 941, 4378, 6797, 5137, 3139, 3902, 2501, 6202, 5972, 3155, 2386, 2002, 7404, 213, 3981, 1994, 2071, 1585, 4481, 5552, 235, 1478, 944, 5446, 930, 1186, 1365, 2390, 2817, 5564, 3924, 2108, 7235, 4729, 4794, 2959, 1155, 2183, 5295, 2763, 6610, 5414, 2914, 856, 1545, 7064, 1295, 1589, 1783, 5203, 2062, 1979, 5783, 2286, 2382, 1080, 2162, 4490, 2716, 2964, 3042, 6461, 419, 3590, 5245, 5116, 330, 3998, 6908, 5735, 1667, 5361, 5678, 5285, 2682, 4454, 5106, 1930, 1748, 2005, 4928, 6943, 3345, 6123, 6694, 6343, 6, 773, 577, 2362, 6906, 3178, 5800, 803, 2962, 1108, 3886, 1873, 5885, 6475, 423, 4603, 4281, 5841, 1966, 160, 53, 7118, 4792, 2483, 263, 542, 6936, 1039, 6151, 517, 3119, 7226, 2791, 1739, 5562, 1579, 3121, 415, 6904, 314, 4889, 2864, 5855, 5756, 1871, 6526, 4293, 1686, 20, 5774, 246, 7319, 4302, 6899, 7250, 1823, 2687, 2871, 3855, 733, 969, 4402, 6575, 6411, 4349, 6798, 720, 3691, 1658, 4333, 3447, 587, 1676, 6579, 3793, 2202, 5163, 4663, 4288, 2369, 3236, 4730, 3334, 4776, 6928, 4032, 4337, 5570, 446, 5708, 1272, 3014, 7352, 3473, 5623, 1735, 4514, 819, 1247, 4318, 3576, 5842, 619, 216, 3994, 3531, 7174, 4851, 700, 7327, 465, 4419, 4415, 652, 6451, 479, 1297, 2176, 7211, 3617, 1280, 3325, 3069, 5097, 2955, 685, 3909, 1784, 3202, 3674, 5755, 6065, 2655, 5881, 3808, 966, 1937, 7173, 1661, 185, 4781, 906, 3695, 3529, 743, 105, 3603, 6976, 6985, 4289, 3340, 2800, 4431, 3668, 2226, 4984, 1190, 3268, 2093, 704, 5625, 7196, 3276, 3064, 1781, 4949, 5129, 5617, 7304, 1790, 1586, 2092, 894, 3213, 5613, 927, 4368, 4717, 612, 1149, 155, 2142, 5820, 1543, 4482, 6241, 269, 538, 5890, 5682, 2105, 615, 5278, 1626, 1568, 4190, 481, 1044, 5661, 5100, 2448, 680, 2589, 823, 4779, 4815, 3355, 4746, 1134, 3105, 2546, 5608, 2425, 5221, 6323, 5220, 1824, 3887, 3357, 2807, 13, 2837, 2538, 6578, 3578, 7072, 951, 2453, 2306, 5461, 6413, 4191, 6813, 1775, 4738, 474, 2436, 7150, 357, 2499, 6326, 1389, 4942, 6333, 3614, 3742, 1451, 2355, 3450, 1664, 4056, 233, 398, 2184, 1264, 989, 1250, 5539, 4524, 3191, 5812, 1002, 3622, 2131, 3108, 3709, 1868, 1537, 2314, 332, 6087, 3248, 1054, 3898, 3386, 5553, 1583, 6234, 3002, 1394, 5489, 5828, 3864, 2037, 3721, 5984, 1359, 2681, 3390, 4503, 2549, 2946, 6617, 5762, 7184, 6261, 2450, 4215, 2235, 2632, 2744, 3423, 7170, 5785, 4027, 4922, 679, 934, 1106, 5384, 349, 5943, 2535, 2741, 3770, 3215, 5555, 2852, 3609, 4762, 2983, 888, 2526, 2801, 1869, 4447, 6021, 5521, 3894, 2704, 1552, 5917, 1941, 1715, 2415, 2189, 5848, 4396, 3117, 724, 5427, 7322, 1857, 1839, 3495, 449, 3552, 5200, 663, 6019, 4900, 937, 4521, 6562, 1118, 1318, 3086, 5767, 49, 3474, 1284, 3156, 33, 139, 5824, 5168, 3298, 6178, 1417, 1397, 5633, 6534, 4360, 1610, 908, 4163, 5396, 480, 7153, 1423, 327, 6270, 4988, 6591, 3451, 1909, 523, 3657, 1465, 1402, 4342, 5668, 5571, 4230, 3973, 3378, 6468, 2493, 1642, 2181, 5581, 5645, 1199, 2220, 6699, 953, 4968, 6489, 1442, 2642, 1285, 1963, 1270, 1360, 3324, 4590, 3446, 4245, 6164, 3719, 192, 1522, 5206, 6732, 3688, 146, 3877, 3219, 667, 5395, 1331, 5739, 4234, 6896, 3539, 1977, 5014, 7078, 157, 5309, 1627, 650, 2240, 1646, 2102, 4754, 3607, 6136, 7242, 62, 3272, 1828, 790, 1692, 1652, 2223, 7039, 1558, 3084, 2393, 6337, 1877, 7258, 3908, 1861, 1544, 2832, 1745, 1139, 2650, 2013, 6271, 3678, 6708, 6806, 6480, 3398, 3664, 1429, 3602, 922, 4218, 5454, 338, 3911, 3041, 4002, 4885, 2574, 6463, 4701, 1045, 2774, 6235, 1408, 971, 5456, 1682, 274, 3200, 2540, 3922, 7119, 6362, 7331, 4566, 3940, 2340, 1090, 2792, 6625, 3088, 4853, 879, 4963, 6615, 2846, 4046, 2986, 4291, 4319, 4380, 2760, 537, 1993, 5531, 4657, 7066, 6607, 5749, 7177, 3778, 5591, 7400, 2218, 6368, 4330, 6129, 1197, 3692, 3077, 4449, 4509, 6902, 3052, 138, 5933, 540, 4772, 2266, 426, 1065, 4580, 2988, 1111, 38, 4442, 562, 1526, 2012, 5557, 3503, 323, 545, 1931, 1296, 3067, 692, 1346, 5624, 5596, 3100, 3001, 1889, 6940, 869, 2278, 924, 6892, 3748, 1202, 2009, 4220, 5351, 2036, 2649, 7166, 6671, 5542, 818, 5093, 4599, 98, 225, 3627, 3175, 5111, 543, 399, 2185, 3011, 5407, 179, 2953, 6287, 6939, 4041, 3382, 4362, 874, 5803, 7267, 661, 1170, 3777, 6987, 1434, 3950, 4973, 6706, 4183, 5213, 3113, 3521, 206, 6205, 7183, 2808, 67, 4511, 3066, 6253, 2866, 6498, 5433, 1974, 3295, 4947, 566, 4250, 768, 2091, 6067, 5301, 1549, 4660, 2017, 2212, 250, 3420, 4722, 1317, 4938, 1582, 2404, 5159, 2127, 6875, 3315, 4558, 1501, 2850, 2996, 5977, 919, 3828, 5807, 1817, 2545, 7323, 6215, 2328, 5579, 4398, 2713, 4534, 2161, 2465, 2856, 404, 1826, 3196, 1803, 5215, 5276, 2614, 389, 7057, 2712, 2740, 1604, 6647, 1598, 7346, 3893, 3737, 5086, 3312, 2132, 2580, 5916, 2906, 5087, 7345, 5091, 5769, 998, 794, 4714, 306, 3775, 2352, 2273, 3765, 2746, 4060, 3279, 4165, 1403, 5352, 5348, 4325, 4434, 6663, 4883, 1972, 1098, 5463, 48, 6281, 5572, 5300, 6827, 2481, 1420, 3181, 2133, 974, 4264, 1917, 2828, 3610, 844, 61, 627, 5032, 2357, 3335, 4193, 2572, 5477, 3240, 3125, 3057, 7112, 1015, 2879, 4065, 3749, 6022, 1601, 4090, 6115, 241, 388, 6314, 5742, 5942, 7347, 484, 3285, 3984, 3396, 4592, 2653, 3784, 6338, 5202, 3427, 4767, 5586, 7126, 5536, 7163, 5266, 2067, 3048, 3826, 2409, 5132, 1222, 2112, 3091, 5154, 5, 4903, 299, 3454, 3261, 1619, 4969, 2153, 772, 4933, 2657, 4377, 1246, 4694, 1635, 3461, 4109, 1656, 1587, 1808, 3568, 4806, 4271, 4756, 1338, 939, 2498, 2709, 722, 7240, 3477, 1062, 791, 3881, 2004, 6243, 5330, 5643, 4108, 3309, 1527, 5131, 1566, 6367, 5417, 3957, 4284, 3136, 4406, 3822, 6176, 108, 5373, 4764, 5846, 2985, 1011, 4732, 3638, 3296, 6398, 2293, 5587, 248, 5590, 880, 935, 218, 7344, 1551, 2912, 5944, 4907, 1620, 7300, 7138, 7008, 3859, 5524, 222, 7307, 371, 2242, 2297, 6491, 4693, 6356, 3216, 6344, 3159, 2298, 3209, 6424, 4045, 3588, 432, 5618, 5387, 2044, 1225, 2720, 7033, 2942, 5955, 3806, 1085, 6352, 3098, 2542, 5370, 4965, 5510, 5422, 4591, 2696, 1209, 6729, 1245, 3982, 491, 4674, 4105, 1168, 6305, 3725, 1395, 6445, 4793, 3032, 1855, 3983, 2336, 200, 1792, 6163, 6247, 4716, 6507, 1956, 1998, 3387, 3720, 456, 5929, 5476, 6064, 1797, 2140, 6705, 385, 624, 3861, 7301, 575, 7006, 6366, 6484, 5110, 1960, 6098, 857, 4186, 2878, 6048, 4576, 6389, 5664, 4726, 7079, 3225, 2902, 451, 3629, 890, 5372, 6225, 2119, 3928, 86, 7285, 5391, 6342, 781, 410, 3290, 5919, 4620, 4557, 1060, 5167, 4864, 1961, 1299, 3030, 5717, 7376, 5493, 630, 5208, 5732, 4092, 3481, 4074, 2531, 4138, 1695, 6076, 4493, 2367, 4436, 6727, 4623, 2660, 307, 646, 6295, 6500, 172, 4961, 2925, 2290, 3190, 3440, 1508, 583, 6122, 6900, 3028, 7103, 2911, 2190, 4843, 5899, 5279, 2562, 4868, 3292, 5018, 5923, 6055, 5013, 6869, 6070, 1058, 5147, 6570, 747, 251, 660, 5318, 4223, 3310, 760, 6816, 2136, 1605, 2826, 5402, 5385, 1261, 7031, 1316, 5700, 5080, 616, 6229, 2858, 6762, 5480, 408, 6056, 854, 2750, 4526, 1669, 2974, 4259, 2402, 5537, 5981, 315, 2575, 6085, 3062, 5252, 2050, 6058, 358, 2718, 6266, 4495, 72, 6948, 88, 3195, 1335, 4529, 5679, 1210, 1119, 2907, 6830, 6494, 2898, 1845, 6709, 2343, 3569, 2857, 6944, 4616, 6264, 5069, 6616, 5386, 5102, 3274, 5515, 4974, 1990, 6712, 3128, 947, 3242, 1769, 2684, 5419, 5022, 1163, 2010, 3683, 6473, 2737, 3305, 5810, 4910, 4174, 6189, 4916, 2954, 3996, 1242, 1850, 2699, 1191, 3731, 5647, 6280, 4504, 4645, 257, 6923, 5240, 4734, 6409, 2525, 4255, 6147, 6438, 1142, 4970, 6717, 3746, 3533, 6598, 4859, 4023, 2014, 2830, 5136, 161, 4835, 2334, 4357, 3029, 5690, 1022, 5696, 572, 7189, 1529, 6711, 6063, 6543, 2847, 6039, 1768, 5133, 753, 4206, 5400, 6436, 6994, 5343, 1412, 1881, 5028, 4477, 1192, 6242, 5304, 6382, 6443, 5672, 1992, 5223, 7341, 437, 1172, 1035, 464, 5657, 5743, 6459, 862, 2086, 2216, 2577, 466, 6013, 3379, 4691, 2316, 5707, 3878, 5704, 3258, 1249, 1364, 3277, 3230, 2374, 1059, 4082, 1171, 5593, 5681, 3648, 6848, 3004, 6886, 5393, 4601, 814, 3095, 1398, 2623, 4799, 3612, 4670, 4684, 6545, 7382, 4025, 1268, 6775, 3698, 4669, 6340, 5752, 1905, 697, 1141, 411, 6834, 1698, 3686, 4236, 4876, 2470, 3766, 2543, 1497, 6349, 3133, 3166, 3681, 4421, 1922, 439, 5482, 1443, 4222, 3145, 3025, 6037, 3043, 3176, 6639, 1818, 6876, 5873, 4807, 4016, 5074, 7361, 5719, 3380, 7122, 459, 6214, 1865, 4473, 1066, 1332, 2928, 4329, 5685, 5949, 1986, 3696, 7367, 3993, 6770, 6385, 3935, 6585, 2629, 2392, 779, 4954, 4662, 197, 2981, 3124, 861, 4488, 7007, 2863, 4494, 1137, 2383, 4263, 6871, 11, 1165, 262, 7405, 5568, 1026, 4836, 3542, 3226, 3466, 5311, 2918, 150, 4905, 910, 5366, 4365, 3936, 3435, 7021, 3977, 5628, 686, 749, 4720, 5196, 6478, 7210, 4460, 1463, 6791, 3867, 6789, 1472, 5786, 4459, 1480, 6967, 2591, 5969, 7249, 3358, 963, 230, 6885, 1535, 3331, 3899, 6483, 6282, 3359, 3072, 3750, 5160, 2941, 7030, 3458, 4878, 3815, 2869, 2209, 5062, 6057, 5145, 734, 7139, 5577, 3110, 4067, 4665, 5335, 3220, 5341, 3459, 2469, 2463, 2513, 4535, 3146, 852, 5747, 6989, 2473, 181, 3015, 4036, 674, 7396, 2445, 506, 6156, 4000, 7381, 4408, 3662, 2581, 5218, 6460, 4750, 4830, 1955, 656, 920, 40, 6124, 1124, 93, 2073, 3361, 3079, 1307, 4682, 3829, 1302, 6691, 7286, 1892, 2730, 4519, 5095, 5015, 4673, 3442, 4470, 7214, 6240, 3115, 337, 7332, 7130, 3520, 1691, 4932, 4619, 7151, 6692, 7365, 5585, 2077, 3304, 4142, 2776, 5990, 5222, 614, 3510, 6520, 1265, 5126, 3774, 3401, 6119, 5519, 1241, 5418, 3070, 4210, 2180, 5326, 3965, 2739, 7295, 17, 1131, 3397, 3418, 2265, 3485, 4004, 22, 1801, 441, 3654, 2151, 6764, 1061, 1859, 4689, 6685, 5601, 1116, 4711, 6870, 4497, 2671, 1164, 1075, 5780, 7299, 4936, 2215, 7192, 6455, 5498, 1953, 1609, 333, 1969, 6670, 223, 6440, 352, 4126, 6515, 5845, 7369, 3040, 1644, 1473, 1206, 3644, 6638, 6221, 4242, 6808, 6840, 793, 4216, 6586, 7275, 3888, 4528, 5939, 6814, 5683, 6230, 5360, 4502, 7333, 7379, 5010, 4124, 2341, 3287, 3035, 1968, 3896, 1588, 921, 6881, 4544, 6038, 1662, 1005, 7187, 4522, 2592, 3479, 1157, 6380, 4812, 5702, 6499, 472, 5794, 3942, 2656, 4381, 3493, 7238, 3969, 7212, 2322, 6237, 4824, 6358, 6918, 7127, 4483, 5689, 4773, 1468, 3551, 658, 2299, 6960, 2728, 3755, 3925, 2973, 5179, 6089, 5375, 2482, 1189, 6937, 3456, 5037, 3508, 5170, 3471, 3739, 6862, 1599, 5547, 859, 6913, 3906, 3193, 4379, 5368, 5637, 5118, 5775, 6080, 978, 804, 653, 390, 4578, 6722, 2539, 2745, 5671, 6012, 4010, 5388, 217, 6595, 2082, 166, 4303, 6510, 1923, 1592, 5030, 6803, 458, 1204, 4704, 6569, 6763, 4033, 1051, 3705, 6961, 1097, 4789, 3183, 7384, 6300, 5233, 1081, 2698, 4736, 647, 1093, 4651, 6802, 76, 2559, 1439, 457, 1639, 6174, 2761, 469, 3843, 2683, 5660, 3912, 865, 2406, 127, 6294, 3530, 5344, 7244, 6096, 5051, 2440, 6268, 5109, 2339, 1711, 826, 6819, 1674, 6428, 3010, 2193, 4499, 6511, 4465, 2038, 7197, 3785, 2919, 6226, 635, 1128, 4857, 6183, 2935, 6656, 3839, 2593, 3910, 4262, 6497, 4838, 7310, 2074, 4238, 1615, 2175, 6790, 1809, 6719, 4533, 2472, 4650, 116, 1489, 4353, 2563, 45, 6289, 444, 6847, 334, 4654, 1795, 678, 1727, 1182, 473, 4821, 5650, 1014, 2003, 2825, 4687, 6538, 6106, 2076, 6309, 1512, 1091, 4581, 6231, 6651, 2387, 3388, 3951, 4322, 1343, 3655, 4902, 5479, 915, 7026, 6548, 5494, 6841, 6771, 7004, 836, 6422, 3640, 2588, 6513, 5823, 1352, 2989, 5684, 4059, 940, 1325, 590, 2236, 4178, 3140, 3954, 1102, 1377, 2520, 2100, 813, 6695, 7109, 2462, 6026, 6603, 3241, 6353, 4719, 6613, 1655, 3646, 2446, 3439, 1952, 3706, 5089, 1981, 2135, 7093, 3467, 6990, 1678, 5835, 5693, 2476, 4018, 6698, 659, 5103, 625, 4479, 1637, 5019, 858, 1856, 3932, 1010, 1841, 560, 4520, 3330, 5081, 5412, 2754, 4829, 515, 4995, 6544, 421, 4162, 1319, 4598, 5520, 2810, 5640, 2551, 6629, 7017, 6209, 1073, 2771, 5380, 2159, 1703, 6541, 4389, 2883, 190, 1964, 4194, 5275, 2717, 2514, 4943, 1618, 1161, 3631, 162, 4879, 6872, 621, 4703, 152, 1530, 4869, 5934, 1290, 3081, 3680, 4125, 2261, 3974, 4084, 7086, 5975, 2885, 4086, 3773, 1750, 3511, 4052, 496, 6821, 4643, 6336, 1741, 1470, 5122, 3850, 2853, 4698, 210, 1342, 956, 5404, 2179, 3045, 4350, 4392, 2778, 2110, 2842, 2916, 350, 746, 606, 1254, 3369, 1575, 4596, 1555, 79, 1561, 207, 100, 6003, 6952, 3013, 101, 6820, 1513, 2134, 573, 670, 6592, 6476, 6386, 4072, 5331, 5609, 777, 7199, 7293, 739, 5992, 2873, 1844, 4808, 4825, 7131, 529, 6040, 4371, 1858, 3647, 2605, 3821, 6843, 4359, 988, 3637, 1220, 320, 7082, 723, 4384, 375, 601, 4184, 5195, 4005, 2272, 7401, 6537, 3673, 644, 1659, 1926, 279, 5286, 290, 7204, 4345, 2378, 3468, 6286, 1819, 4104, 277, 2693, 1396, 6986, 6812, 4991, 66, 3375, 169, 3661, 1275, 6746, 6095, 2640, 2200, 3937, 4331, 154, 6381, 1485, 605, 2511, 6974, 7195, 742, 4826, 1079, 3619, 3472, 3224, 1649, 1872, 4890, 5615, 6606, 6100, 5971, 780, 641, 7058, 788, 6284, 5079, 5115, 1641, 1572, 2998, 2652, 6257, 5426, 2063, 525, 2053, 5447, 6406, 3163, 6212, 5197, 2795, 3671, 475, 695, 6187, 3795, 34, 5127, 4237, 4311, 5441, 6857, 1400, 519, 2435, 4081, 4011, 3718, 7325, 1520, 4610, 3068, 403, 1413, 1693, 4294, 2166, 6728, 4153, 7397, 5235, 4471, 4562, 4594, 6254, 4240, 7137, 7024, 1504, 1829, 5965, 2725, 1173, 2759, 3407, 929, 7219, 294, 714, 6168, 7065, 4253, 4795, 2043, 6311, 6898, 5635, 2356, 3053, 4456, 4940, 555, 1570, 4709, 822, 6220, 5582, 3034, 5505, 6496, 2416, 7156, 7162, 2459, 6364, 7308, 1132, 4809, 2021, 499, 1145, 5974, 128, 5190, 5359, 1934, 6341, 831, 1515, 5994, 3563, 782, 4100, 6315, 4404, 4780, 2743, 4200, 3876, 287, 4110, 1438, 6836, 6152, 3283, 4971, 1811, 6679, 6654, 6733, 688, 7386, 7144, 271, 6915, 6531, 6891, 2427, 4432, 2401, 3227, 2644, 5940, 4955, 2529, 1846, 4948, 6433, 3167, 6672, 2678, 433, 3031, 6560, 2317, 5425, 6469, 4391, 4941, 1216, 4354, 5096, 5998, 3168, 6374, 4777, 2806, 2479, 4723, 1067, 7145, 1788, 7088, 3469, 1563, 7268, 1807, 4844, 6807, 2715, 2379, 4642, 4003, 5040, 7154, 3376, 2221, 5687, 3275, 2275, 5979, 7147, 2256, 4195, 5303, 4803, 4438, 2616, 5589, 247, 7303, 1158, 3786, 1003, 5698, 4044, 2270, 4441, 6600, 7083, 4982, 3801, 5377, 5070, 4177, 6508, 4964, 5530, 1942, 2851, 1294, 681, 2191, 1701, 3463, 3024, 3130, 5535, 4979, 1634, 3300, 1882, 3642, 3889, 5554, 6312, 5255, 4261, 3803, 4299, 199, 3194, 4417, 6894, 1330, 129, 5705, 5847, 809, 2192, 5274, 5052, 6863, 610, 2412, 5921, 5710, 1848, 4463, 2872, 4077, 5770, 2937, 5319, 4017, 5299, 6852, 5353, 868, 2335, 800, 448, 2277, 231, 4204, 5204, 1919, 5533, 6643, 1479, 3968, 3676, 5021, 2794, 6632, 6016, 4229, 318, 1935, 5644, 6288, 594, 7169, 242, 513, 4759, 4382, 240, 438, 1344, 362, 7264, 4572, 3573, 3757, 3293, 7180, 2318, 3572, 4474, 1574, 5573, 925, 3913, 3980, 1260, 1496, 1789, 6238, 2612, 3787, 1477, 3732, 1813, 239, 1078, 1447, 2909, 6829, 2552, 1580, 3895, 983, 6239, 1962, 3724, 737, 6687, 5424, 5802, 5146, 6566, 3412, 254, 848, 1267, 4584, 281, 6260, 3320, 505, 487, 5098, 3933, 396, 1885, 7001, 6062, 6903, 7349, 1127, 3319, 6955, 6015, 2398, 7385, 1671, 6198, 3430, 3333, 3550, 776, 4579, 6144, 189, 4423, 4798, 3505, 335, 3525, 7282, 2583, 6549, 4999, 3289, 4452, 3221, 1049, 7097, 2447, 5914, 5337, 4613, 4516, 2467, 2423, 5491, 584, 2160, 4713, 1303, 3890, 764, 6747, 37, 2194, 7013, 1350, 4328, 440, 981, 2637, 3480, 3779, 2875, 6050, 5960, 7289, 4985, 2573, 3431, 5478, 1328, 6674, 2503, 5925, 2208, 5280, 4530, 1001, 288, 7321, 5866, 4070, 7091, 2059, 4043, 1160, 3405, 1643, 3507, 7236, 837, 2904, 6101, 6580, 2960, 1140, 2116, 5008, 973, 6319, 4753, 1227, 7136, 6387, 6650, 1675, 1301, 4841, 3208, 2103, 366, 6181, 6071, 2485, 7247, 5538, 987, 3685, 5357, 2302, 5777, 4858, 2706, 2855, 4820, 3017, 6735, 3060, 427, 1, 1709, 3269, 2537, 2838, 3204, 5638, 1779, 1229, 949, 2411, 4904, 3174, 2494, 3938, 1431, 4143, 1130, 544, 3523, 7366, 5697, 6755, 3055, 5829, 6942, 7251, 2620, 6529, 3103, 4154, 3693, 2630, 5534, 360, 4757, 107, 645, 3735, 4107, 1308, 1737, 3834, 628, 4871, 3214, 6437, 3449, 135, 380, 7263, 4298, 5597, 4667, 3536, 6970, 2672, 3244, 328, 6185, 6219, 684, 1323, 1312, 6783, 6777, 1110, 4101, 2066, 4390, 6546, 1971, 2031, 6530, 6335, 2751, 4445, 1341, 5175, 6968, 2096, 2323, 3203, 2617, 4542, 6159, 2115, 5438, 5728, 118, 2636, 3623, 6034, 6227, 4120, 5559, 6752, 1101, 5768, 4121, 6865, 2997, 3122, 3019, 2300, 7393, 1831, 5550, 237, 1370, 5334, 2860, 5174, 2284, 2315, 1607, 1488, 7100, 4561, 4224, 4567, 3410, 5230, 4151, 4122, 4872, 4788, 3090, 1459, 3599, 5381, 3726, 5322, 24, 1521, 3827, 322, 5871, 3738, 2361, 1004, 1567, 1948, 6565, 5455, 4632, 6410, 709, 2366, 6605, 2609, 4430, 1718, 3280, 882, 4765, 5911, 1321, 5485, 975, 3478, 2098, 6633, 6743, 5937, 2264, 7067, 4064, 1347, 4048, 1799, 1320, 736, 3715, 5099, 1895, 3799, 2748, 313, 866, 4306, 2813, 2865, 175, 4356, 834, 6103, 1584, 5140, 1068, 3246, 3372, 7135, 2310, 4171, 1724, 2861, 6581, 6113, 6522, 4212, 5551, 6786, 5870, 3641, 3723, 570, 3976, 1166, 2561, 6642, 883, 3837, 2815, 228, 7167, 1024, 43, 3995, 5642, 3332, 5878, 4188, 4917, 5135, 89, 2950, 325, 6778, 3606, 5437, 1345, 7277, 4742, 7253, 889, 1614, 6430, 6082, 2882, 4127, 1269, 5746, 7371, 4786, 6415, 5403, 1363, 3135, 60, 3862, 4506, 2474, 482, 1055, 1243, 3682, 4666, 244, 7259, 3700, 6072, 4071, 163, 2897, 2608, 4267, 6452, 2780, 1147, 6996, 1187, 2173, 4213, 6059, 2118, 1211, 5509, 226, 3807, 2567, 2399, 2497, 4455, 7043, 1507, 5765, 5031, 7125, 5713, 6402, 1996, 2281, 3571, 6532, 4555, 4457, 7232, 1071, 4966, 6298, 917, 5782, 1181, 618, 6359, 5886, 5158, 7089, 7157, 5113, 1481, 1898, 3132, 6195, 4931, 582, 4256, 291, 7186, 5927, 3282, 6018, 3730, 6901, 1450, 376, 5504, 1712, 6912, 2516, 7149, 4595, 3038, 3424, 1899, 1217, 1683, 4373, 3016, 5656, 1957, 4273, 7035, 1836, 7020, 6054, 5532, 6114, 2029, 2719, 1663, 2146, 112, 3926, 6853, 42, 4692, 5976, 1266, 574, 4446, 839, 489, 3545, 3694, 1486, 6716, 413, 2477, 4512, 7052, 2966, 2686, 3916, 5832, 5149, 6945, 5367, 5187, 3744, 4817, 1970, 1235, 2475, 3352, 6991, 7255, 6334, 5487, 1208, 3344, 3509, 6590, 2604, 5308, 2403, 2167, 664, 6749, 3206, 3453, 1787, 1143, 1548, 959, 5256, 3416, 6736, 1523, 2982, 6045, 6196, 2177, 4397, 2554, 6556, 5973, 6140, 992, 564, 5121, 2389, 1099, 4385, 2023, 44, 3947, 253, 5543, 3635, 552, 7143, 483, 2659, 3714, 2155, 4211, 3364, 899, 4370, 4755, 1198, 2785, 4946, 1422, 3099, 3879, 4605, 74, 4837, 3595, 6905, 1936, 1306, 5806, 3771, 5457, 897, 4226, 2732, 5450, 5850, 5670, 787, 3506, 4393, 4387, 5405, 2519, 1064, 5843, 30, 3809, 2225, 6889, 1657, 6850, 4510, 3179, 4639, 6858, 4652, 2338, 4040, 4629, 1495, 5968, 7128, 1219, 4607, 7094, 2560, 3138, 6941, 4258, 6521, 2603, 1821, 75, 1251, 5852, 4683, 5389, 5724, 6668, 5120, 5029, 7252, 3866, 2818, 5733, 945, 6738, 2019, 4164, 2859, 2121, 6501, 5639, 6756, 4784, 3147, 6626, 6471, 2779, 3421, 5483, 7281, 7111, 1870, 1820, 4850, 7230, 3173, 7090, 4340, 5818, 2441, 693, 4814, 3197, 81, 4553, 7246, 4429, 2631, 6977, 4173, 518, 2633, 4140, 1281, 470, 2101, 7076, 853, 3232, 607, 2627, 2939, 6767, 2710, 3985, 1912, 7261, 5460, 2971, 6646, 4925, 28, 378, 2999, 4677, 4248, 5156, 2894, 343, 2137, 6826, 5811, 1612, 5064, 1665, 3656, 297, 2443, 342, 4469, 3575, 2541, 4778, 5948, 761, 252, 2061, 2781, 2844, 2170, 4956, 5592, 6320, 2920, 5411, 7191, 3341, 1729, 4647, 524, 3905, 5548, 6930, 6310, 4559, 636, 3350, 5517, 598, 6927, 7134, 694, 7045, 3923, 6710, 798, 418, 6666, 3883, 7102, 5042, 3556, 643, 5856, 1999, 3406, 392, 6276, 6487, 5809, 4244, 6078, 2823, 4920, 6006, 6667, 1541, 5165, 5999, 5430, 1074, 1409, 6973, 6609, 3257, 97, 6772, 7037, 4606, 3104, 3437, 6139, 4309, 1650, 3074, 1756, 4019, 1516, 2431, 4399, 5244, 1133, 2639, 6824, 946, 1713, 2821, 699, 6879, 5448, 6079, 4209, 4012, 6851, 3205, 4448, 1833, 2049, 1238, 7034, 2408, 923, 6737, 3558, 55, 5989, 6467, 4918, 2452, 1019, 4659, 78, 4134, 2417, 3870, 6431, 2845, 234, 355, 1800, 5993, 6208, 3701, 4597, 5265, 986, 4707, 752, 2424, 3153, 7276, 1289, 1384, 6781, 4747, 3626, 5907, 368, 1757, 2490, 3097, 6303, 2385, 5920, 7194, 2553, 1973, 2835, 3083, 1608, 5271, 1009, 7206, 6175, 3322, 2196, 2078, 3012, 4297, 4552, 6432, 202, 1057, 2460, 4518, 4268, 5527, 2598, 1863, 6669, 7326, 5894, 3769, 4543, 7359, 3818, 2625, 1716, 1759, 3943, 2544, 5669, 373, 85, 1950, 3294, 4257, 397, 3085, 3759, 696, 6833, 2867, 3307, 1647, 4015, 1804, 7051, 2934, 6061, 5659, 6330, 1528, 654, 4939, 4840, 5408, 209, 1978, 604, 4986, 979, 4797, 164, 5513, 705, 6516, 365, 6997, 2008, 1862, 5858, 4763, 1805, 6571, 1484, 7056, 5007, 2786, 5958, 5963, 1356, 5362, 4160, 6267, 2579, 7288, 6029, 997, 3452, 6279, 6296, 1843, 5114, 5512, 629, 4848, 3666, 6414, 215, 2597, 203, 3172, 6105, 3945, 5180, 6954, 5614, 4818, 2576, 6504, 1040, 2305, 2152, 4412, 5612, 3697, 4400, 429, 4155, 7063, 3792, 2557, 3823, 2138, 6693, 770, 5144, 142, 7085, 490, 3301, 4725, 3900, 4987, 3366, 1279, 6442, 4641, 6490, 6277, 6838, 2373, 1453, 3189, 2634, 3565, 3929, 1322, 1791, 476, 148, 5897, 6557, 140, 1146, 344, 828, 6956, 2881, 5556, 3615, 5184, 431, 4775, 5166, 5141, 7022, 1822, 1419, 2262, 1740, 7208, 1435, 1553, 4638, 7042, 7387, 2172, 2149, 3243, 6773, 2757, 4199, 272, 2895, 1891, 3094, 1457, 1349, 817, 3415, 5012, 6998, 2685, 2156, 2375, 468, 6925, 6322, 1454, 7363, 1076, 7198, 7032, 4517, 4646, 5641, 4089, 5467, 1687, 6378, 2518, 2788, 2228, 4737, 5889, 3411, 3535, 3788, 2797, 347, 6584, 5815, 6992, 1223, 1469, 4006, 5895, 3107, 6963, 4051, 6028, 5676, 3438, 2360, 4702, 6112, 4424, 80, 5059, 400, 6768, 1500, 2772, 4912, 5760, 1023, 7360, 3643, 2641, 2735, 1105, 6517, 2727, 4347, 3946, 5741, 7283, 3918, 3046, 4085, 198, 808, 6446, 258, 2908, 2150, 1943, 4133, 4080, 1309, 3514, 871, 4383, 4658, 356, 3849, 7107, 4326, 6780, 276, 2376, 1000, 2051, 1383, 6316, 7193, 6222, 5155, 1734, 3374, 4624, 5267, 4863, 5748, 2958, 5869, 6744, 5736, 5323, 2107, 2032, 2039, 173, 2729, 7011, 3740, 6964, 3051, 6766, 2764, 321, 3562, 435, 2486, 2517, 3009, 4621, 5804, 5066, 311, 4196, 3684, 2833, 3337, 5006, 5345, 3365, 6582, 3231, 1262, 7176, 3504, 3987, 3336, 3800, 5390, 2380, 16, 1866, 6929, 6882, 6470, 1638, 339, 2530, 2219, 4866, 4505, 1959, 568, 3920, 1212, 3065, 7010, 417, 5761, 5884, 1293, 4491, 6350, 2348, 4790, 6914, 31, 4881, 3753, 2060, 6860, 4131, 507, 4568, 2648, 4157, 144, 7000, 7298, 5545, 183, 640, 932, 7202, 1462, 3422, 5453, 1536, 184, 6482, 7265, 4845, 6792, 6203, 5268, 6094, 5701, 1777, 1976, 6132, 2643, 2250, 4118, 1752, 5611, 3948, 1854, 99, 123, 6796, 187, 4574, 270, 4324, 486, 698, 7227, 7280, 5567, 1794, 2253, 2048, 1305, 2977, 1668, 3254, 1946, 6020, 581, 2344, 4537, 5991, 3328, 3313, 2952, 6182, 2814, 3496, 3264, 2451, 2057, 5320, 2992, 2246, 5860, 5874, 5469, 2205, 2726, 4450, 958, 1274, 3470, 2991, 1287, 918, 1120, 6457, 6317, 842, 7257, 689, 1890, 3917, 346, 6093, 5251, 4323, 7358, 7129, 4310, 3211, 3990, 5772, 6683, 1492, 5153, 3185, 774, 4847, 5932, 2613, 447, 6393, 6299, 4710, 4656, 2433, 807, 4976, 845, 3891, 3991, 1282, 2430, 1236, 4439, 3960, 6250, 3218, 5905, 1311, 1177, 4712, 5338, 2886, 6739, 5442, 6466, 7216, 2691, 1012, 3956, 2507, 4539, 5036, 3144, 3708, 4227, 4873, 1534, 4661, 5865, 4571, 387, 3699, 1887, 6621, 3419, 6759, 4039, 5603, 666, 5214, 2570, 1354, 7355, 3164, 6395, 5726, 4731, 4513, 6721, 4978, 7071, 3071, 4653, 1853, 2600, 3291, 1624, 4313, 1830, 1082, 6765, 4888, 65, 962, 2564, 5831, 2747, 3874, 881, 7262, 1832, 3089, 3455, 2556, 835, 7165, 4897, 1037, 7201, 1379, 2027, 1525, 2584, 2011, 391, 4668, 1232, 4135, 3111, 4058, 1938, 3853, 1902, 3919, 6161, 1276, 4407, 1154, 7337, 7343, 556, 3903, 6583, 2756, 2201, 1467, 6201, 2611, 2711, 4093, 3363, 5150, 4251, 4096, 715, 2099, 5169, 3198, 467, 3593, 1231, 5867, 993, 1371, 126, 6659, 2755, 763, 5605, 4983, 1129, 6005, 520, 6657, 558, 1366, 2414, 1122, 4688, 4967, 1255, 4321, 896, 6091, 4536, 5238, 3262, 4116, 6046, 1092, 7292, 2104, 4637, 5078, 7383, 904, 3329, 613, 4, 6365, 4588, 6401, 324, 2130, 5514, 1277, 2331, 2330, 10, 5816, 3426, 7324, 2307, 5627, 719, 3169, 1623, 5711, 634, 6384, 2320, 6951, 286, 5675, 1490, 5212, 3632, 3129, 3392, 7271, 5565, 2527, 2163, 1773, 2368, 6485, 7046, 5056, 5619, 7168, 6133, 5930, 1728, 4175, 2111, 4098, 6244, 1300, 478, 1030, 4951, 3999, 5888, 6959, 3281, 3303, 3992, 5674, 370, 1375, 156, 2701, 5484, 5912, 4148, 280, 1875, 3327, 6493, 5580, 5588, 1441, 70, 3020, 5632, 5358, 141, 1483, 3338, 5987, 901, 4066, 796, 2921, 219, 3958, 1378, 3582, 7340, 1327, 2291, 7070, 1203, 7164, 2647, 5210, 3316, 6842, 6837, 3953, 5284, 1224, 5578, 1576, 4877, 875, 4147, 3114, 2069, 2214, 1278, 5439, 5143, 2877, 296, 7146, 1056, 5610, 5730, 2294, 2089, 3528, 5020, 2932, 1648, 6369, 6278, 292, 1810, 948, 5247, 2635, 2965, 3343, 5518, 7374, 806, 137, 3235, 5281, 5549, 1112, 6420, 3058, 71, 5246, 301, 7171, 3767, 6726, 1884, 2432, 1474, 5205, 5688, 7291, 2394, 7221, 5507, 1654, 2995, 6883, 3711, 302, 6715, 4924, 7016, 69, 6878, 6441, 2271, 5216, 2484, 4894, 4548, 4508, 5857, 493, 9, 1407, 1904, 6360, 2708, 1436, 6274, 548, 3433, 6506, 1498, 712, 4563, 2304, 494, 1533, 5980, 149, 7217, 1283, 1591, 3832, 2120, 4882, 2951, 2207, 5771, 2274, 261, 3975, 1239, 5117, 4352, 7318, 4008, 4292, 7403, 8, 900, 1827, 637, 6351, 4351, 6011, 843, 1593, 4467, 5706, 2980, 4760, 51, 1362, 5445, 3835, 5745, 4394, 7243, 1747, 6725, 4728, 1043, 7370, 4458, 2033, 5935, 6417, 4708, 2976, 6472, 4672, 533, 6810, 201, 7110, 5909, 4339, 5621, 4541, 5005, 4540, 57, 2047, 7190, 639, 6042, 2839, 6199, 3321, 5401, 1755, 5652, 4546, 1916, 668, 3501, 5634, 3360, 1690, 5658, 3820, 2283, 2924, 136, 7287, 1048, 3346, 5435, 864, 3465, 3690, 6439, 4235, 6331, 2407, 1374, 1117, 5369, 2666, 5379, 1879, 622, 3318, 3444, 5947, 4279, 6880, 5365, 1031, 4796, 379, 6652, 412, 1421, 690, 4252, 6552, 1949, 7272, 1672, 7225, 6170, 5834, 2654, 877, 3675, 942, 341, 3971, 884, 3851, 3952, 2506, 245, 710, 1907, 3782, 7158, 6259, 585, 1391, 2243, 994, 3871, 3498, 5526, 2064, 703, 1184, 5092, 3142, 1738, 6210, 1540, 4334, 2722, 748, 2868, 5055, 4699, 886, 275, 3270, 5249, 5189, 701, 4887, 3621, 5125, 4919, 6313, 4332, 2128, 3949, 7141, 5952, 5575, 510, 4013, 2789, 1596, 5996, 600, 3892, 5310, 2532, 2571, 2353, 3781, 1089, 158, 5061, 6044, 305, 7348, 3184, 6458, 6573, 1373, 284, 3783, 4285, 6593, 950, 103, 7009, 4201, 6456, 789, 1248, 1651, 4855, 576, 829, 5883, 6084, 3050, 7224, 4587, 1221, 5282, 7113, 1782, 3393, 2285, 5737, 6822, 6192, 6950, 6972, 5727, 6383, 1679, 3151, 1867, 2125, 369, 2410, 2456, 4228, 620, 5406, 5297, 849, 3842, 2325, 316, 3703, 145, 1351, 3557, 3222, 5191, 450, 6347, 6075, 3955, 3847, 2231, 1874, 5392, 5073, 1234, 6922, 6186, 4280, 1381, 110, 6677, 3112, 6723, 3047, 1834, 1404, 5501, 3308, 382, 7044, 2095, 5394, 364, 5954, 7084, 1838, 4428, 1670, 6653, 6509, 7178, 7048, 2599, 4609, 3448, 6190, 2975, 4802, 2041, 5333, 2267, 792, 5750, 982, 2899, 718, 1126, 6831, 5822, 3586, 898, 63, 1825, 2391, 943, 115, 4168, 5470, 7069, 5474, 534, 2333, 6658, 3526, 651, 5085, 3356, 7215, 2495, 2186, 1835, 3251, 3846, 3921, 2892, 1914, 4416, 2254, 2658, 561, 2124, 3758, 4335, 3149, 4152, 1304, 2731, 5003, 6983, 4975, 1927, 4401, 7015, 1200, 2929, 6009, 3736, 4433, 6576, 6612, 1053, 4315, 7120, 6980, 4316, 2006, 1357, 1358, 4706, 3381, 5926, 5382, 4585, 5546, 1852, 1368, 2210, 6635, 1708, 6962, 2040, 6649, 5217, 2927, 1029, 3574, 1622, 2426, 5287, 151, 2238, 6745, 5751, 3234, 3741, 4472, 3432, 6748, 1410, 6325, 3245, 1762, 4276, 2585, 5123, 2015, 597, 2742, 1095, 4422, 345, 6760, 2439, 1205, 409, 6751, 1034, 1361, 1159, 205, 1932, 5599, 593, 340, 1557, 1094, 2070, 6572, 1313, 3963, 6623, 3761, 7234, 5468, 4655, 1096, 3797, 3601, 15, 4589, 5124, 2364, 1837, 903, 5378, 232, 2085, 4972, 2148, 7315, 7049, 6169, 6817, 876, 3669, 3814, 3848, 1924, 5788, 5492, 6223, 4187, 4239, 928, 1336, 5511, 6128, 5951, 3252, 6272, 1767, 1416, 1849, 4630, 6933, 114, 1310, 1706, 5054, 5778, 3255, 1569, 5721, 4898, 1560, 6400, 3600, 1590, 3988, 36, 3831, 4958, 1630, 3460, 893, 5830, 1731, 1903, 1688, 1440, 1046, 2087, 1476, 4676, 3059, 6553, 2534, 1786, 5058, 4906, 5112, 4865, 5814, 2237, 7273, 6453, 2455, 1928, 3223, 1017, 3722, 2296, 4856, 4862, 4679, 3567, 59, 2926, 4525, 2667, 2268, 5049, 1193, 0, 4810, 5399, 191, 7390, 5791, 1920, 5598, 5466, 7248, 2217, 1445, 6648, 1597, 2831, 811, 7087, 4180, 1559, 177, 5508, 5694, 4290, 3414, 4569, 5290, 4816, 5516, 4538, 2558, 5906, 5840, 3342, 3652, 4386, 4733, 3665, 3566, 2979, 1244, 5529, 25, 1432, 1215, 905, 2628, 3811, 5808, 6551, 6701, 386, 4166, 5796, 5495, 2260, 6167, 5072, 1720, 1577, 6148, 402, 6568, 1388, 6559, 2213, 4409, 3620, 4182, 711, 2782, 2288, 546, 1913, 5347, 5152, 6193, 4042, 3106, 5584, 3018, 3868, 5016, 6465, 4545, 847, 3120, 5636, 912, 1123, 2083, 4202, 5677, 6126, 5879, 1393, 3546, 3351, 2075, 655, 6874, 6379, 4926, 676, 4170, 1020, 810, 2434, 5038, 3927, 6328, 3161, 511, 6624, 4231, 6118, 6025, 5277, 5383, 3796, 7038, 4029, 6416, 5859, 7375, 1633, 260, 1337, 5035, 5420, 2371, 5134, 2827, 4785, 6154, 4091, 2645, 6982, 1798, 6291, 6273, 1929, 6435, 6097, 4020, 5970, 4233, 1461, 3650, 1910, 308, 2602, 18, 2157, 4945, 626, 2178, 1252, 5332, 4301, 4139, 2816, 5314, 4582, 3229, 5983, 4468, 5254, 3302, 3207, 1471, 4405, 3201, 6434, 4103, 1493, 5653, 5986, 1894, 2419, 5882, 2042, 312, 4575, 5398, 2970, 6818, 7254, 1944, 2834, 3385, 2169, 6622, 131, 1699, 1174, 2590, 1176, 2055, 7053, 3524, 1207, 3306, 4475, 2345, 5738, 3628, 6421, 6574, 5606, 26, 4927, 3707, 4024, 5298, 6110, 1446, 4246, 3860, 3717, 633, 4819, 4915, 3543, 5631, 1021, 3634, 1169, 3941, 7059, 2733, 1107, 4054, 1339, 319, 170, 6086, 2550, 3486, 5825, 256, 1376, 195, 122, 6391, 6577, 1680, 6835, 5805, 1135, 6754, 2090, 1539, 4128, 1765, 6077, 3584, 208, 461, 7306, 4132, 2359, 751, 3092, 771, 4782, 2308, 7152, 6856, 909, 46, 113, 106, 4787, 6479, 7081, 675, 2651, 1456, 3605, 2321, 7019, 5228, 4681, 3776, 3944, 3260, 7328, 1878, 5188, 2905, 5854, 5801, 4026, 2758, 2963, 4361, 2677, 4241, 2753, 7014, 1707, 2903, 1385, 6909, 4028, 4705, 4437, 3087, 3856, 5731, 6664, 4414, 4849, 102, 6121, 5296, 4410, 5686, 3039, 5452, 4680, 4828, 6979, 3604, 632, 1088, 744, 3487, 638, 5813, 2422, 6408, 6188, 7266, 1761, 6854, 3592, 3362, 96, 1253, 4852, 972, 5130, 1103, 2884, 2533, 4625, 73, 3756, 58, 1681, 3143, 3537, 6828, 2734, 4695, 3679, 3349, 1886, 2429, 5490, 4102, 4249, 5185, 4675, 188, 6160, 7018, 3339, 5680, 3076, 7229, 3513, 2663, 2081, 6602, 3073, 4363, 6681, 2247, 5250, 477, 2018, 6137, 6938, 2239, 6000, 1180, 3297, 5339, 7320, 1965, 2195, 867, 6675, 6224, 2035, 5162, 6036, 372, 599, 6801, 4198, 6682, 4150, 4487, 4113, 5720, 6060, 1104, 6146, 4492, 3037, 1980, 6066, 631, 5931, 1348, 1153, 6844, 153, 4037, 5616, 1214, 5725, 2793, 2703, 452, 7394, 6594, 5350, 6143, 669, 5371, 7395, 1329, 4895, 6589, 5861, 5229, 1940, 4923, 2381, 673, 6645, 4867, 731, 5263, 2461, 2457, 6554, 5264, 6293, 5541, 221, 5231, 6150, 812, 1988, 4801, 5896, 3618, 3970, 5566, 5758, 4769, 7373, 1298, 462, 6191, 1475, 7316, 3370, 1077, 430, 1263, 2940, 5799, 4114, 5292, 1503, 6481, 3238, 3408, 3158, 3384, 6397, 4217, 7317, 4724, 4834, 4981, 331, 5938, 3188, 4934, 6171, 2664, 383, 821, 6130, 5464, 4094, 3727, 3425, 830, 2566, 6924, 6371, 267, 434, 4631, 6131, 5898, 2418, 5946, 5354, 5033, 497, 2736, 5716, 6007, 3540, 6935, 3961, 6563, 5307, 1148, 6092, 2449, 124, 132, 82, 1491, 4685, 860, 2065, 6301, 1696, 1611, 2766, 7077, 1052, 2568, 1448, 2141, 2252, 4129, 7028, 1732, 6966, 3733, 2421, 6794, 3402, 5496, 1621, 4327, 7182, 3217, 4804, 1896, 2502, 4225, 5077, 7335, 3798, 1499, 6083, 2528, 4087, 6345, 4464, 6292, 1032, 5964, 1723, 4875, 3180, 3157, 6731, 6218, 1430, 7096, 4633, 6053, 1915, 3317, 4608, 2601, 4628, 4287, 1136, 4207, 2388, 4205, 2910, 2324, 2749, 180, 171, 931, 784, 6542, 2619, 778, 672, 1562, 2187, 5241, 649, 554, 2312, 5892, 3005, 5253, 6614, 1257, 6423, 4411, 2056, 1125, 295, 6518, 1230, 3616, 1760, 2313, 4899, 3880, 3810, 2222, 3884, 3608, 6714, 4823, 2669, 3865, 1564, 665, 4644, 6628, 5107, 5962, 2915, 914, 5837, 5654, 5226, 4554, 740, 5910, 5795, 5740, 4990, 3789, 5784, 4403, 2158, 603, 7155, 1452, 955, 5101, 2478, 2084, 4141, 6081, 7312, 5289, 6464, 5651, 617, 1613, 5047, 6864, 5237, 47, 2523, 5374, 990, 4461, 3624, 4304, 2862, 168, 6640, 2555, 3845, 5757, 6157, 7330, 1256, 5718, 4761, 2241, 2784, 1086, 713, 1033, 2889, 514, 1908, 2171, 1494, 3502, 2384, 3476, 662, 536, 6263, 3997, 4743, 6702, 133, 952, 3931, 4913, 984, 1595, 4980, 5225, 4466, 7080, 6684, 6255, 4721, 5486, 4944, 2289, 2594, 5269, 39, 5397, 3192, 3716, 7025, 5844, 1386, 1218, 1876, 1144, 4745, 3515, 6477, 492, 3791, 4014, 4111, 7012, 6524, 1573, 7278, 4440, 1425, 6179, 6689, 6527, 4286, 3907, 1888, 5053, 3056, 2936, 3934, 2203, 1744, 2363, 4593, 2994, 6403, 2688, 6495, 549, 3395, 4739, 5798, 4774, 1714, 2489, 6180, 264, 4740, 2276, 6920, 5356, 6845, 769, 5773, 5523, 50, 6618, 6627, 6809, 6142, 4600, 3063, 657, 5827, 236, 5048, 6868, 4344, 5956, 6346, 1016, 840, 4192, 3872]

In [6]:
Shuffle1 = Shuffle1[:500]
Shuffle2 = Shuffle2[:500]
Shuffle3 = Shuffle3[:500]
Shuffle4 = Shuffle4[:500]
Shuffle5 = Shuffle5[:500]

In [3]:
import random

numbers = list(range(7406))
shuffles = []

# Shuffle 5 times and store each shuffle
for _ in range(5):
    random.shuffle(numbers)
    shuffles.append(numbers.copy())  # Store a copy of the shuffled list

# Print each shuffle
for i, shuffle in enumerate(shuffles):
    print(f"Shuffle {i + 1}: {shuffle}")


Shuffle 1: [6417, 360, 3435, 1563, 3232, 2769, 3692, 2189, 2223, 3883, 3745, 2539, 6361, 6809, 5154, 4187, 349, 6013, 1292, 6210, 3647, 3033, 2105, 3041, 57, 5447, 4988, 7149, 6844, 630, 7195, 913, 591, 6388, 2613, 314, 3930, 5259, 3079, 5370, 2929, 4790, 5316, 655, 3440, 4838, 2761, 2824, 477, 7346, 2258, 169, 5092, 3543, 6331, 1480, 3601, 6100, 5669, 800, 6355, 3783, 855, 4406, 1356, 7102, 4911, 5104, 1822, 5706, 2276, 5428, 2952, 4484, 6655, 2771, 160, 4461, 5783, 1857, 4821, 1436, 216, 78, 3021, 3123, 2200, 4800, 5513, 6442, 5654, 1991, 4177, 5492, 1847, 6123, 4172, 1252, 775, 2898, 4723, 5898, 6088, 1966, 5767, 4624, 6692, 2401, 721, 993, 556, 6623, 1686, 2391, 5060, 5063, 2577, 2979, 653, 3315, 2940, 592, 1135, 5334, 4194, 982, 1832, 1376, 6399, 65, 3942, 3983, 437, 415, 539, 6284, 2284, 3182, 6670, 4230, 4128, 3650, 6403, 4456, 5954, 7383, 3369, 6124, 3995, 5351, 2297, 6749, 4228, 5343, 7358, 3689, 2368, 1378, 3387, 5621, 4274, 787, 1501, 627, 4900, 6591, 3718, 4896, 1434, 6204,

In [ ]:
from agential.eval.metrics.classification import EM, f1, precision, recall


method = CoT(
    llm=llm,
    benchmark="hotpotqa",
)

samples = 500
num_correct = 0
experiment  = [Shuffle1 , Shuffle2 , Shuffle3 , Shuffle4 , Shuffle5]




for i in range(num_experiment):
    for j in range()
for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]

    out = method.generate(
        question=question,
        key=answer,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT, 
        prompt=COT_INSTRUCTION_HOTPOTQA,
        additional_keys={},
        num_retries=1,
        warming=[None],
    )
    is_correct = EM(out.answer, answer)
    print(question)
    print(answer, "\t\t", out.answer, "\t\t", is_correct, end="\n\n")

    num_correct += int(is_correct)

    precision_out = precision(out.answer , answer , True)

    recall_out = recall(out.answer , answer , True)

    f1_out = f1(out.answer , answer , True)
    
print(f"{num_correct}/{samples}")